# Network Science Project

## Initizaliation

### Importing libraries

In [3]:
import networkx as nx #network analysis
import numpy as np 
import matplotlib.pyplot as plt #viz
import random 
import pandas as pd #to manipulate data frames
import math #for logarithm in entropy formula
from IPython.display import display
import os #files manipulation
import time #to measure time

#################################################################
#if you're working on google colab, uncomment the following lines

# from google.colab import drive
# drive.mount('/content/drive')
# inp_path=input('enter ur gdrive folder path:')
# os.chdir(inp_path)
# os.getwcd()
#################################################################

### Loading data

In [4]:
arenas_email = nx.read_gml('../benchmark/arenas-email.gml', label='id')
bn_cat_mixed_species_brain_1 = nx.read_gml('../benchmark/bn-cat-mixed-species_brain_1.gml', label='id')
bn_macaque_rhesus_brain_2 = nx.read_gml('../benchmark/bn-macaque-rhesus_brain_2.gml', label='id')
circuits_s208 = nx.read_gml('../benchmark/circuits s208.gml', label='id')
circuits_s420 = nx.read_gml('../benchmark/circuits s420.gml', label='id')
circuits_s838 = nx.read_gml('../benchmark/circuits s838.gml', label='id')
e_coli = nx.read_gml('../benchmark/E. coli.gml', label='id')
facebook_0 = nx.read_gml('../benchmark/facebook 0.gml', label='id')
facebook_107 = nx.read_gml('../benchmark/facebook 107.gml', label='id')
facebook_1684 = nx.read_gml('../benchmark/facebook 1684.gml', label='id')
facebook_348 = nx.read_gml('../benchmark/facebook 348.gml', label='id')
facebook_414 = nx.read_gml('../benchmark/facebook 414.gml', label='id')
facebook_686 = nx.read_gml('../benchmark/facebook 686.gml', label='id')
polbooks = nx.read_gml('../benchmark/polbooks.gml', label='id')
word_adjacencies = nx.read_gml('../benchmark/word_adjacencies.gml', label='id')
corruption=nx.read_gml('../benchmark/corruption.gml', label='id')
crime_net=nx.read_gml('../benchmark/CrimeNet.gml', label='id')
email=nx.read_gml('../benchmark/Email.gml', label='id')
jazz=nx.read_gml('../benchmark/Jazz.gml', label='id')
petster_hamster=nx.read_gml('../benchmark/Petster-Hamster.gml', label='id')




nets = [bn_macaque_rhesus_brain_2, word_adjacencies, circuits_s208, bn_cat_mixed_species_brain_1, facebook_414, facebook_686, polbooks, jazz, facebook_348, circuits_s420, corruption, facebook_0, circuits_s838, crime_net, facebook_1684]
net_names = ["bn_macaque_rhesus_brain_2", "word_adjacencies", "circuits_s208", "bn_cat_mixed_species_brain_1", "facebook_414", "facebook_686", "polbooks", "jazz", "facebook_348", "circuits_s420", "corruption", "facebook_0", "circuits_s838", "crime_net", "facebook_1684"]

In [ ]:
#get stats for each network (table 1)

def compute_network_stats(G, network_name):
    largest_cc = max(nx.connected_components(G), key=len)
    largest_cc_subgraph = G.subgraph(largest_cc)
    stats = {
        "Network": network_name,
        "|N|": G.number_of_nodes(),
        "|V|": G.number_of_edges(),
        "Density": nx.density(G),
        "<k>": np.mean(list(dict(G.degree()).values())),
        "<cc>": nx.average_clustering(G),
        "<spath>": nx.average_shortest_path_length(largest_cc_subgraph),
        "d": nx.diameter(largest_cc_subgraph)
    }
    
    return stats

arenas_email_stats = compute_network_stats(arenas_email, 'arenas_email')
bn_cat_mixed_species_brain_1_stats = compute_network_stats(bn_cat_mixed_species_brain_1, 'bn_cat_mixed_species_brain_1')
bn_macaque_rhesus_brain_2_stats = compute_network_stats(bn_macaque_rhesus_brain_2, 'bn_macaque_rhesus_brain_2') 
circuits_s208_stats = compute_network_stats(circuits_s208, 'circuits_s208')
circuits_s420_stats = compute_network_stats(circuits_s420, 'circuits_s420')
circuits_s838_stats = compute_network_stats(circuits_s838, 'circuits_s838')
# dolphins_stats = compute_network_stats(dolphins, 'dolphins')
e_coli_stats = compute_network_stats(e_coli, 'e_coli')
facebook_0_stats = compute_network_stats(facebook_0, 'facebook_0')
facebook_107_stats = compute_network_stats(facebook_107, 'facebook_107')
facebook_1684_stats = compute_network_stats(facebook_1684, 'facebook_1684')
facebook_348_stats = compute_network_stats(facebook_348, 'facebook_348')
facebook_414_stats = compute_network_stats(facebook_414, 'facebook_414')
facebook_686_stats = compute_network_stats(facebook_686, 'facebook_686')
# fb_pages_food_stats = compute_network_stats(fb_pages_food, 'fb_pages_food')
# karate_stats = compute_network_stats(karate, 'karate')
polbooks_stats = compute_network_stats(polbooks, 'polbooks')
# soc_firm_hi_tech_stats = compute_network_stats(soc_firm_hi_tech, 'soc_firm_hi_tech')
# soc_tribes_stats = compute_network_stats(soc_tribes, 'soc_tribes')
word_adjacencies_stats = compute_network_stats(word_adjacencies, 'word_adjacencies')
# net_911_stats = compute_network_stats(net_911, 'net_911')
corruption_stats = compute_network_stats(corruption, 'corruption')
crime_net_stats = compute_network_stats(crime_net, 'crime_net')
# digg_stats = compute_network_stats(digg, 'digg')
email_stats = compute_network_stats(email, 'email')
jazz_stats = compute_network_stats(jazz, 'jazz')
petster_hamster_stats = compute_network_stats(petster_hamster, 'petster_hamster')
# router_stats = compute_network_stats(router, 'router')

graph_stats = [arenas_email_stats, bn_cat_mixed_species_brain_1_stats, bn_macaque_rhesus_brain_2_stats, circuits_s208_stats, circuits_s420_stats, circuits_s838_stats, e_coli_stats, facebook_0_stats, facebook_107_stats, facebook_1684_stats, facebook_348_stats, facebook_414_stats, facebook_686_stats, polbooks_stats, word_adjacencies_stats, corruption_stats, crime_net_stats, email_stats, jazz_stats, petster_hamster_stats]
graph_stats = pd.DataFrame(graph_stats)
graph_stats.to_csv('../results/graph_stats.csv')
# display(graph_stats)

## Functions:

### Metrics:

In [5]:
def compute_lcc(G):
    '''takes a graph and returns the size of the largest connected component
    
    parameters:
    G: a graph
    
    returns:
    the size of the largest connected component
    '''
    if G.number_of_nodes() == 0:
        return 0
    else:
        max_comp= max(nx.connected_components(G), key=len)
        return len(max_comp)

def compute_efficiency(G):
    '''takes a graph and returns the efficiency of the graph
    
    parameters:
    G: a graph
    
    returns:
    the efficiency of the graph
    '''
    if G.number_of_nodes() == 0:
        return 0
    else:
        return nx.global_efficiency(G)

def compute_R(lcc:list):
    '''
    takes a list of lccs and returns the R value (lcc initial included)

    parameters:
    lcc: a list of lccs

    returns:
    the R value
    '''
    if len(lcc) == 0:
        return 0
    else:
        return sum(lcc)/len(lcc)
    
def compute_interE(eff:list):
    '''
    takes a list of efficiencies and returns the interE value (eff initial included)

    parameters:
    eff: a list of efficiencies

    returns:
    the interE value
    '''
    if len(eff) == 0:
        return 0
    else:
        return sum(eff)/len(eff)

### Centralities:

In [6]:
BC=nx.betweenness_centrality
DC=nx.degree_centrality
CC=nx.closeness_centrality
NBC=nx.load_centrality

###me
def get_MEB(G, k:int, centrality_dict=None):
    '''
    takes a graph and a node and the dict of the centrality we're using and returns the ME value

    parameters:
    G: a graph
    centrality_dict: a dictionary of centrality
    k:int, the node we're considering

    returns:
    the ME value
    '''
    if centrality_dict == None:
        centrality_dict = BC(G)

    BCk=centrality_dict[k]
    BC_neighbors=list(G.neighbors(k))
    MEB=0
    bet_centrality_neighbors=[centrality_dict[neighbor] for neighbor in BC_neighbors]
    sum_log_neighbors=sum([math.log(centrality_neighbor+1e-06) for centrality_neighbor in bet_centrality_neighbors])
    MEB=-BCk*sum_log_neighbors

    return MEB 

def get_MED(G, k:int, centrality_dict=None):
    '''
    takes a graph and the dict of the centrality we're using and returns the list of MED values

    parameters:
    G: a graph
    centrality_dict: a dictionary of centrality
    k:int, the node we're considering

    returns:
    the list of MED values
    '''
    if centrality_dict == None:
        centrality_dict = DC(G)
    DCk=centrality_dict[k]
    DC_neighbors=list(G.neighbors(k))
    MED=0
    deg_centrality_neighbors=[centrality_dict[neighbor] for neighbor in DC_neighbors]
    sum_log_neighbors=sum([math.log(centrality_neighbor+1e-06) for centrality_neighbor in deg_centrality_neighbors])
    MED=-DCk*sum_log_neighbors

    return MED

def get_MEC(G, k:int, centrality_dict=None):
    '''
    takes a graph and the dict of the centrality we're using and returns the list of MEC values

    parameters:
    G: a graph
    centrality_dict: a dictionary of centrality
    k:int, the node we're considering

    returns:
    the list of MEC values
    '''
    if centrality_dict == None:
        centrality_dict = CC(G)
    CCk=centrality_dict[k]
    CC_neighbors=list(G.neighbors(k))
    MEC=0
    clos_centrality_neighbors=[centrality_dict[neighbor] for neighbor in CC_neighbors]
    sum_log_neighbors=sum([math.log(centrality_neighbor+1e-06) for centrality_neighbor in clos_centrality_neighbors])
    MEC=-CCk*sum_log_neighbors

    return MEC

def MEB(G:nx.Graph, bet=None)->dict:
    '''takes a graph and returns a dictionary of MEB centrality for each node
    
    parameters:
    G: a graph

    returns:
    a dictionary of MEB centrality for each node
    '''
    if bet == None:
        bet = BC(G)
    MEB = {}
    for node in G.nodes():
        MEB[node] = get_MEB(G, node, bet)
    return MEB

def MED(G:nx.Graph, deg=None)->dict:
    '''takes a graph and returns a dictionary of MED centrality for each node
    
    parameters:
    G: a graph

    returns:
    a dictionary of MED centrality for each node
    '''
    if deg == None:
        deg = DC(G)
    MED = {}
    for node in G.nodes():
        MED[node] = get_MED(G, node, deg)
    return MED

def MEC(G:nx.Graph, clo=None)->dict:
    '''takes a graph and returns a dictionary of MEC centrality for each node
    
    parameters:
    G: a graph

    returns:
    a dictionary of MEC centrality for each node
    '''
    if clo == None:
        clo = CC(G)
    MEC = {}
    for node in G.nodes():
        MEC[node] = get_MEC(G, node, clo)
    return MEC
###PROX
def _proximity(node, shortest_paths):
    '''
    takes a graph and a node, returns the proximity centrality of that node
    '''
    sum_of_shortest_paths = sum(shortest_paths[node])
    sum_of_all_pairs_shortest_paths = sum(sum(shortest_paths))
    proximity = sum_of_all_pairs_shortest_paths / sum_of_shortest_paths 
    return proximity

def proximity(G:nx.Graph)->dict:
    '''takes a graph and returns a dictionary of proximity centrality for each node'''
    nodes = list(G.nodes())
    shortest_paths = nx.floyd_warshall_numpy(G)
    pr = {}
    for i, node in enumerate(nodes):
        pr[node] = _proximity(i, shortest_paths)
    return pr
###dwt
def compute_S(G:nx.Graph, i:int, j:int):
    '''takes node  and j and computes Sij'''
    Ni = G.neighbors(i)
    Nj = G.neighbors(j)

    intersection = set(Ni).intersection(set(Nj))

    if len(intersection)==0: #if inter is empty
        S=0
    else:
        ki= G.degree(i)
        kj= G.degree(j)
        S= len(intersection)/math.sqrt(ki*kj)

    return S

def DWT_i(G, i):
    '''takes a graph, a node and computes the DWT_i'''
    ki= G.degree(i)
    Ni= G.neighbors(i)
    DWT=0
    for j in Ni: #loop thru all neighbors and get the Sij
        Sij=compute_S(G, i, j)
        DWT += ( (1+Sij)/ki ) ** 2    
    return DWT

def DWT(G):
    # Calculate the DWT centrality for each node in the graph
    centrality = {}
    for node in G.nodes():
        centrality[node] = DWT_i(G, node)
    return centrality

###plci


#### plci

In [7]:
bn_macaque_rhesus_brain_2_complex_df = pd.read_csv('../plci_results/bn_macaque_rhesus_brain_2.csv', skiprows=1, header=None)
plci_bn_macaque_rhesus_brain_2 = dict(zip(bn_macaque_rhesus_brain_2_complex_df[0]-1, bn_macaque_rhesus_brain_2_complex_df[1]))
plci_bn_macaque_rhesus_brain_2_sorted = sorted(plci_bn_macaque_rhesus_brain_2.items(), key=lambda x: x[1], reverse=True)
plci_bn_macaque_rhesus_brain_2_nodes =[x[0] for x in plci_bn_macaque_rhesus_brain_2_sorted]

word_adjacencies_complex_df = pd.read_csv('../plci_results/word_adjacencies.csv', skiprows=1, header=None)
plci_word_adjacencies = dict(zip(word_adjacencies_complex_df[0]-1, word_adjacencies_complex_df[1]))
plci_word_adjacencies_sorted = sorted(plci_word_adjacencies.items(), key=lambda x: x[1], reverse=True)
plci_word_adjacencies_nodes =[x[0] for x in plci_word_adjacencies_sorted]

circuits_s208_complex_df = pd.read_csv('../plci_results/circuits s208.csv', skiprows=1, header=None)
plci_circuits_s208 = dict(zip(circuits_s208_complex_df[0]-1, circuits_s208_complex_df[1]))
plci_circuits_s208_sorted = sorted(plci_circuits_s208.items(), key=lambda x: x[1], reverse=True)
plci_circuits_s208_nodes =[x[0] for x in plci_circuits_s208_sorted]

bn_cat_mixed_species_brain_1_complex_df = pd.read_csv('../plci_results/bn_cat_mixed_species_brain_1.csv', skiprows=1, header=None)
plci_bn_cat_mixed_species_brain_1 = dict(zip(bn_cat_mixed_species_brain_1_complex_df[0]-1, bn_cat_mixed_species_brain_1_complex_df[1]))
plci_bn_cat_mixed_species_brain_1_sorted = sorted(plci_bn_cat_mixed_species_brain_1.items(), key=lambda x: x[1], reverse=True)
plci_bn_cat_mixed_species_brain_1_nodes =[x[0] for x in plci_bn_cat_mixed_species_brain_1_sorted]

facebook_414_complex_df = pd.read_csv('../plci_results/facebook 414.csv', skiprows=1, header=None)
plci_facebook_414 = dict(zip(facebook_414_complex_df[0]-1, facebook_414_complex_df[1]))
plci_facebook_414_sorted = sorted(plci_facebook_414.items(), key=lambda x: x[1], reverse=True)
plci_facebook_414_nodes =[x[0] for x in plci_facebook_414_sorted]

facebook_686_complex_df = pd.read_csv('../plci_results/facebook 686.csv', skiprows=1, header=None)
plci_facebook_686 = dict(zip(facebook_686_complex_df[0]-1, facebook_686_complex_df[1]))
plci_facebook_686_sorted = sorted(plci_facebook_686.items(), key=lambda x: x[1], reverse=True)
plci_facebook_686_nodes =[x[0] for x in plci_facebook_686_sorted]

polbooks_complex_df = pd.read_csv('../plci_results/polbooks.csv', skiprows=1, header=None)
plci_polbooks = dict(zip(polbooks_complex_df[0]-1, polbooks_complex_df[1]))
plci_polbooks_sorted = sorted(plci_polbooks.items(), key=lambda x: x[1], reverse=True)
plci_polbooks_nodes =[x[0] for x in plci_polbooks_sorted]

jazz_complex_df = pd.read_csv('../plci_results/jazz.csv', skiprows=1, header=None)  
plci_jazz = dict(zip(jazz_complex_df[0]-1, jazz_complex_df[1]))
plci_jazz_sorted = sorted(plci_jazz.items(), key=lambda x: x[1], reverse=True)
plci_jazz_nodes =[x[0] for x in plci_jazz_sorted]

facebook_348_complex_df = pd.read_csv('../plci_results/facebook 348.csv', skiprows=1, header=None)
plci_facebook_348 = dict(zip(facebook_348_complex_df[0]-1, facebook_348_complex_df[1]))
plci_facebook_348_sorted = sorted(plci_facebook_348.items(), key=lambda x: x[1], reverse=True)
plci_facebook_348_nodes =[x[0] for x in plci_facebook_348_sorted]

circuits_s420_complex_df = pd.read_csv('../plci_results/circuits s420.csv', skiprows=1, header=None)
plci_circuits_s420 = dict(zip(circuits_s420_complex_df[0]-1, circuits_s420_complex_df[1]))
plci_circuits_s420_sorted = sorted(plci_circuits_s420.items(), key=lambda x: x[1], reverse=True)
plci_circuits_s420_nodes =[x[0] for x in plci_circuits_s420_sorted]

corruption_complex_df = pd.read_csv('../plci_results/corruption.csv', skiprows=1, header=None)  
plci_corruption = dict(zip(corruption_complex_df[0]-1, corruption_complex_df[1]))
plci_corruption_sorted = sorted(plci_corruption.items(), key=lambda x: x[1], reverse=True)
plci_corruption_nodes =[x[0] for x in plci_corruption_sorted]

facebook_0_complex_df = pd.read_csv('../plci_results/facebook 0.csv', skiprows=1, header=None)
plci_facebook_0 = dict(zip(facebook_0_complex_df[0]-1, facebook_0_complex_df[1]))
plci_facebook_0_sorted = sorted(plci_facebook_0.items(), key=lambda x: x[1], reverse=True)
plci_facebook_0_nodes =[x[0] for x in plci_facebook_0_sorted]

circuits_s838_complex_df = pd.read_csv('../plci_results/circuits s838.csv', skiprows=1, header=None)
plci_circuits_s838 = dict(zip(circuits_s838_complex_df[0]-1, circuits_s838_complex_df[1]))
plci_circuits_s838_sorted = sorted(plci_circuits_s838.items(), key=lambda x: x[1], reverse=True)
plci_circuits_s838_nodes =[x[0] for x in plci_circuits_s838_sorted]

crime_net_complex_df = pd.read_csv('../plci_results/CrimeNet.csv', skiprows=1, header=None)
plci_crime_net = dict(zip(crime_net_complex_df[0]-1, crime_net_complex_df[1]))
plci_crime_net_sorted = sorted(plci_crime_net.items(), key=lambda x: x[1], reverse=True)
plci_crime_net_nodes =[x[0] for x in plci_crime_net_sorted]

facebook_1684_complex_df = pd.read_csv('../plci_results/facebook 1684.csv', skiprows=1, header=None)
plci_facebook_1684 = dict(zip(facebook_1684_complex_df[0]-1, facebook_1684_complex_df[1]))
plci_facebook_1684_sorted = sorted(plci_facebook_1684.items(), key=lambda x: x[1], reverse=True)
plci_facebook_1684_nodes =[x[0] for x in plci_facebook_1684_sorted]

### Attacks

#### node

In [101]:
nets = [bn_macaque_rhesus_brain_2, word_adjacencies, circuits_s208, bn_cat_mixed_species_brain_1, facebook_414, facebook_686, polbooks, jazz, facebook_348, circuits_s420, corruption, facebook_0, circuits_s838, crime_net, facebook_1684]
net_names = ["bn_macaque_rhesus_brain_2", "word_adjacencies", "circuits_s208", "bn_cat_mixed_species_brain_1", "facebook_414", "facebook_686", "polbooks", "jazz", "facebook_348", "circuits_s420", "corruption", "facebook_0", "circuits_s838", "crime_net", "facebook_1684"]
centrality_functions=[BC, DC, CC, NBC, MEB, MED, MEC, proximity, DWT]

#helper method to split list into batches
def partition_list(input_list, num=100):
    n = len(input_list) // num
    remainder = len(input_list) % num
    partitions = []
    for i in range(num):
        start = i * n + min(i, remainder)
        end = start + n + (i < remainder)
        partitions.append(input_list[start:end])
    return partitions

#main targeted node attack method
def attack_node(g:nx.Graph, sorted_nodes, batch_num=100):
    ''''
    takes a graph and returns the lcc and efficiency after removing each node in the graph

    parameters:
    g: a graph
    sorted_nodes: a list of nodes sorted by centrality
    batch_num: the number of batches to divide the nodes into

    returns:
    [0]: a dictionary of lccs after removing each node
    [1]: a dictionary of efficiencies after removing each node
    [2]: a dictionary of lccs after removing each node sequentially
    [3]: a dictionary of efficiencies after removing each node sequentially
    [4]: a list of R values 
    [5]: the interE value
    [6]: the time it took to run the function
    [7]: a list of graphs after removing each 10% of the nodes

    '''

    G=g.copy() ; N=G.number_of_nodes()
    pi = partition_list(sorted_nodes, batch_num)

    lcc=0;norm_lcc=0;e=0
    norm_LCC={};E={}
    norm_LCC_seq={};E_seq={}
    R=[]; InterE=[]
    graphs=[[] for i in range(10)]
    graphs[0]=G.copy()

    t=time.time()

    lcc_i=compute_lcc(G); norm_lcc_i=lcc_i/N; e_i=compute_efficiency(G)
    R.append(norm_lcc_i); InterE.append(e_i)
    norm_LCC[0]=norm_lcc_i; E[0]=e_i
    norm_LCC_seq[0]=norm_lcc_i; E_seq[0]=e_i
    R_value=0;InterE_value=0
    c=0;n=0;g_num=0

    for batch in pi: #looping through each batch
        c+=1 #batch number removed
        for node in batch: #looping through each node in the batch
            n+=1 #node number removed
            G.remove_node(node)
            lcc = compute_lcc(G); norm_lcc = lcc/N
            e = nx.algorithms.global_efficiency(G)

            R.append(norm_lcc); InterE.append(e)
            norm_LCC_seq[n]=norm_lcc; E_seq[n]=e

        if c%10 ==0: #each 10% removed save the graph
            g_num+=1
            if c!=100:
                graphs[g_num]=G.copy()
                

        norm_LCC[c]=norm_lcc
        E[c]=e

    end=time.time()
    t=end-t

    #R at halftime doesn not hold according to formula
    R_value=sum(R)/len(R)
    InterE_value=sum(InterE)/len(InterE)

    graphs = [nx.Graph(edges) for edges in graphs]

    return norm_LCC, E, norm_LCC_seq, E_seq, R_value , InterE_value, t, graphs

#main random node attack method - uses attack_node
def random_attack_node(G:nx.Graph):
    '''
    randomly shuffles nodes and attack them
    '''
    nodes = list(G.nodes())
    random.shuffle(nodes)
    return attack_node(G, nodes)

#helper method to create sorted greeedy list of nodes
def greedy_list(g) ->list:
    '''
    takes a graph and computes a list of node that follows the greedy attack scheme
    Get an initial list of best nodes to attack - network of n nodes, try n times:
        Copy the network in a new test network
        Remove a node from the test
        Assess robustness score after its removal
    Do this for the n nodes, sort them by order of most impactful attacks and emply the sequential 
    '''
    n=g.number_of_nodes()
    nodes=list(g.nodes())
    stat={}
    for i in range(n):
        G=g.copy()
        G.remove_node(nodes[i])
        lcc=compute_lcc(G)
        stat[nodes[i]]=lcc
    #sort the nodes keys in ascending order of their values in the dictionary
    nodes_sorted =  sorted(stat.items(), key=lambda x: x[1], reverse=False) 
    nodes_sorted_list = [x[0] for x in nodes_sorted]
    return nodes_sorted_list

#main greedy node attack method - uses attack_node
def greedy_attack_node(G:nx.Graph):
    '''
    takes a graph, and computes the greedy attack on it
    '''
    sorted_nodes= greedy_list(G)
    return attack_node(G, sorted_nodes)

#helper method to print progress
def done(net_Str):
    print("done with ", net_Str, "; ", end='')

###########################################################################################################################################################################################

###############################          TEST THESE          #########################################


#helper method to get boundary nodes from a graph already split into sommunities
def get_boundary_nodes(G, partition):   
    '''takes graph and partiton - list of sets of nodes - as input and returns the boundary nodes in a list
    param:
    G: a graph
    partition: a list of sets of nodes

    returns:
    a list of boundary nodes
    '''
    boundary_edges = set()
    for i in range(len(partition)):
        for j in range(len(partition)):
            if i != j:
                boundary_edges.update(nx.edge_boundary(G, partition[i], partition[j]))

    b_nodes = []
    for i in range(len(boundary_edges)):
        b_nodes.append(list(boundary_edges)[i][0])
        b_nodes.append(list(boundary_edges)[i][1])
    b_nodes = list(set(b_nodes))
    return b_nodes

#helper method that takes a list of nodes and sort them 1. by boundary/internal and 2. by centrality (they come sorted by default)
def sort_bounded(sorted_list, boundary_list):
    '''
    takes a sorted list and a list of boundary nodes and returns a sorted list where the boundary nodes are at the start
    param:
    sorted_list: a list of sorted nodes
    boundary_list: a list of boundary nodes

    returns:
    a sorted list where the boundary nodes are at the start
    '''
    new_list=[]
    sorted_list_q=sorted_list.copy()
    for i in range(len(sorted_list)):
        if sorted_list[i] in boundary_list:
            new_list.append(sorted_list[i])
            sorted_list_q.remove(sorted_list[i])
        else:
            continue
    for i in range(len(sorted_list_q)):
        new_list.append(sorted_list_q[i])
        return new_list

#helper method to split a graph to communities
def split_to_communities(G:nx.Graph):
    '''
    takes a graph and partitions it into communities by running Louvain 10 times and picking the one with highest modularity

    returns:
    a list of communities (list of sets of nodes)
    '''
    best_p=[]; best_modularity=0
    for i in range(10):
        p=nx.community.louvain_communities(G, seed=random.randint(0,1000))
        q=nx.community.modularity(G, p)
        if q>best_modularity:
            best_modularity=q
            best_p=p
    return best_p

#main method for community based attack on nodes
def community_attack_node(G:nx.Graph, G_communities, centrality_func):

    '''takes a graph and a list of communities and returns the attack - relies on attack_node method'''


    sorted_nodes=sorted_nodes=[x[0] for x in sorted(centrality_func(G).items(), key=lambda x: x[1], reverse=True)]

    partitions=G_communities
    boundary_nodes=get_boundary_nodes(G, partitions)
    boundary_sorted = [x[0] for x in sorted(centrality_func(G).items(), key=lambda x: x[1], reverse=True) if x[0] in boundary_nodes]
    
    sorted_nodes=sort_bounded(sorted_nodes, boundary_sorted)

    return attack_node(G, sorted_nodes)



#### edge

In [ ]:
def attack_edge(g:nx.Graph, sorted_edges, batch_num=100):
    ''''
    takes a graph and returns the lcc and efficiency after removing each edge in the graph

    parameters:
    g: a graph
    sorted_edges: a list of edges sorted by centrality
    batch_num: the number of batches to divide the edges into

    returns:
    [0]: a dictionary of lccs after removing each edge
    [1]: a dictionary of efficiencies after removing each edge
    [2]: a dictionary of lccs after removing each edge sequentially
    [3]: a dictionary of efficiencies after removing each edge sequentially
    [4]: a list of R values 
    [5]: the interE value
    [6]: the time it took to run the function
    [7]: a list of graphs after removing each 10% of the edges

    '''

    G=g.copy() ; M=G.number_of_edges()
    pi = partition_list(sorted_edges, batch_num)

    lcc=0;norm_lcc=0;e=0
    norm_LCC={};E={}
    norm_LCC_seq={};E_seq={}
    R=[]; InterE=[]
    graphs=[]

    t=time.time()

    lcc_i=compute_lcc(G); norm_lcc_i=lcc_i/M; e_i=compute_efficiency(G)
    R.append(norm_lcc_i); InterE.append(e_i)
    norm_LCC[0]=norm_lcc_i; E[0]=e_i
    norm_LCC_seq[0]=norm_lcc_i; E_seq[0]=e_i
    R_value=0;InterE_value=0
    c=0;n=0

    for batch in pi: #looping through each batch
        c+=1 #batch number removed
        for edge in batch: #looping through each node in the batch
            n+=1 #node number removed
            G.remove_edge(min(edge[0], edge[1]), max(edge[0], edge[1]))
            lcc = compute_lcc(G); norm_lcc = lcc/M
            e = nx.algorithms.global_efficiency(G)

            R.append(norm_lcc); InterE.append(e)
            norm_LCC_seq[n]=norm_lcc; E_seq[n]=e

        if (c)%10 ==0: #each 10% removed save the graph
            graphs.append(G)

        norm_LCC[c]=norm_lcc
        E[c]=e

    end=time.time()
    t=end-t

    #R at halftime doesn not hold according to formula
    R_value=sum(R)/len(R)
    InterE_value=sum(InterE)/len(InterE)

    return norm_LCC, E, norm_LCC_seq, E_seq, R_value , InterE_value, t, graphs


def random_attack_edge(G:nx.Graph):
    '''
    randomly shuffles edges and attack them
    '''
    edges = list(G.edges())
    random.shuffle(edges)
    return attack_edge(G, edges)

#helper method to create sorted greeedy list of nodes
def greedy_list_edges(g) ->list:
    '''
    takes a graph and computes a list of edges that follows the greedy attack scheme
    Get an initial list of best nodes to attack - network of m edges, try m times:
        Copy the network in a new test network
        Remove a edge from the test
        Assess robustness score after its removal
    Do this for the m edges, sort them by order of most impactful attacks and emply the sequential 
    '''
    m=g.number_of_edges()
    edges=list(g.edges())
    stat={}
    for i in range(m):
        G=g.copy()
        G.remove_edge(edges[i][0], edges[i][1])
        lcc=compute_lcc(G)
        stat[edges[i]]=lcc
    #sort the nodes keys in ascending order of their values in the dictionary
    edges_sorted =  sorted(stat.items(), key=lambda x: x[1], reverse=False) 
    edges_sorted_list = [x[0] for x in edges_sorted]
    return edges_sorted_list

#main greedy node attack method - uses attack_node
def greedy_attack_node_edges(G:nx.Graph):
    '''
    takes a graph, and computes the greedy attack on it
    '''
    sorted_edges= greedy_list_edges(G)
    return attack_edge(G, sorted_edges)

def get_intercommunity_edges(G, partition):   
    '''takes graph and partiton - list of sets of edges - as input and returns the intercommunity edges in a list
    param:
    G: a graph
    partition: a list of sets of nodes

    returns:
    a list of intercommunity edges
    '''
    boundary_edges = set()
    for i in range(len(partition)):
        for j in range(len(partition)):
            if i != j:
                boundary_edges.update(nx.edge_boundary(G, partition[i], partition[j]))
    return boundary_edges

def sort_bounded_edges(sorted_list, boundary_list):
    '''
    takes a sorted list and a list of boundary edges and returns a sorted list where the boundary edges are at the start
    param:
    sorted_list: a list of sorted edges
    boundary_list: a list of boundary edges

    returns:
    a sorted list where the boundary edges are at the start
    '''
    new_list=[]
    sorted_list_q=sorted_list.copy()
    for i in range(len(sorted_list)):
        if sorted_list[i] in boundary_list:
            new_list.append(sorted_list[i])
            sorted_list_q.remove(sorted_list[i])
        else:
            continue
    for i in range(len(sorted_list_q)):
        new_list.append(sorted_list_q[i])
        return new_list

def community_attack_edge(G:nx.Graph, G_communities, centrality_func=nx.edge_betweenness_centrality):
    '''takes a graph and a list of communities and returns the attack - relies on attack_node method'''

    sorted_edges=[x[0] for x in sorted((centrality_func(bn_macaque_rhesus_brain_2)).items(), key=lambda x: x[1], reverse=True)]

    partitions=G_communities
    boundary_edges=get_intercommunity_edges(G, partitions)
    boundary_sorted = [x[0] for x in sorted(centrality_func(G).items(), key=lambda x: x[1], reverse=True) if x[0] in boundary_edges] #not necessary to sort in fact
    
    sorted_edges=sort_bounded(sorted_edges, boundary_sorted)

    return attack_edge(G, sorted_edges)

## Applications:

### node-targeted attacks:

#### All centralities:

In [206]:
BC_bn_macaque_rhesus_brain_2_attack=""; DC_bn_macaque_rhesus_brain_2_attack=""; CC_bn_macaque_rhesus_brain_2_attack=""; NBC_bn_macaque_rhesus_brain_2_attack="";  MEB_bn_macaque_rhesus_brain_2_attack=""; MED_bn_macaque_rhesus_brain_2_attack=""; MEC_bn_macaque_rhesus_brain_2_attack=""; PR_bn_macaque_rhesus_brain_2_attack=""; DWT_bn_macaque_rhesus_brain_2_attack=""
bn_macaque_rhesus_brain_2_centrality_nodes_attacks=[BC_bn_macaque_rhesus_brain_2_attack, DC_bn_macaque_rhesus_brain_2_attack, CC_bn_macaque_rhesus_brain_2_attack, NBC_bn_macaque_rhesus_brain_2_attack,  MEB_bn_macaque_rhesus_brain_2_attack, MED_bn_macaque_rhesus_brain_2_attack, MEC_bn_macaque_rhesus_brain_2_attack, PR_bn_macaque_rhesus_brain_2_attack, DWT_bn_macaque_rhesus_brain_2_attack]
for i in range(len(bn_macaque_rhesus_brain_2_centrality_nodes_attacks)):
    bn_macaque_rhesus_brain_2_centrality_nodes_attacks[i]=attack_node(bn_macaque_rhesus_brain_2, [x[0] for x in sorted((centrality_functions[i](bn_macaque_rhesus_brain_2)).items(), key=lambda x: x[1], reverse=True)])
print("done with bn_macaque_rhesus_brain_2; ", end='')
BC_bn_macaque_rhesus_brain_2_attack=bn_macaque_rhesus_brain_2_centrality_nodes_attacks[0]; DC_bn_macaque_rhesus_brain_2_attack=bn_macaque_rhesus_brain_2_centrality_nodes_attacks[1]; CC_bn_macaque_rhesus_brain_2_attack=bn_macaque_rhesus_brain_2_centrality_nodes_attacks[2]; NBC_bn_macaque_rhesus_brain_2_attack=bn_macaque_rhesus_brain_2_centrality_nodes_attacks[3];  MEB_bn_macaque_rhesus_brain_2_attack=bn_macaque_rhesus_brain_2_centrality_nodes_attacks[4]; MED_bn_macaque_rhesus_brain_2_attack=bn_macaque_rhesus_brain_2_centrality_nodes_attacks[5]; MEC_bn_macaque_rhesus_brain_2_attack=bn_macaque_rhesus_brain_2_centrality_nodes_attacks[6]; PR_bn_macaque_rhesus_brain_2_attack=bn_macaque_rhesus_brain_2_centrality_nodes_attacks[7]; DWT_bn_macaque_rhesus_brain_2_attack=bn_macaque_rhesus_brain_2_centrality_nodes_attacks[8]

BC_word_adjacencies_attack=""; DC_word_adjacencies_attack=""; CC_word_adjacencies_attack=""; NBC_word_adjacencies_attack="";  MEB_word_adjacencies_attack=""; MED_word_adjacencies_attack=""; MEC_word_adjacencies_attack=""; PR_word_adjacencies_attack=""; DWT_word_adjacencies_attack=""
word_adjacencies_centrality_nodes_attacks=[BC_word_adjacencies_attack, DC_word_adjacencies_attack, CC_word_adjacencies_attack, NBC_word_adjacencies_attack,  MEB_word_adjacencies_attack, MED_word_adjacencies_attack, MEC_word_adjacencies_attack, PR_word_adjacencies_attack, DWT_word_adjacencies_attack]
for i in range(len(word_adjacencies_centrality_nodes_attacks)):
    word_adjacencies_centrality_nodes_attacks[i]=attack_node(word_adjacencies, [x[0] for x in sorted((centrality_functions[i](word_adjacencies)).items(), key=lambda x: x[1], reverse=True)])
print("done with word_adjacencies; ", end='')
BC_word_adjacencies_attack=word_adjacencies_centrality_nodes_attacks[0]; DC_word_adjacencies_attack=word_adjacencies_centrality_nodes_attacks[1]; CC_word_adjacencies_attack=word_adjacencies_centrality_nodes_attacks[2]; NBC_word_adjacencies_attack=word_adjacencies_centrality_nodes_attacks[3];  MEB_word_adjacencies_attack=word_adjacencies_centrality_nodes_attacks[4]; MED_word_adjacencies_attack=word_adjacencies_centrality_nodes_attacks[5]; MEC_word_adjacencies_attack=word_adjacencies_centrality_nodes_attacks[6]; PR_word_adjacencies_attack=word_adjacencies_centrality_nodes_attacks[7]; DWT_word_adjacencies_attack=word_adjacencies_centrality_nodes_attacks[8]

BC_circuits_s208_attack=""; DC_circuits_s208_attack=""; CC_circuits_s208_attack=""; NBC_circuits_s208_attack="";  MEB_circuits_s208_attack=""; MED_circuits_s208_attack=""; MEC_circuits_s208_attack=""; PR_circuits_s208_attack=""; DWT_circuits_s208_attack=""
circuits_s208_centrality_nodes_attacks=[BC_circuits_s208_attack, DC_circuits_s208_attack, CC_circuits_s208_attack, NBC_circuits_s208_attack,  MEB_circuits_s208_attack, MED_circuits_s208_attack, MEC_circuits_s208_attack, PR_circuits_s208_attack, DWT_circuits_s208_attack]
for i in range(len(circuits_s208_centrality_nodes_attacks)):
    circuits_s208_centrality_nodes_attacks[i]=attack_node(circuits_s208, [x[0] for x in sorted((centrality_functions[i](circuits_s208)).items(), key=lambda x: x[1], reverse=True)])
print("done with circuits_s208; ", end='')
BC_circuits_s208_attack=circuits_s208_centrality_nodes_attacks[0]; DC_circuits_s208_attack=circuits_s208_centrality_nodes_attacks[1]; CC_circuits_s208_attack=circuits_s208_centrality_nodes_attacks[2]; NBC_circuits_s208_attack=circuits_s208_centrality_nodes_attacks[3];  MEB_circuits_s208_attack=circuits_s208_centrality_nodes_attacks[4]; MED_circuits_s208_attack=circuits_s208_centrality_nodes_attacks[5]; MEC_circuits_s208_attack=circuits_s208_centrality_nodes_attacks[6]; PR_circuits_s208_attack=circuits_s208_centrality_nodes_attacks[7]; DWT_circuits_s208_attack=circuits_s208_centrality_nodes_attacks[8]

BC_bn_cat_mixed_species_brain_1_attack=""; DC_bn_cat_mixed_species_brain_1_attack=""; CC_bn_cat_mixed_species_brain_1_attack=""; NBC_bn_cat_mixed_species_brain_1_attack="";  MEB_bn_cat_mixed_species_brain_1_attack=""; MED_bn_cat_mixed_species_brain_1_attack=""; MEC_bn_cat_mixed_species_brain_1_attack=""; PR_bn_cat_mixed_species_brain_1_attack=""; DWT_bn_cat_mixed_species_brain_1_attack=""
bn_cat_mixed_species_brain_1_centrality_nodes_attacks=[BC_bn_cat_mixed_species_brain_1_attack, DC_bn_cat_mixed_species_brain_1_attack, CC_bn_cat_mixed_species_brain_1_attack, NBC_bn_cat_mixed_species_brain_1_attack,  MEB_bn_cat_mixed_species_brain_1_attack, MED_bn_cat_mixed_species_brain_1_attack, MEC_bn_cat_mixed_species_brain_1_attack, PR_bn_cat_mixed_species_brain_1_attack, DWT_bn_cat_mixed_species_brain_1_attack]
for i in range(len(bn_cat_mixed_species_brain_1_centrality_nodes_attacks)):
    bn_cat_mixed_species_brain_1_centrality_nodes_attacks[i]=attack_node(bn_cat_mixed_species_brain_1, [x[0] for x in sorted((centrality_functions[i](bn_cat_mixed_species_brain_1)).items(), key=lambda x: x[1], reverse=True)])
print("done with bn_cat_mixed_species_brain_1; ", end='')
BC_bn_cat_mixed_species_brain_1_attack=bn_cat_mixed_species_brain_1_centrality_nodes_attacks[0]; DC_bn_cat_mixed_species_brain_1_attack=bn_cat_mixed_species_brain_1_centrality_nodes_attacks[1]; CC_bn_cat_mixed_species_brain_1_attack=bn_cat_mixed_species_brain_1_centrality_nodes_attacks[2]; NBC_bn_cat_mixed_species_brain_1_attack=bn_cat_mixed_species_brain_1_centrality_nodes_attacks[3];  MEB_bn_cat_mixed_species_brain_1_attack=bn_cat_mixed_species_brain_1_centrality_nodes_attacks[4]; MED_bn_cat_mixed_species_brain_1_attack=bn_cat_mixed_species_brain_1_centrality_nodes_attacks[5]; MEC_bn_cat_mixed_species_brain_1_attack=bn_cat_mixed_species_brain_1_centrality_nodes_attacks[6]; PR_bn_cat_mixed_species_brain_1_attack=bn_cat_mixed_species_brain_1_centrality_nodes_attacks[7]; DWT_bn_cat_mixed_species_brain_1_attack=bn_cat_mixed_species_brain_1_centrality_nodes_attacks[8]


BC_facebook_414_attack=""; DC_facebook_414_attack=""; CC_facebook_414_attack=""; NBC_facebook_414_attack="";  MEB_facebook_414_attack=""; MED_facebook_414_attack=""; MEC_facebook_414_attack=""; PR_facebook_414_attack=""; DWT_facebook_414_attack=""
facebook_414_centrality_nodes_attacks=[BC_facebook_414_attack, DC_facebook_414_attack, CC_facebook_414_attack, NBC_facebook_414_attack,  MEB_facebook_414_attack, MED_facebook_414_attack, MEC_facebook_414_attack, PR_facebook_414_attack, DWT_facebook_414_attack]
for i in range(len(facebook_414_centrality_nodes_attacks)):
    facebook_414_centrality_nodes_attacks[i]=attack_node(facebook_414, [x[0] for x in sorted((centrality_functions[i](facebook_414)).items(), key=lambda x: x[1], reverse=True)])
print("done with facebook_414; ", end="")
BC_facebook_414_attack=facebook_414_centrality_nodes_attacks[0]; DC_facebook_414_attack=facebook_414_centrality_nodes_attacks[1]; CC_facebook_414_attack=facebook_414_centrality_nodes_attacks[2]; NBC_facebook_414_attack=facebook_414_centrality_nodes_attacks[3];  MEB_facebook_414_attack=facebook_414_centrality_nodes_attacks[4]; MED_facebook_414_attack=facebook_414_centrality_nodes_attacks[5]; MEC_facebook_414_attack=facebook_414_centrality_nodes_attacks[6]; PR_facebook_414_attack=facebook_414_centrality_nodes_attacks[7]; DWT_facebook_414_attack=facebook_414_centrality_nodes_attacks[8]


BC_facebook_686_attack=""; DC_facebook_686_attack=""; CC_facebook_686_attack=""; NBC_facebook_686_attack="";  MEB_facebook_686_attack=""; MED_facebook_686_attack=""; MEC_facebook_686_attack=""; PR_facebook_686_attack=""; DWT_facebook_686_attack=""
facebook_686_centrality_nodes_attacks=[BC_facebook_686_attack, DC_facebook_686_attack, CC_facebook_686_attack, NBC_facebook_686_attack,  MEB_facebook_686_attack, MED_facebook_686_attack, MEC_facebook_686_attack, PR_facebook_686_attack, DWT_facebook_686_attack]
for i in range(len(facebook_686_centrality_nodes_attacks)):
    facebook_686_centrality_nodes_attacks[i]=attack_node(facebook_686, [x[0] for x in sorted((centrality_functions[i](facebook_686)).items(), key=lambda x: x[1], reverse=True)])
print("done with facebook_686; ", end='')
BC_facebook_686_attack=facebook_686_centrality_nodes_attacks[0]; DC_facebook_686_attack=facebook_686_centrality_nodes_attacks[1]; CC_facebook_686_attack=facebook_686_centrality_nodes_attacks[2]; NBC_facebook_686_attack=facebook_686_centrality_nodes_attacks[3];  MEB_facebook_686_attack=facebook_686_centrality_nodes_attacks[4]; MED_facebook_686_attack=facebook_686_centrality_nodes_attacks[5]; MEC_facebook_686_attack=facebook_686_centrality_nodes_attacks[6]; PR_facebook_686_attack=facebook_686_centrality_nodes_attacks[7]; DWT_facebook_686_attack=facebook_686_centrality_nodes_attacks[8]


BC_polbooks_attack=""; DC_polbooks_attack=""; CC_polbooks_attack=""; NBC_polbooks_attack="";  MEB_polbooks_attack=""; MED_polbooks_attack=""; MEC_polbooks_attack=""; PR_polbooks_attack=""; DWT_polbooks_attack=""
polbooks_centrality_nodes_attacks=[BC_polbooks_attack, DC_polbooks_attack, CC_polbooks_attack, NBC_polbooks_attack,  MEB_polbooks_attack, MED_polbooks_attack, MEC_polbooks_attack, PR_polbooks_attack, DWT_polbooks_attack]
for i in range(len(polbooks_centrality_nodes_attacks)):
    polbooks_centrality_nodes_attacks[i]=attack_node(polbooks, [x[0] for x in sorted((centrality_functions[i](polbooks)).items(), key=lambda x: x[1], reverse=True)])
print("done with polbooks; ", end='')
BC_polbooks_attack=polbooks_centrality_nodes_attacks[0]; DC_polbooks_attack=polbooks_centrality_nodes_attacks[1]; CC_polbooks_attack=polbooks_centrality_nodes_attacks[2]; NBC_polbooks_attack=polbooks_centrality_nodes_attacks[3];  MEB_polbooks_attack=polbooks_centrality_nodes_attacks[4]; MED_polbooks_attack=polbooks_centrality_nodes_attacks[5]; MEC_polbooks_attack=polbooks_centrality_nodes_attacks[6]; PR_polbooks_attack=polbooks_centrality_nodes_attacks[7]; DWT_polbooks_attack=polbooks_centrality_nodes_attacks[8]

BC_jazz_attack=""; DC_jazz_attack=""; CC_jazz_attack=""; NBC_jazz_attack="";  MEB_jazz_attack=""; MED_jazz_attack=""; MEC_jazz_attack=""; PR_jazz_attack=""; DWT_jazz_attack=""
jazz_centrality_nodes_attacks=[BC_jazz_attack, DC_jazz_attack, CC_jazz_attack, NBC_jazz_attack,  MEB_jazz_attack, MED_jazz_attack, MEC_jazz_attack, PR_jazz_attack, DWT_jazz_attack]
for i in range(len(jazz_centrality_nodes_attacks)):
    jazz_centrality_nodes_attacks[i]=attack_node(jazz, [x[0] for x in sorted((centrality_functions[i](jazz)).items(), key=lambda x: x[1], reverse=True)])
print("done with jazz; ", end='')
BC_jazz_attack=jazz_centrality_nodes_attacks[0]; DC_jazz_attack=jazz_centrality_nodes_attacks[1]; CC_jazz_attack=jazz_centrality_nodes_attacks[2]; NBC_jazz_attack=jazz_centrality_nodes_attacks[3];  MEB_jazz_attack=jazz_centrality_nodes_attacks[4]; MED_jazz_attack=jazz_centrality_nodes_attacks[5]; MEC_jazz_attack=jazz_centrality_nodes_attacks[6]; PR_jazz_attack=jazz_centrality_nodes_attacks[7]; DWT_jazz_attack=jazz_centrality_nodes_attacks[8]


BC_facebook_348_attack=""; DC_facebook_348_attack=""; CC_facebook_348_attack=""; NBC_facebook_348_attack="";  MEB_facebook_348_attack=""; MED_facebook_348_attack=""; MEC_facebook_348_attack=""; PR_facebook_348_attack=""; DWT_facebook_348_attack=""
facebook_348_centrality_nodes_attacks=[BC_facebook_348_attack, DC_facebook_348_attack, CC_facebook_348_attack, NBC_facebook_348_attack,  MEB_facebook_348_attack, MED_facebook_348_attack, MEC_facebook_348_attack, PR_facebook_348_attack, DWT_facebook_348_attack]
for i in range(len(facebook_348_centrality_nodes_attacks)):
    facebook_348_centrality_nodes_attacks[i]=attack_node(facebook_348, [x[0] for x in sorted((centrality_functions[i](facebook_348)).items(), key=lambda x: x[1], reverse=True)])
print("done with facebook_348; ", end='')
BC_facebook_348_attack=facebook_348_centrality_nodes_attacks[0]; DC_facebook_348_attack=facebook_348_centrality_nodes_attacks[1]; CC_facebook_348_attack=facebook_348_centrality_nodes_attacks[2]; NBC_facebook_348_attack=facebook_348_centrality_nodes_attacks[3];  MEB_facebook_348_attack=facebook_348_centrality_nodes_attacks[4]; MED_facebook_348_attack=facebook_348_centrality_nodes_attacks[5]; MEC_facebook_348_attack=facebook_348_centrality_nodes_attacks[6]; PR_facebook_348_attack=facebook_348_centrality_nodes_attacks[7]; DWT_facebook_348_attack=facebook_348_centrality_nodes_attacks[8]


BC_circuits_s420_attack=""; DC_circuits_s420_attack=""; CC_circuits_s420_attack=""; NBC_circuits_s420_attack="";  MEB_circuits_s420_attack=""; MED_circuits_s420_attack=""; MEC_circuits_s420_attack=""; PR_circuits_s420_attack=""; DWT_circuits_s420_attack=""
circuits_s420_centrality_nodes_attacks=[BC_circuits_s420_attack, DC_circuits_s420_attack, CC_circuits_s420_attack, NBC_circuits_s420_attack,  MEB_circuits_s420_attack, MED_circuits_s420_attack, MEC_circuits_s420_attack, PR_circuits_s420_attack, DWT_circuits_s420_attack]
for i in range(len(circuits_s420_centrality_nodes_attacks)):
    circuits_s420_centrality_nodes_attacks[i]=attack_node(circuits_s420, [x[0] for x in sorted((centrality_functions[i](circuits_s420)).items(), key=lambda x: x[1], reverse=True)])
print("done with circuits_s420; ", end='')
BC_circuits_s420_attack=circuits_s420_centrality_nodes_attacks[0]; DC_circuits_s420_attack=circuits_s420_centrality_nodes_attacks[1]; CC_circuits_s420_attack=circuits_s420_centrality_nodes_attacks[2]; NBC_circuits_s420_attack=circuits_s420_centrality_nodes_attacks[3];  MEB_circuits_s420_attack=circuits_s420_centrality_nodes_attacks[4]; MED_circuits_s420_attack=circuits_s420_centrality_nodes_attacks[5]; MEC_circuits_s420_attack=circuits_s420_centrality_nodes_attacks[6]; PR_circuits_s420_attack=circuits_s420_centrality_nodes_attacks[7]; DWT_circuits_s420_attack=circuits_s420_centrality_nodes_attacks[8]


BC_corruption_attack=""; DC_corruption_attack=""; CC_corruption_attack=""; NBC_corruption_attack="";  MEB_corruption_attack=""; MED_corruption_attack=""; MEC_corruption_attack=""; PR_corruption_attack=""; DWT_corruption_attack=""
corruption_centrality_nodes_attacks=[BC_corruption_attack, DC_corruption_attack, CC_corruption_attack, NBC_corruption_attack,  MEB_corruption_attack, MED_corruption_attack, MEC_corruption_attack, PR_corruption_attack, DWT_corruption_attack]
for i in range(len(corruption_centrality_nodes_attacks)):
    corruption_centrality_nodes_attacks[i]=attack_node(corruption, [x[0] for x in sorted((centrality_functions[i](corruption)).items(), key=lambda x: x[1], reverse=True)])
print("done with corruption; ", end='')
BC_corruption_attack=corruption_centrality_nodes_attacks[0]; DC_corruption_attack=corruption_centrality_nodes_attacks[1]; CC_corruption_attack=corruption_centrality_nodes_attacks[2]; NBC_corruption_attack=corruption_centrality_nodes_attacks[3];  MEB_corruption_attack=corruption_centrality_nodes_attacks[4]; MED_corruption_attack=corruption_centrality_nodes_attacks[5]; MEC_corruption_attack=corruption_centrality_nodes_attacks[6]; PR_corruption_attack=corruption_centrality_nodes_attacks[7]; DWT_corruption_attack=corruption_centrality_nodes_attacks[8]


BC_facebook_0_attack=""; DC_facebook_0_attack=""; CC_facebook_0_attack=""; NBC_facebook_0_attack="";  MEB_facebook_0_attack=""; MED_facebook_0_attack=""; MEC_facebook_0_attack=""; PR_facebook_0_attack=""; DWT_facebook_0_attack=""
facebook_0_centrality_nodes_attacks=[BC_facebook_0_attack, DC_facebook_0_attack, CC_facebook_0_attack, NBC_facebook_0_attack,  MEB_facebook_0_attack, MED_facebook_0_attack, MEC_facebook_0_attack, PR_facebook_0_attack, DWT_facebook_0_attack]
for i in range(len(facebook_0_centrality_nodes_attacks)):
    facebook_0_centrality_nodes_attacks[i]=attack_node(facebook_0, [x[0] for x in sorted((centrality_functions[i](facebook_0)).items(), key=lambda x: x[1], reverse=True)])
print("done with facebook_0; ", end='')
BC_facebook_0_attack=facebook_0_centrality_nodes_attacks[0]; DC_facebook_0_attack=facebook_0_centrality_nodes_attacks[1]; CC_facebook_0_attack=facebook_0_centrality_nodes_attacks[2]; NBC_facebook_0_attack=facebook_0_centrality_nodes_attacks[3];  MEB_facebook_0_attack=facebook_0_centrality_nodes_attacks[4]; MED_facebook_0_attack=facebook_0_centrality_nodes_attacks[5]; MEC_facebook_0_attack=facebook_0_centrality_nodes_attacks[6]; PR_facebook_0_attack=facebook_0_centrality_nodes_attacks[7]; DWT_facebook_0_attack=facebook_0_centrality_nodes_attacks[8]


BC_circuits_s838_attack=""; DC_circuits_s838_attack=""; CC_circuits_s838_attack=""; NBC_circuits_s838_attack="";  MEB_circuits_s838_attack=""; MED_circuits_s838_attack=""; MEC_circuits_s838_attack=""; PR_circuits_s838_attack=""; DWT_circuits_s838_attack=""
circuits_s838_centrality_nodes_attacks=[BC_circuits_s838_attack, DC_circuits_s838_attack, CC_circuits_s838_attack, NBC_circuits_s838_attack,  MEB_circuits_s838_attack, MED_circuits_s838_attack, MEC_circuits_s838_attack, PR_circuits_s838_attack, DWT_circuits_s838_attack]
for i in range(len(circuits_s838_centrality_nodes_attacks)):
    circuits_s838_centrality_nodes_attacks[i]=attack_node(circuits_s838, [x[0] for x in sorted((centrality_functions[i](circuits_s838)).items(), key=lambda x: x[1], reverse=True)])
print("done with circuits_s838; ", end='')
BC_circuits_s838_attack=circuits_s838_centrality_nodes_attacks[0]; DC_circuits_s838_attack=circuits_s838_centrality_nodes_attacks[1]; CC_circuits_s838_attack=circuits_s838_centrality_nodes_attacks[2]; NBC_circuits_s838_attack=circuits_s838_centrality_nodes_attacks[3];  MEB_circuits_s838_attack=circuits_s838_centrality_nodes_attacks[4]; MED_circuits_s838_attack=circuits_s838_centrality_nodes_attacks[5]; MEC_circuits_s838_attack=circuits_s838_centrality_nodes_attacks[6]; PR_circuits_s838_attack=circuits_s838_centrality_nodes_attacks[7]; DWT_circuits_s838_attack=circuits_s838_centrality_nodes_attacks[8]


BC_crime_net_attack=""; DC_crime_net_attack=""; CC_crime_net_attack=""; NBC_crime_net_attack="";  MEB_crime_net_attack=""; MED_crime_net_attack=""; MEC_crime_net_attack=""; PR_crime_net_attack=""; DWT_crime_net_attack=""
crime_net_centrality_nodes_attacks=[BC_crime_net_attack, DC_crime_net_attack, CC_crime_net_attack, NBC_crime_net_attack,  MEB_crime_net_attack, MED_crime_net_attack, MEC_crime_net_attack, PR_crime_net_attack, DWT_crime_net_attack]
for i in range(len(crime_net_centrality_nodes_attacks)):
    crime_net_centrality_nodes_attacks[i]=attack_node(crime_net, [x[0] for x in sorted((centrality_functions[i](crime_net)).items(), key=lambda x: x[1], reverse=True)])
print("done with crime_net; ", end='')
BC_crime_net_attack=crime_net_centrality_nodes_attacks[0]; DC_crime_net_attack=crime_net_centrality_nodes_attacks[1]; CC_crime_net_attack=crime_net_centrality_nodes_attacks[2]; NBC_crime_net_attack=crime_net_centrality_nodes_attacks[3];  MEB_crime_net_attack=crime_net_centrality_nodes_attacks[4]; MED_crime_net_attack=crime_net_centrality_nodes_attacks[5]; MEC_crime_net_attack=crime_net_centrality_nodes_attacks[6]; PR_crime_net_attack=crime_net_centrality_nodes_attacks[7]; DWT_crime_net_attack=crime_net_centrality_nodes_attacks[8]


BC_facebook_1684_attack=""; DC_facebook_1684_attack=""; CC_facebook_1684_attack=""; NBC_facebook_1684_attack="";  MEB_facebook_1684_attack=""; MED_facebook_1684_attack=""; MEC_facebook_1684_attack=""; PR_facebook_1684_attack=""; DWT_facebook_1684_attack=""
facebook_1684_centrality_nodes_attacks=[BC_facebook_1684_attack, DC_facebook_1684_attack, CC_facebook_1684_attack, NBC_facebook_1684_attack,  MEB_facebook_1684_attack, MED_facebook_1684_attack, MEC_facebook_1684_attack, PR_facebook_1684_attack, DWT_facebook_1684_attack]
for i in range(len(facebook_1684_centrality_nodes_attacks)):
    facebook_1684_centrality_nodes_attacks[i]=attack_node(facebook_1684, [x[0] for x in sorted((centrality_functions[i](facebook_1684)).items(), key=lambda x: x[1], reverse=True)])
done("facebook_1684")
BC_facebook_1684_attack=facebook_1684_centrality_nodes_attacks[0]; DC_facebook_1684_attack=facebook_1684_centrality_nodes_attacks[1]; CC_facebook_1684_attack=facebook_1684_centrality_nodes_attacks[2]; NBC_facebook_1684_attack=facebook_1684_centrality_nodes_attacks[3];  MEB_facebook_1684_attack=facebook_1684_centrality_nodes_attacks[4]; MED_facebook_1684_attack=facebook_1684_centrality_nodes_attacks[5]; MEC_facebook_1684_attack=facebook_1684_centrality_nodes_attacks[6]; PR_facebook_1684_attack=facebook_1684_centrality_nodes_attacks[7]; DWT_facebook_1684_attack=facebook_1684_centrality_nodes_attacks[8]

# BC_facebook_107_attack=""; DC_facebook_107_attack=""; CC_facebook_107_attack=""; NBC_facebook_107_attack="";  MEB_facebook_107_attack=""; MED_facebook_107_attack=""; MEC_facebook_107_attack=""; PR_facebook_107_attack=""; DWT_facebook_107_attack=""
# facebook_107_centrality_nodes_attacks=[BC_facebook_107_attack, DC_facebook_107_attack, CC_facebook_107_attack, NBC_facebook_107_attack,  MEB_facebook_107_attack, MED_facebook_107_attack, MEC_facebook_107_attack, PR_facebook_107_attack, DWT_facebook_107_attack]
# for i in range(len(facebook_107_centrality_nodes_attacks)):
#     facebook_107_centrality_nodes_attacks[i]=attack_node(facebook_107, [x[0] for x in sorted((centrality_functions[i](facebook_107)).items(), key=lambda x: x[1], reverse=True)])
# print("done with facebook_107; ", end='')
# for i in range(len(facebook_107_centrality_nodes_attacks)):
#     BC_facebook_107_attack=facebook_107_centrality_nodes_attacks[i]; DC_facebook_107_attack=facebook_107_centrality_nodes_attacks[i]; CC_facebook_107_attack=facebook_107_centrality_nodes_attacks[i]; NBC_facebook_107_attack=facebook_107_centrality_nodes_attacks[i];  MEB_facebook_107_attack=facebook_107_centrality_nodes_attacks[i]; MED_facebook_107_attack=facebook_107_centrality_nodes_attacks[i]; MEC_facebook_107_attack=facebook_107_centrality_nodes_attacks[i]; PR_facebook_107_attack=facebook_107_centrality_nodes_attacks[i]; DWT_facebook_107_attack=facebook_107_centrality_nodes_attacks[i]

done with bn_macaque_rhesus_brain_2; done with word_adjacencies; done with circuits_s208; done with bn_cat_mixed_species_brain_1; 

C:\Users\NEL Project\AppData\Local\Temp\ipykernel_5648\3576216100.py:131: RuntimeWarning: invalid value encountered in scalar divide
  proximity = sum_of_all_pairs_shortest_paths / sum_of_shortest_paths


done with facebook_414; done with facebook_686; done with polbooks; done with jazz; done with facebook_348; done with circuits_s420; done with corruption; done with facebook_0; done with circuits_s838; done with crime_net; done with  facebook_1684 ; 

#### PLCi from csv:

In [158]:
#running complex centralities:
PLCi_bn_macaque_rhesus_brain_2_attack = attack_node( bn_macaque_rhesus_brain_2, plci_bn_macaque_rhesus_brain_2_nodes)
done("bn_macaque_rhesus_brain_2")

PLCi_word_adjacencies_attack = attack_node( word_adjacencies, plci_word_adjacencies_nodes)
done("word_adjacencies")

PLCi_circuits_s208_attack = attack_node( circuits_s208, plci_circuits_s208_nodes)
done("circuits_s208")

PLCi_bn_cat_mixed_species_brain_1_attack = attack_node( bn_cat_mixed_species_brain_1, plci_bn_cat_mixed_species_brain_1_nodes)
done("bn_cat_mixed_species_brain_1")

PLCi_facebook_414_attack = attack_node( facebook_414, plci_facebook_414_nodes)
done("facebook_414")

PLCi_facebook_686_attack = attack_node( facebook_686, plci_facebook_686_nodes)
done("facebook_686")

PLCi_polbooks_attack = attack_node( polbooks, plci_polbooks_nodes)
done("polbooks")

PLCi_jazz_attack = attack_node( jazz, plci_jazz_nodes)
done("jazz")

PLCi_facebook_348_attack = attack_node( facebook_348, plci_facebook_348_nodes)
done("facebook_348")

PLCi_circuits_s420_attack = attack_node( circuits_s420, plci_circuits_s420_nodes)
done("circuits_s420")

PLCi_corruption_attack = attack_node( corruption, plci_corruption_nodes)
done("corruption")

PLCi_facebook_0_attack = attack_node( facebook_0, plci_facebook_0_nodes)
done("facebook_0")

PLCi_circuits_s838_attack = attack_node( circuits_s838, plci_circuits_s838_nodes)
done("circuits_s838")

PLCi_crime_net_attack = attack_node( crime_net, plci_crime_net_nodes)
done("crime_net")

PLCi_facebook_1684_attack = attack_node( facebook_1684, plci_facebook_1684_nodes)
done("facebook_1684")

# PLCi_facebook_107_attack = attack_node( facebook_107, plci_facebook_107_nodes)
# done("facebook_107")

done with  bn_macaque_rhesus_brain_2 ; done with  word_adjacencies ; done with  circuits_s208 ; done with  bn_cat_mixed_species_brain_1 ; done with  facebook_414 ; done with  facebook_686 ; done with  polbooks ; done with  jazz ; done with  facebook_348 ; done with  circuits_s420 ; done with  corruption ; done with  facebook_0 ; done with  circuits_s838 ; done with  crime_net ; done with  facebook_1684 ; 

#### random:

In [159]:
random_bn_macaque_rhesus_brain_2_attack = random_attack_node( bn_macaque_rhesus_brain_2)
done("bn_macaque_rhesus_brain_2")

random_word_adjacencies_attack = random_attack_node( word_adjacencies)
done("word_adjacencies")

random_circuits_s208_attack = random_attack_node( circuits_s208)
done("circuits_s208")

random_bn_cat_mixed_species_brain_1_attack = random_attack_node( bn_cat_mixed_species_brain_1)
done("bn_cat_mixed_species_brain_1")

random_facebook_414_attack = random_attack_node( facebook_414)
done("facebook_414")

random_facebook_686_attack = random_attack_node( facebook_686)
done("facebook_686")

random_polbooks_attack = random_attack_node( polbooks)
done("polbooks")

random_jazz_attack = random_attack_node( jazz)
done("jazz")

random_facebook_348_attack = random_attack_node( facebook_348)
done("facebook_348")

random_circuits_s420_attack = random_attack_node( circuits_s420)
done("circuits_s420")

random_corruption_attack = random_attack_node( corruption)
done("corruption")

random_facebook_0_attack = random_attack_node( facebook_0)
done("facebook_0")

random_circuits_s838_attack = random_attack_node( circuits_s838)
done("circuits_s838")

random_crime_net_attack = random_attack_node( crime_net)
done("crime_net")

random_facebook_1684_attack = random_attack_node( facebook_1684)
done("facebook_1684")

# random_facebook_107_attack = random_attack_node( facebook_107)
# done("facebook_107")

done with  bn_macaque_rhesus_brain_2 ; done with  word_adjacencies ; done with  circuits_s208 ; done with  bn_cat_mixed_species_brain_1 ; done with  facebook_414 ; done with  facebook_686 ; done with  polbooks ; done with  jazz ; done with  facebook_348 ; done with  circuits_s420 ; done with  corruption ; done with  facebook_0 ; done with  circuits_s838 ; done with  crime_net ; done with  facebook_1684 ; 

#### greedy:

In [160]:
greedy_bn_macaque_rhesus_brain_2_attack = greedy_attack_node(bn_macaque_rhesus_brain_2)
done("bn_macaque_rhesus_brain_2")

greedy_word_adjacencies_attack = greedy_attack_node(word_adjacencies)
done("word_adjacencies")

greedy_circuits_s208_attack = greedy_attack_node(circuits_s208)
done("circuits_s208")

greedy_bn_cat_mixed_species_brain_1_attack = greedy_attack_node(bn_cat_mixed_species_brain_1)
done("bn_cat_mixed_species_brain_1")

greedy_facebook_414_attack = greedy_attack_node(facebook_414)
done("facebook_414")

greedy_facebook_686_attack = greedy_attack_node(facebook_686)
done("facebook_686")

greedy_polbooks_attack = greedy_attack_node(polbooks)
done("polbooks")

greedy_jazz_attack = greedy_attack_node(jazz)
done("jazz")

greedy_facebook_348_attack = greedy_attack_node(facebook_348)
done("facebook_348")

greedy_circuits_s420_attack = greedy_attack_node(circuits_s420)
done("circuits_s420")

greedy_corruption_attack = greedy_attack_node(corruption)
done("corruption")

greedy_facebook_0_attack = greedy_attack_node(facebook_0)
done("facebook_0")

greedy_circuits_s838_attack = greedy_attack_node(circuits_s838)
done("circuits_s838")

greedy_crime_net_attack = greedy_attack_node(crime_net)
done("crime_net")

greedy_facebook_1684_attack = greedy_attack_node(facebook_1684)
done("facebook_1684")

# greedy_facebook_107_attack = greedy_attack_node(facebook_107)
# done("facebook_107")

done with  bn_macaque_rhesus_brain_2 ; done with  word_adjacencies ; done with  circuits_s208 ; done with  bn_cat_mixed_species_brain_1 ; done with  facebook_414 ; done with  facebook_686 ; done with  polbooks ; done with  jazz ; done with  facebook_348 ; done with  circuits_s420 ; done with  corruption ; done with  facebook_0 ; done with  circuits_s838 ; done with  crime_net ; done with  facebook_1684 ; 

### community-based node attacks:

We will use the top 5 centralities to study here. We will also compute the time it takes to compute each centrality function for the largest network to study their computational efficiency along wth their accuracy of attack.

In [13]:
BC_time=time.time(); BC(facebook_1684); BC_time=time.time()-BC_time
done(f"BC in {BC_time} seconds")
NBC_time=time.time(); NBC(facebook_1684); NBC_time=time.time()-NBC_time
done(f"NBC in {NBC_time} seconds")
MEB_time=time.time(); MEB(facebook_1684); MEB_time=time.time()-MEB_time
done(f"MEB in {MEB_time} seconds")
MED_time=time.time(); MED(facebook_1684); MED_time=time.time()-MED_time
done(f"MED in {MED_time} seconds")
MEC_time=time.time(); MEC(facebook_1684); MEC_time=time.time()-MEC_time
done(f"MEC in {MEC_time} seconds")

done with  BC in 6.5745134353637695 seconds ; done with  NBC in 5.518618822097778 seconds ; done with  MEB in 6.8793675899505615 seconds ; done with  MED in 0.014960765838623047 seconds ; done with  MEC in 1.7121260166168213 seconds ; 

This is the partition of the networks into communities, louvain 10 seeds, pick the one with highest modularity (computed once to not lose the seed)

In [111]:
community_partition_bn_macaque_rhesus_brain_2 = split_to_communities(bn_macaque_rhesus_brain_2)
community_partition_word_adjacencies = split_to_communities(word_adjacencies)
community_partition_circuits_s208 = split_to_communities(circuits_s208)
community_partition_bn_cat_mixed_species_brain_1 = split_to_communities(bn_cat_mixed_species_brain_1)
community_partition_facebook_414 = split_to_communities(facebook_414)
community_partition_facebook_686 = split_to_communities(facebook_686)
community_partition_polbooks = split_to_communities(polbooks)
community_partition_jazz = split_to_communities(jazz)
community_partition_facebook_348 = split_to_communities(facebook_348)
community_partition_circuits_s420 = split_to_communities(circuits_s420)
community_partition_corruption = split_to_communities(corruption)
community_partition_facebook_0 = split_to_communities(facebook_0)
community_partition_circuits_s838 = split_to_communities(circuits_s838)
community_partition_crime_net = split_to_communities(crime_net)
community_partition_facebook_1684 = split_to_communities(facebook_1684)

#save them in df
partitions_df = pd.DataFrame(columns=["bn_macaque_rhesus_brain_2", "word_adjacencies", "circuits_s208", "bn_cat_mixed_species_brain_1", "facebook_414", "facebook_686", "polbooks", "jazz", "facebook_348", "circuits_s420", "corruption", "facebook_0", "circuits_s838", "crime_net", "facebook_1684"])
partitions["bn_macaque_rhesus_brain_2"] = str(community_partition_bn_macaque_rhesus_brain_2); partitions["word_adjacencies"] = str(community_partition_word_adjacencies); partitions["circuits_s208"] = str(community_partition_circuits_s208); partitions["bn_cat_mixed_species_brain_1"] = str(community_partition_bn_cat_mixed_species_brain_1); partitions["facebook_414"] = str(community_partition_facebook_414); partitions["facebook_686"] = str(community_partition_facebook_686); partitions["polbooks"] = str(community_partition_polbooks); partitions["jazz"] = str(community_partition_jazz); partitions["facebook_348"] = str(community_partition_facebook_348); partitions["circuits_s420"] = str(community_partition_circuits_s420); partitions["corruption"] = str(community_partition_corruption); partitions["facebook_0"] = str(community_partition_facebook_0); partitions["circuits_s838"] = str(community_partition_circuits_s838); partitions["crime_net"] = str(community_partition_crime_net); partitions["facebook_1684"] = str(community_partition_facebook_1684)
partitions.to_csv("../results/community_partitions.csv")

In [12]:
centrality_functions_for_community_based = [BC, NBC, MEB, MEC, MED]

BC_bn_macaque_rhesus_brain_2_community_attack=""; NBC_bn_macaque_rhesus_brain_2_community_attack=""; MEB_bn_macaque_rhesus_brain_2_community_attack=""; MEC_bn_macaque_rhesus_brain_2_community_attack=""; MED_bn_macaque_rhesus_brain_2_community_attack=""
bn_macaque_rhesus_brain_2_community_attacks=[BC_bn_cat_mixed_species_brain_1_community_attack, NBC_bn_cat_mixed_species_brain_1_community_attack, MEB_bn_cat_mixed_species_brain_1_community_attack, MEC_bn_cat_mixed_species_brain_1_community_attack, MED_bn_cat_mixed_species_brain_1_community_attack]
for i in range(len(bn_macaque_rhesus_brain_2_community_attacks)):
    bn_macaque_rhesus_brain_2_community_attacks[i]=community_attack_node(bn_cat_mixed_species_brain_1, community_partition_bn_cat_mixed_species_brain_1, centrality_functions_for_community_based[i])
BC_bn_macaque_rhesus_brain_2_community_attack=bn_macaque_rhesus_brain_2_community_attacks[0]; NBC_bn_macaque_rhesus_brain_2_community_attack=bn_macaque_rhesus_brain_2_community_attacks[1]; MEB_bn_macaque_rhesus_brain_2_community_attack=bn_macaque_rhesus_brain_2_community_attacks[2]; MEC_bn_macaque_rhesus_brain_2_community_attack=bn_macaque_rhesus_brain_2_community_attacks[3]; MED_bn_macaque_rhesus_brain_2_community_attack=bn_macaque_rhesus_brain_2_community_attacks[4]
done("bn_cat_mixed_species_brain_1")

BC_word_adjacencies_community_attack="";NBC_word_adjacencies_community_attack="";MEB_word_adjacencies_community_attack="";MEC_word_adjacencies_community_attack="";MED_word_adjacencies_community_attack=""
word_adjacencies_community_attacks=[BC_word_adjacencies_community_attack, NBC_word_adjacencies_community_attack, MEB_word_adjacencies_community_attack, MEC_word_adjacencies_community_attack, MED_word_adjacencies_community_attack]
for i in range(len(word_adjacencies_community_attacks)):
    word_adjacencies_community_attacks[i]=community_attack_node(word_adjacencies, community_partition_word_adjacencies, centrality_functions_for_community_based[i])
BC_word_adjacencies_community_attack=word_adjacencies_community_attacks[0]; NBC_word_adjacencies_community_attack=word_adjacencies_community_attacks[1]; MEB_word_adjacencies_community_attack=word_adjacencies_community_attacks[2]; MEC_word_adjacencies_community_attack=word_adjacencies_community_attacks[3]; MED_word_adjacencies_community_attack=word_adjacencies_community_attacks[4]
done("word_adjacencies")

BC_circuits_s208_community_attack="";NBC_circuits_s208_community_attack="";MEB_circuits_s208_community_attack="";MEC_circuits_s208_community_attack="";MED_circuits_s208_community_attack=""
circuits_s208_community_attacks=[BC_circuits_s208_community_attack, NBC_circuits_s208_community_attack, MEB_circuits_s208_community_attack, MEC_circuits_s208_community_attack, MED_circuits_s208_community_attack]
for i in range(len(circuits_s208_community_attacks)):
    circuits_s208_community_attacks[i]=community_attack_node(circuits_s208, community_partition_circuits_s208, centrality_functions_for_community_based[i])
BC_circuits_s208_community_attack=circuits_s208_community_attacks[0]; NBC_circuits_s208_community_attack=circuits_s208_community_attacks[1]; MEB_circuits_s208_community_attack=circuits_s208_community_attacks[2]; MEC_circuits_s208_community_attack=circuits_s208_community_attacks[3]; MED_circuits_s208_community_attack=circuits_s208_community_attacks[4]
done("circuits_s208")

BC_bn_cat_mixed_species_brain_1_community_attack="";NBC_bn_cat_mixed_species_brain_1_community_attack="";MEB_bn_cat_mixed_species_brain_1_community_attack="";MEC_bn_cat_mixed_species_brain_1_community_attack="";MED_bn_cat_mixed_species_brain_1_community_attack=""
bn_cat_mixed_species_brain_1_community_attacks=[BC_bn_cat_mixed_species_brain_1_community_attack, NBC_bn_cat_mixed_species_brain_1_community_attack, MEB_bn_cat_mixed_species_brain_1_community_attack, MEC_bn_cat_mixed_species_brain_1_community_attack, MED_bn_cat_mixed_species_brain_1_community_attack]
for i in range(len(bn_cat_mixed_species_brain_1_community_attacks)):
    bn_cat_mixed_species_brain_1_community_attacks[i]=community_attack_node(bn_cat_mixed_species_brain_1, community_partition_bn_cat_mixed_species_brain_1, centrality_functions_for_community_based[i])
BC_bn_cat_mixed_species_brain_1_community_attack=bn_cat_mixed_species_brain_1_community_attacks[0]; NBC_bn_cat_mixed_species_brain_1_community_attack=bn_cat_mixed_species_brain_1_community_attacks[1]; MEB_bn_cat_mixed_species_brain_1_community_attack=bn_cat_mixed_species_brain_1_community_attacks[2]; MEC_bn_cat_mixed_species_brain_1_community_attack=bn_cat_mixed_species_brain_1_community_attacks[3]; MED_bn_cat_mixed_species_brain_1_community_attack=bn_cat_mixed_species_brain_1_community_attacks[4]

done("bn_cat_mixed_species_brain_1")

BC_facebook_414_community_attack="";NBC_facebook_414_community_attack="";MEB_facebook_414_community_attack="";MEC_facebook_414_community_attack="";MED_facebook_414_community_attack=""
facebook_414_community_attacks=[BC_facebook_414_community_attack, NBC_facebook_414_community_attack, MEB_facebook_414_community_attack, MEC_facebook_414_community_attack, MED_facebook_414_community_attack]
for i in range(len(facebook_414_community_attacks)):
    facebook_414_community_attacks[i]=community_attack_node(facebook_414, community_partition_facebook_414, centrality_functions_for_community_based[i])
BC_facebook_414_community_attack=facebook_414_community_attacks[0]; NBC_facebook_414_community_attack=facebook_414_community_attacks[1]; MEB_facebook_414_community_attack=facebook_414_community_attacks[2]; MEC_facebook_414_community_attack=facebook_414_community_attacks[3]; MED_facebook_414_community_attack=facebook_414_community_attacks[4]
done("facebook_414")

BC_facebook_686_community_attack="";NBC_facebook_686_community_attack="";MEB_facebook_686_community_attack="";MEC_facebook_686_community_attack="";MED_facebook_686_community_attack=""
facebook_686_community_attacks=[BC_facebook_686_community_attack, NBC_facebook_686_community_attack, MEB_facebook_686_community_attack, MEC_facebook_686_community_attack, MED_facebook_686_community_attack]
for i in range(len(facebook_686_community_attacks)):
    facebook_686_community_attacks[i]=community_attack_node(facebook_686, community_partition_facebook_686, centrality_functions_for_community_based[i])
BC_facebook_686_community_attack=facebook_686_community_attacks[0]; NBC_facebook_686_community_attack=facebook_686_community_attacks[1]; MEB_facebook_686_community_attack=facebook_686_community_attacks[2]; MEC_facebook_686_community_attack=facebook_686_community_attacks[3]; MED_facebook_686_community_attack=facebook_686_community_attacks[4]
done("facebook_686")

BC_polbooks_community_attack="";NBC_polbooks_community_attack="";MEB_polbooks_community_attack="";MEC_polbooks_community_attack="";MED_polbooks_community_attack=""
polbooks_community_attacks=[BC_polbooks_community_attack, NBC_polbooks_community_attack, MEB_polbooks_community_attack, MEC_polbooks_community_attack, MED_polbooks_community_attack]
for i in range(len(polbooks_community_attacks)):
    polbooks_community_attacks[i]=community_attack_node(polbooks, community_partition_polbooks, centrality_functions_for_community_based[i])
BC_polbooks_community_attack=polbooks_community_attacks[0]; NBC_polbooks_community_attack=polbooks_community_attacks[1]; MEB_polbooks_community_attack=polbooks_community_attacks[2]; MEC_polbooks_community_attack=polbooks_community_attacks[3]; MED_polbooks_community_attack=polbooks_community_attacks[4]
done("polbooks")

BC_jazz_community_attack="";NBC_jazz_community_attack="";MEB_jazz_community_attack="";MEC_jazz_community_attack="";MED_jazz_community_attack=""
jazz_community_attacks=[BC_jazz_community_attack, NBC_jazz_community_attack, MEB_jazz_community_attack, MEC_jazz_community_attack, MED_jazz_community_attack]
for i in range(len(jazz_community_attacks)):
    jazz_community_attacks[i]=community_attack_node(jazz, community_partition_jazz, centrality_functions_for_community_based[i])
BC_jazz_community_attack=jazz_community_attacks[0]; NBC_jazz_community_attack=jazz_community_attacks[1]; MEB_jazz_community_attack=jazz_community_attacks[2]; MEC_jazz_community_attack=jazz_community_attacks[3]; MED_jazz_community_attack=jazz_community_attacks[4]
done("jazz")

BC_facebook_348_community_attack="";NBC_facebook_348_community_attack="";MEB_facebook_348_community_attack="";MEC_facebook_348_community_attack="";MED_facebook_348_community_attack=""
facebook_348_community_attacks=[BC_facebook_348_community_attack, NBC_facebook_348_community_attack, MEB_facebook_348_community_attack, MEC_facebook_348_community_attack, MED_facebook_348_community_attack]
for i in range(len(facebook_348_community_attacks)):
    facebook_348_community_attacks[i]=community_attack_node(facebook_348, community_partition_facebook_348, centrality_functions_for_community_based[i])
BC_facebook_348_community_attack=facebook_348_community_attacks[0]; NBC_facebook_348_community_attack=facebook_348_community_attacks[1]; MEB_facebook_348_community_attack=facebook_348_community_attacks[2]; MEC_facebook_348_community_attack=facebook_348_community_attacks[3]; MED_facebook_348_community_attack=facebook_348_community_attacks[4]
done("facebook_348")

BC_circuits_s420_community_attack="";NBC_circuits_s420_community_attack="";MEB_circuits_s420_community_attack="";MEC_circuits_s420_community_attack="";MED_circuits_s420_community_attack=""
circuits_s420_community_attacks=[BC_circuits_s420_community_attack, NBC_circuits_s420_community_attack, MEB_circuits_s420_community_attack, MEC_circuits_s420_community_attack, MED_circuits_s420_community_attack]
for i in range(len(circuits_s420_community_attacks)):
    circuits_s420_community_attacks[i]=community_attack_node(circuits_s420, community_partition_circuits_s420, centrality_functions_for_community_based[i])
BC_circuits_s420_community_attack=circuits_s420_community_attacks[0]; NBC_circuits_s420_community_attack=circuits_s420_community_attacks[1]; MEB_circuits_s420_community_attack=circuits_s420_community_attacks[2]; MEC_circuits_s420_community_attack=circuits_s420_community_attacks[3]; MED_circuits_s420_community_attack=circuits_s420_community_attacks[4]
done("circuits_s420")

BC_corruption_community_attack="";NBC_corruption_community_attack="";MEB_corruption_community_attack="";MEC_corruption_community_attack="";MED_corruption_community_attack=""
corruption_community_attacks=[BC_corruption_community_attack, NBC_corruption_community_attack, MEB_corruption_community_attack, MEC_corruption_community_attack, MED_corruption_community_attack]
for i in range(len(corruption_community_attacks)):
    corruption_community_attacks[i]=community_attack_node(corruption, community_partition_corruption, centrality_functions_for_community_based[i])
BC_corruption_community_attack=corruption_community_attacks[0]; NBC_corruption_community_attack=corruption_community_attacks[1]; MEB_corruption_community_attack=corruption_community_attacks[2]; MEC_corruption_community_attack=corruption_community_attacks[3]; MED_corruption_community_attack=corruption_community_attacks[4]

done("corruption")

BC_facebook_0_community_attack="";NBC_facebook_0_community_attack="";MEB_facebook_0_community_attack="";MEC_facebook_0_community_attack="";MED_facebook_0_community_attack=""
facebook_0_community_attacks=[BC_facebook_0_community_attack, NBC_facebook_0_community_attack, MEB_facebook_0_community_attack, MEC_facebook_0_community_attack, MED_facebook_0_community_attack]
for i in range(len(facebook_0_community_attacks)):
    facebook_0_community_attacks[i]=community_attack_node(facebook_0, community_partition_facebook_0, centrality_functions_for_community_based[i])
BC_facebook_0_community_attack=facebook_0_community_attacks[0]; NBC_facebook_0_community_attack=facebook_0_community_attacks[1]; MEB_facebook_0_community_attack=facebook_0_community_attacks[2]; MEC_facebook_0_community_attack=facebook_0_community_attacks[3]; MED_facebook_0_community_attack=facebook_0_community_attacks[4]

done("facebook_0")

BC_circuits_s838_community_attack="";NBC_circuits_s838_community_attack="";MEB_circuits_s838_community_attack="";MEC_circuits_s838_community_attack="";MED_circuits_s838_community_attack=""
circuits_s838_community_attacks=[BC_circuits_s838_community_attack, NBC_circuits_s838_community_attack, MEB_circuits_s838_community_attack, MEC_circuits_s838_community_attack, MED_circuits_s838_community_attack]
for i in range(len(circuits_s838_community_attacks)):
    circuits_s838_community_attacks[i]=community_attack_node(circuits_s838, community_partition_circuits_s838, centrality_functions_for_community_based[i])
BC_circuits_s838_community_attack=circuits_s838_community_attacks[0]; NBC_circuits_s838_community_attack=circuits_s838_community_attacks[1]; MEB_circuits_s838_community_attack=circuits_s838_community_attacks[2]; MEC_circuits_s838_community_attack=circuits_s838_community_attacks[3]; MED_circuits_s838_community_attack=circuits_s838_community_attacks[4]
done("circuits_s838")

BC_crime_net_community_attack="";NBC_crime_net_community_attack="";MEB_crime_net_community_attack="";MEC_crime_net_community_attack="";MED_crime_net_community_attack=""
crime_net_community_attacks=[BC_crime_net_community_attack, NBC_crime_net_community_attack, MEB_crime_net_community_attack, MEC_crime_net_community_attack, MED_crime_net_community_attack]
for i in range(len(crime_net_community_attacks)):
    crime_net_community_attacks[i]=community_attack_node(crime_net, community_partition_crime_net, centrality_functions_for_community_based[i])
BC_crime_net_community_attack=crime_net_community_attacks[0]; NBC_crime_net_community_attack=crime_net_community_attacks[1]; MEB_crime_net_community_attack=crime_net_community_attacks[2]; MEC_crime_net_community_attack=crime_net_community_attacks[3]; MED_crime_net_community_attack=crime_net_community_attacks[4]
done("crime_net")

BC_facebook_1684_community_attack="";NBC_facebook_1684_community_attack="";MEB_facebook_1684_community_attack="";MEC_facebook_1684_community_attack="";MED_facebook_1684_community_attack=""
facebook_1684_community_attacks=[BC_facebook_1684_community_attack, NBC_facebook_1684_community_attack, MEB_facebook_1684_community_attack, MEC_facebook_1684_community_attack, MED_facebook_1684_community_attack]
for i in range(len(facebook_1684_community_attacks)):
    facebook_1684_community_attacks[i]=community_attack_node(facebook_1684, community_partition_facebook_1684, centrality_functions_for_community_based[i])
BC_facebook_1684_community_attack=facebook_1684_community_attacks[0]; NBC_facebook_1684_community_attack=facebook_1684_community_attacks[1]; MEB_facebook_1684_community_attack=facebook_1684_community_attacks[2]; MEC_facebook_1684_community_attack=facebook_1684_community_attacks[3]; MED_facebook_1684_community_attack=facebook_1684_community_attacks[4]
done("facebook_1684")



done with  bn_cat_mixed_species_brain_1 ; done with  word_adjacencies ; done with  circuits_s208 ; done with  bn_cat_mixed_species_brain_1 ; done with  facebook_414 ; done with  facebook_686 ; done with  polbooks ; done with  jazz ; done with  facebook_348 ; done with  circuits_s420 ; done with  corruption ; done with  facebook_0 ; done with  circuits_s838 ; done with  crime_net ; done with  facebook_1684 ; 

### edge-targeted attacks:

In [253]:
# attack on betweenness centrality
BC_bn_macaque_rhesus_brain_2_edge_attack = attack_edge(bn_macaque_rhesus_brain_2, [x[0] for x in sorted((nx.edge_betweenness_centrality(bn_macaque_rhesus_brain_2)).items(), key=lambda x: x[1], reverse=True)])
done("bn_macaque_rhesus_brain_2")
BC_word_adjacencies_edge_attack = attack_edge(word_adjacencies, [x[0] for x in sorted((nx.edge_betweenness_centrality(word_adjacencies)).items(), key=lambda x: x[1], reverse=True)])
done("word_adjacencies")
BC_circuits_s208_edge_attack = attack_edge(circuits_s208, [x[0] for x in sorted((nx.edge_betweenness_centrality(circuits_s208)).items(), key=lambda x: x[1], reverse=True)])
done("circuits_s208")
BC_bn_cat_mixed_species_brain_1_edge_attack = attack_edge(bn_cat_mixed_species_brain_1, [x[0] for x in sorted((nx.edge_betweenness_centrality(bn_cat_mixed_species_brain_1)).items(), key=lambda x: x[1], reverse=True)])
done("bn_cat_mixed_species_brain_1")
BC_facebook_414_edge_attack = attack_edge(facebook_414, [x[0] for x in sorted((nx.edge_betweenness_centrality(facebook_414)).items(), key=lambda x: x[1], reverse=True)])
done("facebook_414")
BC_facebook_686_edge_attack = attack_edge(facebook_686, [x[0] for x in sorted((nx.edge_betweenness_centrality(facebook_686)).items(), key=lambda x: x[1], reverse=True)])
done("facebook_686")
BC_polbooks_edge_attack = attack_edge(polbooks, [x[0] for x in sorted((nx.edge_betweenness_centrality(polbooks)).items(), key=lambda x: x[1], reverse=True)])
done("polbooks")
BC_jazz_edge_attack = attack_edge(jazz, [x[0] for x in sorted((nx.edge_betweenness_centrality(jazz)).items(), key=lambda x: x[1], reverse=True)])
done("jazz")
BC_facebook_348_edge_attack = attack_edge(facebook_348, [x[0] for x in sorted((nx.edge_betweenness_centrality(facebook_348)).items(), key=lambda x: x[1], reverse=True)])
done("facebook_348")
BC_circuits_s420_edge_attack = attack_edge(circuits_s420, [x[0] for x in sorted((nx.edge_betweenness_centrality(circuits_s420)).items(), key=lambda x: x[1], reverse=True)])
done("circuits_s420")
BC_corruption_edge_attack = attack_edge(corruption, [x[0] for x in sorted((nx.edge_betweenness_centrality(corruption)).items(), key=lambda x: x[1], reverse=True)])
done("corruption")
BC_facebook_0_edge_attack = attack_edge(facebook_0, [x[0] for x in sorted((nx.edge_betweenness_centrality(facebook_0)).items(), key=lambda x: x[1], reverse=True)])
done("facebook_0")
BC_circuits_s838_edge_attack = attack_edge(circuits_s838, [x[0] for x in sorted((nx.edge_betweenness_centrality(circuits_s838)).items(), key=lambda x: x[1], reverse=True)])
done("circuits_s838")
BC_crime_net_edge_attack = attack_edge(crime_net, [x[0] for x in sorted((nx.edge_betweenness_centrality(crime_net)).items(), key=lambda x: x[1], reverse=True)])
done("crime_net")
BC_facebook_1684_edge_attack = attack_edge(facebook_1684, [x[0] for x in sorted((nx.edge_betweenness_centrality(facebook_1684)).items(), key=lambda x: x[1], reverse=True)])
done("facebook_1684")


done with  bn_macaque_rhesus_brain_2 ; done with  word_adjacencies ; done with  circuits_s208 ; done with  bn_cat_mixed_species_brain_1 ; done with  facebook_414 ; done with  facebook_686 ; done with  polbooks ; done with  jazz ; done with  facebook_348 ; done with  circuits_s420 ; done with  corruption ; done with  facebook_0 ; done with  circuits_s838 ; done with  crime_net ; done with  facebook_1684 ; 

In [299]:
# NBC_bn_macaque_rhesus_brain_2_edge_attack = attack_edge(bn_macaque_rhesus_brain_2, [x[0] for x in sorted((nx.edge_load_centrality(bn_macaque_rhesus_brain_2)).items(), key=lambda x: x[1], reverse=True)])
# done("bn_macaque_rhesus_brain_2")
# NBC_word_adjacencies_edge_attack = attack_edge(word_adjacencies, [x[0] for x in sorted(nx.edge_load_centrality(word_adjacencies).items(), key=lambda x: x[1], reverse=True)])
# done("word_adjacencies")
# NBC_circuits_s208_edge_attack = attack_edge(circuits_s208, [x[0] for x in sorted(nx.edge_load_centrality(circuits_s208).items(), key=lambda x: x[1], reverse=True)])
# done("circuits_s208")
# NBC_bn_cat_mixed_species_brain_1_edge_attack = attack_edge(bn_cat_mixed_species_brain_1, [x[0] for x in sorted(nx.edge_load_centrality(bn_cat_mixed_species_brain_1).items(), key=lambda x: x[1], reverse=True)])
# done("bn_cat_mixed_species_brain_1")
# NBC_facebook_414_edge_attack = attack_edge(facebook_414, [x[0] for x in sorted(nx.edge_load_centrality(facebook_414).items(), key=lambda x: x[1], reverse=True)])
# done("facebook_414")
# NBC_facebook_686_edge_attack = attack_edge(facebook_686, [x[0] for x in sorted(nx.edge_load_centrality(facebook_686).items(), key=lambda x: x[1], reverse=True)])
# done("facebook_686")
# NBC_polbooks_edge_attack = attack_edge(polbooks, [x[0] for x in sorted(nx.edge_load_centrality(polbooks).items(), key=lambda x: x[1], reverse=True)])
# done("polbooks")
# NBC_jazz_edge_attack = attack_edge(jazz, [x[0] for x in sorted(nx.edge_load_centrality(jazz).items(), key=lambda x: x[1], reverse=True)])
# done("jazz")
# NBC_facebook_348_edge_attack = attack_edge(facebook_348, [x[0] for x in sorted(nx.edge_load_centrality(facebook_348).items(), key=lambda x: x[1], reverse=True)])
# done("facebook_348")
# NBC_circuits_s420_edge_attack = attack_edge(circuits_s420, [x[0] for x in sorted(nx.edge_load_centrality(circuits_s420).items(), key=lambda x: x[1], reverse=True)])
# done("circuits_s420")
# NBC_corruption_edge_attack = attack_edge(corruption, [x[0] for x in sorted(nx.edge_load_centrality(corruption).items(), key=lambda x: x[1], reverse=True)])
# done("corruption")
# NBC_facebook_0_edge_attack = attack_edge(facebook_0, [x[0] for x in sorted(nx.edge_load_centrality(facebook_0).items(), key=lambda x: x[1], reverse=True)])
# done("facebook_0")
# NBC_circuits_s838_edge_attack = attack_edge(circuits_s838, [x[0] for x in sorted(nx.edge_load_centrality(circuits_s838).items(), key=lambda x: x[1], reverse=True)])
# done("circuits_s838")
# NBC_crime_net_edge_attack = attack_edge(crime_net, [x[0] for x in sorted(nx.edge_load_centrality(crime_net).items(), key=lambda x: x[1], reverse=True)])
# done("crime_net")



greedy_bn_macaque_rhesus_brain_2_edge_attack = greedy_attack_node_edges(bn_macaque_rhesus_brain_2)
done("bn_macaque_rhesus_brain_2")
greedy_word_adjacencies_edge_attack = greedy_attack_node_edges(word_adjacencies)
done("word_adjacencies")
greedy_circuits_s208_edge_attack = greedy_attack_node_edges(circuits_s208)
done("circuits_s208")
greedy_bn_cat_mixed_species_brain_1_edge_attack = greedy_attack_node_edges(bn_cat_mixed_species_brain_1)
done("bn_cat_mixed_species_brain_1")
greedy_facebook_414_edge_attack = greedy_attack_node_edges(facebook_414)
done("facebook_414")
greedy_facebook_686_edge_attack = greedy_attack_node_edges(facebook_686)
done("facebook_686")
greedy_polbooks_edge_attack = greedy_attack_node_edges(polbooks)
done("polbooks")
greedy_jazz_edge_attack = greedy_attack_node_edges(jazz)
done("jazz")
greedy_facebook_348_edge_attack = greedy_attack_node_edges(facebook_348)
done("facebook_348")
greedy_circuits_s420_edge_attack = greedy_attack_node_edges(circuits_s420)
done("circuits_s420")
greedy_corruption_edge_attack = greedy_attack_node_edges(corruption)
done("corruption")
greedy_facebook_0_edge_attack = greedy_attack_node_edges(facebook_0)
done("facebook_0")
greedy_circuits_s838_edge_attack = greedy_attack_node_edges(circuits_s838)
done("circuits_s838")
greedy_crime_net_edge_attack = greedy_attack_node_edges(crime_net)
done("crime_net")
# greedy_facebook_1684_edge_attack = greedy_attack_node_edges(facebook_1684)
# done("facebook_1684")


done with  bn_macaque_rhesus_brain_2 ; done with  word_adjacencies ; done with  circuits_s208 ; done with  bn_cat_mixed_species_brain_1 ; done with  facebook_414 ; done with  facebook_686 ; done with  polbooks ; done with  jazz ; done with  facebook_348 ; done with  circuits_s420 ; done with  corruption ; done with  facebook_0 ; done with  circuits_s838 ; done with  crime_net ; 

In [289]:
random_bn_macaque_rhesus_brain_2_edge_attack = random_attack_edge(bn_macaque_rhesus_brain_2)
done("bn_macaque_rhesus_brain_2")
random_word_adjacencies_edge_attack = random_attack_edge(word_adjacencies)
done("word_adjacencies")
random_circuits_s208_edge_attack = random_attack_edge(circuits_s208)
done("circuits_s208")
random_bn_cat_mixed_species_brain_1_edge_attack = random_attack_edge(bn_cat_mixed_species_brain_1)
done("bn_cat_mixed_species_brain_1")
random_facebook_414_edge_attack = random_attack_edge(facebook_414)
done("facebook_414")
random_facebook_686_edge_attack = random_attack_edge(facebook_686)
done("facebook_686")
random_polbooks_edge_attack = random_attack_edge(polbooks)
done("polbooks")
random_jazz_edge_attack = random_attack_edge(jazz)
done("jazz")
random_facebook_348_edge_attack = random_attack_edge(facebook_348)
done("facebook_348")
random_circuits_s420_edge_attack = random_attack_edge(circuits_s420)
done("circuits_s420")
random_corruption_edge_attack = random_attack_edge(corruption)
done("corruption")
random_facebook_0_edge_attack = random_attack_edge(facebook_0)
done("facebook_0")
random_circuits_s838_edge_attack = random_attack_edge(circuits_s838)
done("circuits_s838")
random_crime_net_edge_attack = random_attack_edge(crime_net)
done("crime_net")
# random_facebook_1684_edge_attack = random_attack_edge(facebook_1684)
# done("facebook_1684")

done with  bn_macaque_rhesus_brain_2 ; done with  word_adjacencies ; done with  circuits_s208 ; Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\NEL Project\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\NEL Project\AppData\Local\Temp\ipykernel_5648\1451417382.py", line 7, in <module>
    random_bn_cat_mixed_species_brain_1_edge_attack = random_attack_edge(bn_cat_mixed_species_brain_1)
                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\NEL Project\AppData\Local\Temp\ipykernel_5648\3342696101.py", line 75, in random_attack_edge
    return attack_edge(G, edges)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\NEL Project\AppData\Local\Temp\ipykernel_5648\3342696101.py", line 46, in attack_edge
    e = nx.algorithms.global_efficiency(G)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<class 'networkx.utils.decorators.argmap'> compilation 12", line 4, in argmap_global_eff

In [ ]:
community_BC_bn_macaque_rhesus_brain_2_edge_attack = community_attack_edge(bn_macaque_rhesus_brain_2, community_partition_bn_macaque_rhesus_brain_2)
done("bn_macaque_rhesus_brain_2")
community_BC_word_adjacencies_edge_attack = community_attack_edge(word_adjacencies, community_partition_word_adjacencies)
done("word_adjacencies")
community_BC_circuits_s208_edge_attack = community_attack_edge(circuits_s208, community_partition_circuits_s208)
done("circuits_s208")
community_BC_bn_cat_mixed_species_brain_1_edge_attack = community_attack_edge(bn_cat_mixed_species_brain_1, community_partition_bn_cat_mixed_species_brain_1)
done("bn_cat_mixed_species_brain_1")
community_BC_facebook_414_edge_attack = community_attack_edge(facebook_414, community_partition_facebook_414)
done("facebook_414")
community_BC_facebook_686_edge_attack = community_attack_edge(facebook_686, community_partition_facebook_686)
done("facebook_686")
community_BC_polbooks_edge_attack = community_attack_edge(polbooks, community_partition_polbooks)
done("polbooks")
community_BC_jazz_edge_attack = community_attack_edge(jazz, community_partition_jazz)
done("jazz")
community_BC_facebook_348_edge_attack = community_attack_edge(facebook_348, community_partition_facebook_348)
done("facebook_348")
community_BC_circuits_s420_edge_attack = community_attack_edge(circuits_s420, community_partition_circuits_s420)
done("circuits_s420")
community_BC_corruption_edge_attack = community_attack_edge(corruption, community_partition_corruption)
done("corruption")
community_BC_facebook_0_edge_attack = community_attack_edge(facebook_0, community_partition_facebook_0)
done("facebook_0")
community_BC_circuits_s838_edge_attack = community_attack_edge(circuits_s838, community_partition_circuits_s838)
done("circuits_s838")
community_BC_crime_net_edge_attack = community_attack_edge(crime_net, community_partition_crime_net)
done("crime_net")
# community_BC_facebook_1684_edge_attack = community_attack_edge(facebook_1684, community_partition_facebook_1684)
# done("facebook_1684")

### Checkpoints:

Saving all results in csv files and graphs in .gml files.

#### LCC, Efficiency, R, InterE, time:

save 7 csv files: lcc batches, efficiency batches, lcc sequential, efficiency sequential, R, InterE, time

##### targeted_attacks

In [279]:
cols=['network', 'BC', 'CC','DC', 'NBC','PLCi','PR','MEB','MEC','MED', 'DWT', 'random', 'greedy' ]

data_frame_lcc_batch = pd.DataFrame(columns=cols)
data_frame_lcc_batch['network']=net_names

data_frame_lcc_batch['BC']=[list(BC_bn_macaque_rhesus_brain_2_attack[0].values()), list(BC_word_adjacencies_attack[0].values()), list(BC_circuits_s208_attack[0].values()), list(BC_bn_cat_mixed_species_brain_1_attack[0].values()), list(BC_facebook_414_attack[0].values()), list(BC_facebook_686_attack[0].values()), list(BC_polbooks_attack[0].values()), list(BC_jazz_attack[0].values()), list(BC_facebook_348_attack[0].values()), list(BC_circuits_s420_attack[0].values()), list(BC_corruption_attack[0].values()), list(BC_facebook_0_attack[0].values()), list(BC_circuits_s838_attack[0].values()), list(BC_crime_net_attack[0].values()), list(BC_facebook_1684_attack[0].values())]
data_frame_lcc_batch['CC']=[list(CC_bn_macaque_rhesus_brain_2_attack[0].values()), list(CC_word_adjacencies_attack[0].values()), list(CC_circuits_s208_attack[0].values()), list(CC_bn_cat_mixed_species_brain_1_attack[0].values()), list(CC_facebook_414_attack[0].values()), list(CC_facebook_686_attack[0].values()), list(CC_polbooks_attack[0].values()), list(CC_jazz_attack[0].values()), list(CC_facebook_348_attack[0].values()), list(CC_circuits_s420_attack[0].values()), list(CC_corruption_attack[0].values()), list(CC_facebook_0_attack[0].values()), list(CC_circuits_s838_attack[0].values()), list(CC_crime_net_attack[0].values()), list(CC_facebook_1684_attack[0].values())]
data_frame_lcc_batch['DC']=[list(DC_bn_macaque_rhesus_brain_2_attack[0].values()), list(DC_word_adjacencies_attack[0].values()), list(DC_circuits_s208_attack[0].values()), list(DC_bn_cat_mixed_species_brain_1_attack[0].values()), list(DC_facebook_414_attack[0].values()), list(DC_facebook_686_attack[0].values()), list(DC_polbooks_attack[0].values()), list(DC_jazz_attack[0].values()), list(DC_facebook_348_attack[0].values()), list(DC_circuits_s420_attack[0].values()), list(DC_corruption_attack[0].values()), list(DC_facebook_0_attack[0].values()), list(DC_circuits_s838_attack[0].values()), list(DC_crime_net_attack[0].values()), list(DC_facebook_1684_attack[0].values())]
data_frame_lcc_batch['NBC']=[list(NBC_bn_macaque_rhesus_brain_2_attack[0].values()), list(NBC_word_adjacencies_attack[0].values()), list(NBC_circuits_s208_attack[0].values()), list(NBC_bn_cat_mixed_species_brain_1_attack[0].values()), list(NBC_facebook_414_attack[0].values()), list(NBC_facebook_686_attack[0].values()), list(NBC_polbooks_attack[0].values()), list(NBC_jazz_attack[0].values()), list(NBC_facebook_348_attack[0].values()), list(NBC_circuits_s420_attack[0].values()), list(NBC_corruption_attack[0].values()), list(NBC_facebook_0_attack[0].values()), list(NBC_circuits_s838_attack[0].values()), list(NBC_crime_net_attack[0].values()), list(NBC_facebook_1684_attack[0].values())]
data_frame_lcc_batch['PLCi']=[list(PLCi_bn_macaque_rhesus_brain_2_attack[0].values()), list(PLCi_word_adjacencies_attack[0].values()), list(PLCi_circuits_s208_attack[0].values()), list(PLCi_bn_cat_mixed_species_brain_1_attack[0].values()), list(PLCi_facebook_414_attack[0].values()), list(PLCi_facebook_686_attack[0].values()), list(PLCi_polbooks_attack[0].values()), list(PLCi_jazz_attack[0].values()), list(PLCi_facebook_348_attack[0].values()), list(PLCi_circuits_s420_attack[0].values()), list(PLCi_corruption_attack[0].values()), list(PLCi_facebook_0_attack[0].values()), list(PLCi_circuits_s838_attack[0].values()), list(PLCi_crime_net_attack[0].values()), list(PLCi_facebook_1684_attack[0].values())]
data_frame_lcc_batch['PR']=[list(PR_bn_macaque_rhesus_brain_2_attack[0].values()), list(PR_word_adjacencies_attack[0].values()), list(PR_circuits_s208_attack[0].values()), list(PR_bn_cat_mixed_species_brain_1_attack[0].values()), list(PR_facebook_414_attack[0].values()), list(PR_facebook_686_attack[0].values()), list(PR_polbooks_attack[0].values()), list(PR_jazz_attack[0].values()), list(PR_facebook_348_attack[0].values()), list(PR_circuits_s420_attack[0].values()), list(PR_corruption_attack[0].values()), list(PR_facebook_0_attack[0].values()), list(PR_circuits_s838_attack[0].values()), list(PR_crime_net_attack[0].values()), list(PR_facebook_1684_attack[0].values())]
data_frame_lcc_batch['MEB']=[list(MEB_bn_macaque_rhesus_brain_2_attack[0].values()), list(MEB_word_adjacencies_attack[0].values()), list(MEB_circuits_s208_attack[0].values()), list(MEB_bn_cat_mixed_species_brain_1_attack[0].values()), list(MEB_facebook_414_attack[0].values()), list(MEB_facebook_686_attack[0].values()), list(MEB_polbooks_attack[0].values()), list(MEB_jazz_attack[0].values()), list(MEB_facebook_348_attack[0].values()), list(MEB_circuits_s420_attack[0].values()), list(MEB_corruption_attack[0].values()), list(MEB_facebook_0_attack[0].values()), list(MEB_circuits_s838_attack[0].values()), list(MEB_crime_net_attack[0].values()), list(MEB_facebook_1684_attack[0].values())]
data_frame_lcc_batch['MEC']=[list(MEC_bn_macaque_rhesus_brain_2_attack[0].values()), list(MEC_word_adjacencies_attack[0].values()), list(MEC_circuits_s208_attack[0].values()), list(MEC_bn_cat_mixed_species_brain_1_attack[0].values()), list(MEC_facebook_414_attack[0].values()), list(MEC_facebook_686_attack[0].values()), list(MEC_polbooks_attack[0].values()), list(MEC_jazz_attack[0].values()), list(MEC_facebook_348_attack[0].values()), list(MEC_circuits_s420_attack[0].values()), list(MEC_corruption_attack[0].values()), list(MEC_facebook_0_attack[0].values()), list(MEC_circuits_s838_attack[0].values()), list(MEC_crime_net_attack[0].values()), list(MEC_facebook_1684_attack[0].values())]
data_frame_lcc_batch['MED']=[list(MED_bn_macaque_rhesus_brain_2_attack[0].values()), list(MED_word_adjacencies_attack[0].values()), list(MED_circuits_s208_attack[0].values()), list(MED_bn_cat_mixed_species_brain_1_attack[0].values()), list(MED_facebook_414_attack[0].values()), list(MED_facebook_686_attack[0].values()), list(MED_polbooks_attack[0].values()), list(MED_jazz_attack[0].values()), list(MED_facebook_348_attack[0].values()), list(MED_circuits_s420_attack[0].values()), list(MED_corruption_attack[0].values()), list(MED_facebook_0_attack[0].values()), list(MED_circuits_s838_attack[0].values()), list(MED_crime_net_attack[0].values()), list(MED_facebook_1684_attack[0].values())]
data_frame_lcc_batch['DWT']=[list(DWT_bn_macaque_rhesus_brain_2_attack[0].values()), list(DWT_word_adjacencies_attack[0].values()), list(DWT_circuits_s208_attack[0].values()), list(DWT_bn_cat_mixed_species_brain_1_attack[0].values()), list(DWT_facebook_414_attack[0].values()), list(DWT_facebook_686_attack[0].values()), list(DWT_polbooks_attack[0].values()), list(DWT_jazz_attack[0].values()), list(DWT_facebook_348_attack[0].values()), list(DWT_circuits_s420_attack[0].values()), list(DWT_corruption_attack[0].values()), list(DWT_facebook_0_attack[0].values()), list(DWT_circuits_s838_attack[0].values()), list(DWT_crime_net_attack[0].values()), list(DWT_facebook_1684_attack[0].values())]
data_frame_lcc_batch['random']=[list(random_bn_macaque_rhesus_brain_2_attack[0].values()), list(random_word_adjacencies_attack[0].values()), list(random_circuits_s208_attack[0].values()), list(random_bn_cat_mixed_species_brain_1_attack[0].values()), list(random_facebook_414_attack[0].values()), list(random_facebook_686_attack[0].values()), list(random_polbooks_attack[0].values()), list(random_jazz_attack[0].values()), list(random_facebook_348_attack[0].values()), list(random_circuits_s420_attack[0].values()), list(random_corruption_attack[0].values()), list(random_facebook_0_attack[0].values()), list(random_circuits_s838_attack[0].values()), list(random_crime_net_attack[0].values()), list(random_facebook_1684_attack[0].values())]
data_frame_lcc_batch['greedy']=[list(greedy_bn_macaque_rhesus_brain_2_attack[0].values()), list(greedy_word_adjacencies_attack[0].values()), list(greedy_circuits_s208_attack[0].values()), list(greedy_bn_cat_mixed_species_brain_1_attack[0].values()), list(greedy_facebook_414_attack[0].values()), list(greedy_facebook_686_attack[0].values()), list(greedy_polbooks_attack[0].values()), list(greedy_jazz_attack[0].values()), list(greedy_facebook_348_attack[0].values()), list(greedy_circuits_s420_attack[0].values()), list(greedy_corruption_attack[0].values()), list(greedy_facebook_0_attack[0].values()), list(greedy_circuits_s838_attack[0].values()), list(greedy_crime_net_attack[0].values()), list(greedy_facebook_1684_attack[0].values())]
data_frame_lcc_batch.to_csv("../results/lcc_batch_values.csv")
data_frame_efficiency_batch = pd.DataFrame(columns=cols)
data_frame_efficiency_batch['network']=net_names

data_frame_efficiency_batch['BC']=[list(BC_bn_macaque_rhesus_brain_2_attack[1].values()), list(BC_word_adjacencies_attack[1].values()), list(BC_circuits_s208_attack[1].values()), list(BC_bn_cat_mixed_species_brain_1_attack[1].values()), list(BC_facebook_414_attack[1].values()), list(BC_facebook_686_attack[1].values()), list(BC_polbooks_attack[1].values()), list(BC_jazz_attack[1].values()), list(BC_facebook_348_attack[1].values()), list(BC_circuits_s420_attack[1].values()), list(BC_corruption_attack[1].values()), list(BC_facebook_0_attack[1].values()), list(BC_circuits_s838_attack[1].values()), list(BC_crime_net_attack[1].values()), list(BC_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['CC']=[list(CC_bn_macaque_rhesus_brain_2_attack[1].values()), list(CC_word_adjacencies_attack[1].values()), list(CC_circuits_s208_attack[1].values()), list(CC_bn_cat_mixed_species_brain_1_attack[1].values()), list(CC_facebook_414_attack[1].values()), list(CC_facebook_686_attack[1].values()), list(CC_polbooks_attack[1].values()), list(CC_jazz_attack[1].values()), list(CC_facebook_348_attack[1].values()), list(CC_circuits_s420_attack[1].values()), list(CC_corruption_attack[1].values()), list(CC_facebook_0_attack[1].values()), list(CC_circuits_s838_attack[1].values()), list(CC_crime_net_attack[1].values()), list(CC_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['DC']=[list(DC_bn_macaque_rhesus_brain_2_attack[1].values()), list(DC_word_adjacencies_attack[1].values()), list(DC_circuits_s208_attack[1].values()), list(DC_bn_cat_mixed_species_brain_1_attack[1].values()), list(DC_facebook_414_attack[1].values()), list(DC_facebook_686_attack[1].values()), list(DC_polbooks_attack[1].values()), list(DC_jazz_attack[1].values()), list(DC_facebook_348_attack[1].values()), list(DC_circuits_s420_attack[1].values()), list(DC_corruption_attack[1].values()), list(DC_facebook_0_attack[1].values()), list(DC_circuits_s838_attack[1].values()), list(DC_crime_net_attack[1].values()), list(DC_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['NBC']=[list(NBC_bn_macaque_rhesus_brain_2_attack[1].values()), list(NBC_word_adjacencies_attack[1].values()), list(NBC_circuits_s208_attack[1].values()), list(NBC_bn_cat_mixed_species_brain_1_attack[1].values()), list(NBC_facebook_414_attack[1].values()), list(NBC_facebook_686_attack[1].values()), list(NBC_polbooks_attack[1].values()), list(NBC_jazz_attack[1].values()), list(NBC_facebook_348_attack[1].values()), list(NBC_circuits_s420_attack[1].values()), list(NBC_corruption_attack[1].values()), list(NBC_facebook_0_attack[1].values()), list(NBC_circuits_s838_attack[1].values()), list(NBC_crime_net_attack[1].values()), list(NBC_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['PLCi']=[list(PLCi_bn_macaque_rhesus_brain_2_attack[1].values()), list(PLCi_word_adjacencies_attack[1].values()), list(PLCi_circuits_s208_attack[1].values()), list(PLCi_bn_cat_mixed_species_brain_1_attack[1].values()), list(PLCi_facebook_414_attack[1].values()), list(PLCi_facebook_686_attack[1].values()), list(PLCi_polbooks_attack[1].values()), list(PLCi_jazz_attack[1].values()), list(PLCi_facebook_348_attack[1].values()), list(PLCi_circuits_s420_attack[1].values()), list(PLCi_corruption_attack[1].values()), list(PLCi_facebook_0_attack[1].values()), list(PLCi_circuits_s838_attack[1].values()), list(PLCi_crime_net_attack[1].values()), list(PLCi_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['PR']=[list(PR_bn_macaque_rhesus_brain_2_attack[1].values()), list(PR_word_adjacencies_attack[1].values()), list(PR_circuits_s208_attack[1].values()), list(PR_bn_cat_mixed_species_brain_1_attack[1].values()), list(PR_facebook_414_attack[1].values()), list(PR_facebook_686_attack[1].values()), list(PR_polbooks_attack[1].values()), list(PR_jazz_attack[1].values()), list(PR_facebook_348_attack[1].values()), list(PR_circuits_s420_attack[1].values()), list(PR_corruption_attack[1].values()), list(PR_facebook_0_attack[1].values()), list(PR_circuits_s838_attack[1].values()), list(PR_crime_net_attack[1].values()), list(PR_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['MEB']=[list(MEB_bn_macaque_rhesus_brain_2_attack[1].values()), list(MEB_word_adjacencies_attack[1].values()), list(MEB_circuits_s208_attack[1].values()), list(MEB_bn_cat_mixed_species_brain_1_attack[1].values()), list(MEB_facebook_414_attack[1].values()), list(MEB_facebook_686_attack[1].values()), list(MEB_polbooks_attack[1].values()), list(MEB_jazz_attack[1].values()), list(MEB_facebook_348_attack[1].values()), list(MEB_circuits_s420_attack[1].values()), list(MEB_corruption_attack[1].values()), list(MEB_facebook_0_attack[1].values()), list(MEB_circuits_s838_attack[1].values()), list(MEB_crime_net_attack[1].values()), list(MEB_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['MEC']=[list(MEC_bn_macaque_rhesus_brain_2_attack[1].values()), list(MEC_word_adjacencies_attack[1].values()), list(MEC_circuits_s208_attack[1].values()), list(MEC_bn_cat_mixed_species_brain_1_attack[1].values()), list(MEC_facebook_414_attack[1].values()), list(MEC_facebook_686_attack[1].values()), list(MEC_polbooks_attack[1].values()), list(MEC_jazz_attack[1].values()), list(MEC_facebook_348_attack[1].values()), list(MEC_circuits_s420_attack[1].values()), list(MEC_corruption_attack[1].values()), list(MEC_facebook_0_attack[1].values()), list(MEC_circuits_s838_attack[1].values()), list(MEC_crime_net_attack[1].values()), list(MEC_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['MED']=[list(MED_bn_macaque_rhesus_brain_2_attack[1].values()), list(MED_word_adjacencies_attack[1].values()), list(MED_circuits_s208_attack[1].values()), list(MED_bn_cat_mixed_species_brain_1_attack[1].values()), list(MED_facebook_414_attack[1].values()), list(MED_facebook_686_attack[1].values()), list(MED_polbooks_attack[1].values()), list(MED_jazz_attack[1].values()), list(MED_facebook_348_attack[1].values()), list(MED_circuits_s420_attack[1].values()), list(MED_corruption_attack[1].values()), list(MED_facebook_0_attack[1].values()), list(MED_circuits_s838_attack[1].values()), list(MED_crime_net_attack[1].values()), list(MED_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['DWT']=[list(DWT_bn_macaque_rhesus_brain_2_attack[1].values()), list(DWT_word_adjacencies_attack[1].values()), list(DWT_circuits_s208_attack[1].values()), list(DWT_bn_cat_mixed_species_brain_1_attack[1].values()), list(DWT_facebook_414_attack[1].values()), list(DWT_facebook_686_attack[1].values()), list(DWT_polbooks_attack[1].values()), list(DWT_jazz_attack[1].values()), list(DWT_facebook_348_attack[1].values()), list(DWT_circuits_s420_attack[1].values()), list(DWT_corruption_attack[1].values()), list(DWT_facebook_0_attack[1].values()), list(DWT_circuits_s838_attack[1].values()), list(DWT_crime_net_attack[1].values()), list(DWT_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['random']=[list(random_bn_macaque_rhesus_brain_2_attack[1].values()), list(random_word_adjacencies_attack[1].values()), list(random_circuits_s208_attack[1].values()), list(random_bn_cat_mixed_species_brain_1_attack[1].values()), list(random_facebook_414_attack[1].values()), list(random_facebook_686_attack[1].values()), list(random_polbooks_attack[1].values()), list(random_jazz_attack[1].values()), list(random_facebook_348_attack[1].values()), list(random_circuits_s420_attack[1].values()), list(random_corruption_attack[1].values()), list(random_facebook_0_attack[1].values()), list(random_circuits_s838_attack[1].values()), list(random_crime_net_attack[1].values()), list(random_facebook_1684_attack[1].values())]
data_frame_efficiency_batch['greedy']=[list(greedy_bn_macaque_rhesus_brain_2_attack[1].values()), list(greedy_word_adjacencies_attack[1].values()), list(greedy_circuits_s208_attack[1].values()), list(greedy_bn_cat_mixed_species_brain_1_attack[1].values()), list(greedy_facebook_414_attack[1].values()), list(greedy_facebook_686_attack[1].values()), list(greedy_polbooks_attack[1].values()), list(greedy_jazz_attack[1].values()), list(greedy_facebook_348_attack[1].values()), list(greedy_circuits_s420_attack[1].values()), list(greedy_corruption_attack[1].values()), list(greedy_facebook_0_attack[1].values()), list(greedy_circuits_s838_attack[1].values()), list(greedy_crime_net_attack[1].values()), list(greedy_facebook_1684_attack[1].values())]
data_frame_efficiency_batch.to_csv("../results/efficiency_batch_values.csv")

data_frame_lcc_seq = pd.DataFrame(columns=cols)
data_frame_lcc_seq['network']=net_names

data_frame_lcc_seq['BC']=[list(BC_bn_macaque_rhesus_brain_2_attack[2].values()), list(BC_word_adjacencies_attack[2].values()), list(BC_circuits_s208_attack[2].values()), list(BC_bn_cat_mixed_species_brain_1_attack[2].values()), list(BC_facebook_414_attack[2].values()), list(BC_facebook_686_attack[2].values()), list(BC_polbooks_attack[2].values()), list(BC_jazz_attack[2].values()), list(BC_facebook_348_attack[2].values()), list(BC_circuits_s420_attack[2].values()), list(BC_corruption_attack[2].values()), list(BC_facebook_0_attack[2].values()), list(BC_circuits_s838_attack[2].values()), list(BC_crime_net_attack[2].values()), list(BC_facebook_1684_attack[2].values())]
data_frame_lcc_seq['CC']=[list(CC_bn_macaque_rhesus_brain_2_attack[2].values()), list(CC_word_adjacencies_attack[2].values()), list(CC_circuits_s208_attack[2].values()), list(CC_bn_cat_mixed_species_brain_1_attack[2].values()), list(CC_facebook_414_attack[2].values()), list(CC_facebook_686_attack[2].values()), list(CC_polbooks_attack[2].values()), list(CC_jazz_attack[2].values()), list(CC_facebook_348_attack[2].values()), list(CC_circuits_s420_attack[2].values()), list(CC_corruption_attack[2].values()), list(CC_facebook_0_attack[2].values()), list(CC_circuits_s838_attack[2].values()), list(CC_crime_net_attack[2].values()), list(CC_facebook_1684_attack[2].values())]
data_frame_lcc_seq['DC']=[list(DC_bn_macaque_rhesus_brain_2_attack[2].values()), list(DC_word_adjacencies_attack[2].values()), list(DC_circuits_s208_attack[2].values()), list(DC_bn_cat_mixed_species_brain_1_attack[2].values()), list(DC_facebook_414_attack[2].values()), list(DC_facebook_686_attack[2].values()), list(DC_polbooks_attack[2].values()), list(DC_jazz_attack[2].values()), list(DC_facebook_348_attack[2].values()), list(DC_circuits_s420_attack[2].values()), list(DC_corruption_attack[2].values()), list(DC_facebook_0_attack[2].values()), list(DC_circuits_s838_attack[2].values()), list(DC_crime_net_attack[2].values()), list(DC_facebook_1684_attack[2].values())]
data_frame_lcc_seq['NBC']=[list(NBC_bn_macaque_rhesus_brain_2_attack[2].values()), list(NBC_word_adjacencies_attack[2].values()), list(NBC_circuits_s208_attack[2].values()), list(NBC_bn_cat_mixed_species_brain_1_attack[2].values()), list(NBC_facebook_414_attack[2].values()), list(NBC_facebook_686_attack[2].values()), list(NBC_polbooks_attack[2].values()), list(NBC_jazz_attack[2].values()), list(NBC_facebook_348_attack[2].values()), list(NBC_circuits_s420_attack[2].values()), list(NBC_corruption_attack[2].values()), list(NBC_facebook_0_attack[2].values()), list(NBC_circuits_s838_attack[2].values()), list(NBC_crime_net_attack[2].values()), list(NBC_facebook_1684_attack[2].values())]
data_frame_lcc_seq['PLCi']=[list(PLCi_bn_macaque_rhesus_brain_2_attack[2].values()), list(PLCi_word_adjacencies_attack[2].values()), list(PLCi_circuits_s208_attack[2].values()), list(PLCi_bn_cat_mixed_species_brain_1_attack[2].values()), list(PLCi_facebook_414_attack[2].values()), list(PLCi_facebook_686_attack[2].values()), list(PLCi_polbooks_attack[2].values()), list(PLCi_jazz_attack[2].values()), list(PLCi_facebook_348_attack[2].values()), list(PLCi_circuits_s420_attack[2].values()), list(PLCi_corruption_attack[2].values()), list(PLCi_facebook_0_attack[2].values()), list(PLCi_circuits_s838_attack[2].values()), list(PLCi_crime_net_attack[2].values()), list(PLCi_facebook_1684_attack[2].values())]
data_frame_lcc_seq['PR']=[list(PR_bn_macaque_rhesus_brain_2_attack[2].values()), list(PR_word_adjacencies_attack[2].values()), list(PR_circuits_s208_attack[2].values()), list(PR_bn_cat_mixed_species_brain_1_attack[2].values()), list(PR_facebook_414_attack[2].values()), list(PR_facebook_686_attack[2].values()), list(PR_polbooks_attack[2].values()), list(PR_jazz_attack[2].values()), list(PR_facebook_348_attack[2].values()), list(PR_circuits_s420_attack[2].values()), list(PR_corruption_attack[2].values()), list(PR_facebook_0_attack[2].values()), list(PR_circuits_s838_attack[2].values()), list(PR_crime_net_attack[2].values()), list(PR_facebook_1684_attack[2].values())]
data_frame_lcc_seq['MEB']=[list(MEB_bn_macaque_rhesus_brain_2_attack[2].values()), list(MEB_word_adjacencies_attack[2].values()), list(MEB_circuits_s208_attack[2].values()), list(MEB_bn_cat_mixed_species_brain_1_attack[2].values()), list(MEB_facebook_414_attack[2].values()), list(MEB_facebook_686_attack[2].values()), list(MEB_polbooks_attack[2].values()), list(MEB_jazz_attack[2].values()), list(MEB_facebook_348_attack[2].values()), list(MEB_circuits_s420_attack[2].values()), list(MEB_corruption_attack[2].values()), list(MEB_facebook_0_attack[2].values()), list(MEB_circuits_s838_attack[2].values()), list(MEB_crime_net_attack[2].values()), list(MEB_facebook_1684_attack[2].values())]
data_frame_lcc_seq['MEC']=[list(MEC_bn_macaque_rhesus_brain_2_attack[2].values()), list(MEC_word_adjacencies_attack[2].values()), list(MEC_circuits_s208_attack[2].values()), list(MEC_bn_cat_mixed_species_brain_1_attack[2].values()), list(MEC_facebook_414_attack[2].values()), list(MEC_facebook_686_attack[2].values()), list(MEC_polbooks_attack[2].values()), list(MEC_jazz_attack[2].values()), list(MEC_facebook_348_attack[2].values()), list(MEC_circuits_s420_attack[2].values()), list(MEC_corruption_attack[2].values()), list(MEC_facebook_0_attack[2].values()), list(MEC_circuits_s838_attack[2].values()), list(MEC_crime_net_attack[2].values()), list(MEC_facebook_1684_attack[2].values())]
data_frame_lcc_seq['MED']=[list(MED_bn_macaque_rhesus_brain_2_attack[2].values()), list(MED_word_adjacencies_attack[2].values()), list(MED_circuits_s208_attack[2].values()), list(MED_bn_cat_mixed_species_brain_1_attack[2].values()), list(MED_facebook_414_attack[2].values()), list(MED_facebook_686_attack[2].values()), list(MED_polbooks_attack[2].values()), list(MED_jazz_attack[2].values()), list(MED_facebook_348_attack[2].values()), list(MED_circuits_s420_attack[2].values()), list(MED_corruption_attack[2].values()), list(MED_facebook_0_attack[2].values()), list(MED_circuits_s838_attack[2].values()), list(MED_crime_net_attack[2].values()), list(MED_facebook_1684_attack[2].values())]
data_frame_lcc_seq['DWT']=[list(DWT_bn_macaque_rhesus_brain_2_attack[2].values()), list(DWT_word_adjacencies_attack[2].values()), list(DWT_circuits_s208_attack[2].values()), list(DWT_bn_cat_mixed_species_brain_1_attack[2].values()), list(DWT_facebook_414_attack[2].values()), list(DWT_facebook_686_attack[2].values()), list(DWT_polbooks_attack[2].values()), list(DWT_jazz_attack[2].values()), list(DWT_facebook_348_attack[2].values()), list(DWT_circuits_s420_attack[2].values()), list(DWT_corruption_attack[2].values()), list(DWT_facebook_0_attack[2].values()), list(DWT_circuits_s838_attack[2].values()), list(DWT_crime_net_attack[2].values()), list(DWT_facebook_1684_attack[2].values())]
data_frame_lcc_seq['random']=[list(random_bn_macaque_rhesus_brain_2_attack[2].values()), list(random_word_adjacencies_attack[2].values()), list(random_circuits_s208_attack[2].values()), list(random_bn_cat_mixed_species_brain_1_attack[2].values()), list(random_facebook_414_attack[2].values()), list(random_facebook_686_attack[2].values()), list(random_polbooks_attack[2].values()), list(random_jazz_attack[2].values()), list(random_facebook_348_attack[2].values()), list(random_circuits_s420_attack[2].values()), list(random_corruption_attack[2].values()), list(random_facebook_0_attack[2].values()), list(random_circuits_s838_attack[2].values()), list(random_crime_net_attack[2].values()), list(random_facebook_1684_attack[2].values())]
data_frame_lcc_seq['greedy']=[list(greedy_bn_macaque_rhesus_brain_2_attack[2].values()), list(greedy_word_adjacencies_attack[2].values()), list(greedy_circuits_s208_attack[2].values()), list(greedy_bn_cat_mixed_species_brain_1_attack[2].values()), list(greedy_facebook_414_attack[2].values()), list(greedy_facebook_686_attack[2].values()), list(greedy_polbooks_attack[2].values()), list(greedy_jazz_attack[2].values()), list(greedy_facebook_348_attack[2].values()), list(greedy_circuits_s420_attack[2].values()), list(greedy_corruption_attack[2].values()), list(greedy_facebook_0_attack[2].values()), list(greedy_circuits_s838_attack[2].values()), list(greedy_crime_net_attack[2].values()), list(greedy_facebook_1684_attack[2].values())]
data_frame_lcc_seq.to_csv("../results/lcc_seq_values.csv")

data_frame_efficiency_seq = pd.DataFrame(columns=cols)
data_frame_efficiency_seq['network']=net_names

data_frame_efficiency_seq['BC']=[list(BC_bn_macaque_rhesus_brain_2_attack[3].values()), list(BC_word_adjacencies_attack[3].values()), list(BC_circuits_s208_attack[3].values()), list(BC_bn_cat_mixed_species_brain_1_attack[3].values()), list(BC_facebook_414_attack[3].values()), list(BC_facebook_686_attack[3].values()), list(BC_polbooks_attack[3].values()), list(BC_jazz_attack[3].values()), list(BC_facebook_348_attack[3].values()), list(BC_circuits_s420_attack[3].values()), list(BC_corruption_attack[3].values()), list(BC_facebook_0_attack[3].values()), list(BC_circuits_s838_attack[3].values()), list(BC_crime_net_attack[3].values()), list(BC_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['CC']=[list(CC_bn_macaque_rhesus_brain_2_attack[3].values()), list(CC_word_adjacencies_attack[3].values()), list(CC_circuits_s208_attack[3].values()), list(CC_bn_cat_mixed_species_brain_1_attack[3].values()), list(CC_facebook_414_attack[3].values()), list(CC_facebook_686_attack[3].values()), list(CC_polbooks_attack[3].values()), list(CC_jazz_attack[3].values()), list(CC_facebook_348_attack[3].values()), list(CC_circuits_s420_attack[3].values()), list(CC_corruption_attack[3].values()), list(CC_facebook_0_attack[3].values()), list(CC_circuits_s838_attack[3].values()), list(CC_crime_net_attack[3].values()), list(CC_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['DC']=[list(DC_bn_macaque_rhesus_brain_2_attack[3].values()), list(DC_word_adjacencies_attack[3].values()), list(DC_circuits_s208_attack[3].values()), list(DC_bn_cat_mixed_species_brain_1_attack[3].values()), list(DC_facebook_414_attack[3].values()), list(DC_facebook_686_attack[3].values()), list(DC_polbooks_attack[3].values()), list(DC_jazz_attack[3].values()), list(DC_facebook_348_attack[3].values()), list(DC_circuits_s420_attack[3].values()), list(DC_corruption_attack[3].values()), list(DC_facebook_0_attack[3].values()), list(DC_circuits_s838_attack[3].values()), list(DC_crime_net_attack[3].values()), list(DC_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['NBC']=[list(NBC_bn_macaque_rhesus_brain_2_attack[3].values()), list(NBC_word_adjacencies_attack[3].values()), list(NBC_circuits_s208_attack[3].values()), list(NBC_bn_cat_mixed_species_brain_1_attack[3].values()), list(NBC_facebook_414_attack[3].values()), list(NBC_facebook_686_attack[3].values()), list(NBC_polbooks_attack[3].values()), list(NBC_jazz_attack[3].values()), list(NBC_facebook_348_attack[3].values()), list(NBC_circuits_s420_attack[3].values()), list(NBC_corruption_attack[3].values()), list(NBC_facebook_0_attack[3].values()), list(NBC_circuits_s838_attack[3].values()), list(NBC_crime_net_attack[3].values()), list(NBC_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['PLCi']=[list(PLCi_bn_macaque_rhesus_brain_2_attack[3].values()), list(PLCi_word_adjacencies_attack[3].values()), list(PLCi_circuits_s208_attack[3].values()), list(PLCi_bn_cat_mixed_species_brain_1_attack[3].values()), list(PLCi_facebook_414_attack[3].values()), list(PLCi_facebook_686_attack[3].values()), list(PLCi_polbooks_attack[3].values()), list(PLCi_jazz_attack[3].values()), list(PLCi_facebook_348_attack[3].values()), list(PLCi_circuits_s420_attack[3].values()), list(PLCi_corruption_attack[3].values()), list(PLCi_facebook_0_attack[3].values()), list(PLCi_circuits_s838_attack[3].values()), list(PLCi_crime_net_attack[3].values()), list(PLCi_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['PR']=[list(PR_bn_macaque_rhesus_brain_2_attack[3].values()), list(PR_word_adjacencies_attack[3].values()), list(PR_circuits_s208_attack[3].values()), list(PR_bn_cat_mixed_species_brain_1_attack[3].values()), list(PR_facebook_414_attack[3].values()), list(PR_facebook_686_attack[3].values()), list(PR_polbooks_attack[3].values()), list(PR_jazz_attack[3].values()), list(PR_facebook_348_attack[3].values()), list(PR_circuits_s420_attack[3].values()), list(PR_corruption_attack[3].values()), list(PR_facebook_0_attack[3].values()), list(PR_circuits_s838_attack[3].values()), list(PR_crime_net_attack[3].values()), list(PR_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['MEB']=[list(MEB_bn_macaque_rhesus_brain_2_attack[3].values()), list(MEB_word_adjacencies_attack[3].values()), list(MEB_circuits_s208_attack[3].values()), list(MEB_bn_cat_mixed_species_brain_1_attack[3].values()), list(MEB_facebook_414_attack[3].values()), list(MEB_facebook_686_attack[3].values()), list(MEB_polbooks_attack[3].values()), list(MEB_jazz_attack[3].values()), list(MEB_facebook_348_attack[3].values()), list(MEB_circuits_s420_attack[3].values()), list(MEB_corruption_attack[3].values()), list(MEB_facebook_0_attack[3].values()), list(MEB_circuits_s838_attack[3].values()), list(MEB_crime_net_attack[3].values()), list(MEB_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['MEC']=[list(MEC_bn_macaque_rhesus_brain_2_attack[3].values()), list(MEC_word_adjacencies_attack[3].values()), list(MEC_circuits_s208_attack[3].values()), list(MEC_bn_cat_mixed_species_brain_1_attack[3].values()), list(MEC_facebook_414_attack[3].values()), list(MEC_facebook_686_attack[3].values()), list(MEC_polbooks_attack[3].values()), list(MEC_jazz_attack[3].values()), list(MEC_facebook_348_attack[3].values()), list(MEC_circuits_s420_attack[3].values()), list(MEC_corruption_attack[3].values()), list(MEC_facebook_0_attack[3].values()), list(MEC_circuits_s838_attack[3].values()), list(MEC_crime_net_attack[3].values()), list(MEC_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['MED']=[list(MED_bn_macaque_rhesus_brain_2_attack[3].values()), list(MED_word_adjacencies_attack[3].values()), list(MED_circuits_s208_attack[3].values()), list(MED_bn_cat_mixed_species_brain_1_attack[3].values()), list(MED_facebook_414_attack[3].values()), list(MED_facebook_686_attack[3].values()), list(MED_polbooks_attack[3].values()), list(MED_jazz_attack[3].values()), list(MED_facebook_348_attack[3].values()), list(MED_circuits_s420_attack[3].values()), list(MED_corruption_attack[3].values()), list(MED_facebook_0_attack[3].values()), list(MED_circuits_s838_attack[3].values()), list(MED_crime_net_attack[3].values()), list(MED_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['DWT']=[list(DWT_bn_macaque_rhesus_brain_2_attack[3].values()), list(DWT_word_adjacencies_attack[3].values()), list(DWT_circuits_s208_attack[3].values()), list(DWT_bn_cat_mixed_species_brain_1_attack[3].values()), list(DWT_facebook_414_attack[3].values()), list(DWT_facebook_686_attack[3].values()), list(DWT_polbooks_attack[3].values()), list(DWT_jazz_attack[3].values()), list(DWT_facebook_348_attack[3].values()), list(DWT_circuits_s420_attack[3].values()), list(DWT_corruption_attack[3].values()), list(DWT_facebook_0_attack[3].values()), list(DWT_circuits_s838_attack[3].values()), list(DWT_crime_net_attack[3].values()), list(DWT_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['random']=[list(random_bn_macaque_rhesus_brain_2_attack[3].values()), list(random_word_adjacencies_attack[3].values()), list(random_circuits_s208_attack[3].values()), list(random_bn_cat_mixed_species_brain_1_attack[3].values()), list(random_facebook_414_attack[3].values()), list(random_facebook_686_attack[3].values()), list(random_polbooks_attack[3].values()), list(random_jazz_attack[3].values()), list(random_facebook_348_attack[3].values()), list(random_circuits_s420_attack[3].values()), list(random_corruption_attack[3].values()), list(random_facebook_0_attack[3].values()), list(random_circuits_s838_attack[3].values()), list(random_crime_net_attack[3].values()), list(random_facebook_1684_attack[3].values())]
data_frame_efficiency_seq['greedy']=[list(greedy_bn_macaque_rhesus_brain_2_attack[3].values()), list(greedy_word_adjacencies_attack[3].values()), list(greedy_circuits_s208_attack[3].values()), list(greedy_bn_cat_mixed_species_brain_1_attack[3].values()), list(greedy_facebook_414_attack[3].values()), list(greedy_facebook_686_attack[3].values()), list(greedy_polbooks_attack[3].values()), list(greedy_jazz_attack[3].values()), list(greedy_facebook_348_attack[3].values()), list(greedy_circuits_s420_attack[3].values()), list(greedy_corruption_attack[3].values()), list(greedy_facebook_0_attack[3].values()), list(greedy_circuits_s838_attack[3].values()), list(greedy_crime_net_attack[3].values()), list(greedy_facebook_1684_attack[3].values())]
data_frame_efficiency_seq.to_csv("../results/efficiency_seq_values.csv")

data_frame_R= pd.DataFrame(columns=cols)
data_frame_R['network']=net_names

data_frame_R['BC']=[BC_bn_macaque_rhesus_brain_2_attack[4], BC_word_adjacencies_attack[4], BC_circuits_s208_attack[4], BC_bn_cat_mixed_species_brain_1_attack[4], BC_facebook_414_attack[4], BC_facebook_686_attack[4], BC_polbooks_attack[4], BC_jazz_attack[4], BC_facebook_348_attack[4], BC_circuits_s420_attack[4], BC_corruption_attack[4], BC_facebook_0_attack[4], BC_circuits_s838_attack[4], BC_crime_net_attack[4], BC_facebook_1684_attack[4]]
data_frame_R['CC']=[CC_bn_macaque_rhesus_brain_2_attack[4], CC_word_adjacencies_attack[4], CC_circuits_s208_attack[4], CC_bn_cat_mixed_species_brain_1_attack[4], CC_facebook_414_attack[4], CC_facebook_686_attack[4], CC_polbooks_attack[4], CC_jazz_attack[4], CC_facebook_348_attack[4], CC_circuits_s420_attack[4], CC_corruption_attack[4], CC_facebook_0_attack[4], CC_circuits_s838_attack[4], CC_crime_net_attack[4], CC_facebook_1684_attack[4]]
data_frame_R['DC']=[DC_bn_macaque_rhesus_brain_2_attack[4], DC_word_adjacencies_attack[4], DC_circuits_s208_attack[4], DC_bn_cat_mixed_species_brain_1_attack[4], DC_facebook_414_attack[4], DC_facebook_686_attack[4], DC_polbooks_attack[4], DC_jazz_attack[4], DC_facebook_348_attack[4], DC_circuits_s420_attack[4], DC_corruption_attack[4], DC_facebook_0_attack[4], DC_circuits_s838_attack[4], DC_crime_net_attack[4], DC_facebook_1684_attack[4]]
data_frame_R['NBC']=[NBC_bn_macaque_rhesus_brain_2_attack[4], NBC_word_adjacencies_attack[4], NBC_circuits_s208_attack[4], NBC_bn_cat_mixed_species_brain_1_attack[4], NBC_facebook_414_attack[4], NBC_facebook_686_attack[4], NBC_polbooks_attack[4], NBC_jazz_attack[4], NBC_facebook_348_attack[4], NBC_circuits_s420_attack[4], NBC_corruption_attack[4], NBC_facebook_0_attack[4], NBC_circuits_s838_attack[4], NBC_crime_net_attack[4], NBC_facebook_1684_attack[4]]
data_frame_R['PLCi']=[PLCi_bn_macaque_rhesus_brain_2_attack[4], PLCi_word_adjacencies_attack[4], PLCi_circuits_s208_attack[4], PLCi_bn_cat_mixed_species_brain_1_attack[4], PLCi_facebook_414_attack[4], PLCi_facebook_686_attack[4], PLCi_polbooks_attack[4], PLCi_jazz_attack[4], PLCi_facebook_348_attack[4], PLCi_circuits_s420_attack[4], PLCi_corruption_attack[4], PLCi_facebook_0_attack[4], PLCi_circuits_s838_attack[4], PLCi_crime_net_attack[4], PLCi_facebook_1684_attack[4]]
data_frame_R['PR']=[PR_bn_macaque_rhesus_brain_2_attack[4], PR_word_adjacencies_attack[4], PR_circuits_s208_attack[4], PR_bn_cat_mixed_species_brain_1_attack[4], PR_facebook_414_attack[4], PR_facebook_686_attack[4], PR_polbooks_attack[4], PR_jazz_attack[4], PR_facebook_348_attack[4], PR_circuits_s420_attack[4], PR_corruption_attack[4], PR_facebook_0_attack[4], PR_circuits_s838_attack[4], PR_crime_net_attack[4], PR_facebook_1684_attack[4]]
data_frame_R['MEB']=[MEB_bn_macaque_rhesus_brain_2_attack[4], MEB_word_adjacencies_attack[4], MEB_circuits_s208_attack[4], MEB_bn_cat_mixed_species_brain_1_attack[4], MEB_facebook_414_attack[4], MEB_facebook_686_attack[4], MEB_polbooks_attack[4], MEB_jazz_attack[4], MEB_facebook_348_attack[4], MEB_circuits_s420_attack[4], MEB_corruption_attack[4], MEB_facebook_0_attack[4], MEB_circuits_s838_attack[4], MEB_crime_net_attack[4], MEB_facebook_1684_attack[4]]
data_frame_R['MEC']=[MEC_bn_macaque_rhesus_brain_2_attack[4], MEC_word_adjacencies_attack[4], MEC_circuits_s208_attack[4], MEC_bn_cat_mixed_species_brain_1_attack[4], MEC_facebook_414_attack[4], MEC_facebook_686_attack[4], MEC_polbooks_attack[4], MEC_jazz_attack[4], MEC_facebook_348_attack[4], MEC_circuits_s420_attack[4], MEC_corruption_attack[4], MEC_facebook_0_attack[4], MEC_circuits_s838_attack[4], MEC_crime_net_attack[4], MEC_facebook_1684_attack[4]]
data_frame_R['MED']=[MED_bn_macaque_rhesus_brain_2_attack[4], MED_word_adjacencies_attack[4], MED_circuits_s208_attack[4], MED_bn_cat_mixed_species_brain_1_attack[4], MED_facebook_414_attack[4], MED_facebook_686_attack[4], MED_polbooks_attack[4], MED_jazz_attack[4], MED_facebook_348_attack[4], MED_circuits_s420_attack[4], MED_corruption_attack[4], MED_facebook_0_attack[4], MED_circuits_s838_attack[4], MED_crime_net_attack[4], MED_facebook_1684_attack[4]]
data_frame_R['DWT']=[DWT_bn_macaque_rhesus_brain_2_attack[4], DWT_word_adjacencies_attack[4], DWT_circuits_s208_attack[4], DWT_bn_cat_mixed_species_brain_1_attack[4], DWT_facebook_414_attack[4], DWT_facebook_686_attack[4], DWT_polbooks_attack[4], DWT_jazz_attack[4], DWT_facebook_348_attack[4], DWT_circuits_s420_attack[4], DWT_corruption_attack[4], DWT_facebook_0_attack[4], DWT_circuits_s838_attack[4], DWT_crime_net_attack[4], DWT_facebook_1684_attack[4]]
data_frame_R['random']=[random_bn_macaque_rhesus_brain_2_attack[4], random_word_adjacencies_attack[4], random_circuits_s208_attack[4], random_bn_cat_mixed_species_brain_1_attack[4], random_facebook_414_attack[4], random_facebook_686_attack[4], random_polbooks_attack[4], random_jazz_attack[4], random_facebook_348_attack[4], random_circuits_s420_attack[4], random_corruption_attack[4], random_facebook_0_attack[4], random_circuits_s838_attack[4], random_crime_net_attack[4], random_facebook_1684_attack[4]]
data_frame_R['greedy']=[greedy_bn_macaque_rhesus_brain_2_attack[4], greedy_word_adjacencies_attack[4], greedy_circuits_s208_attack[4], greedy_bn_cat_mixed_species_brain_1_attack[4], greedy_facebook_414_attack[4], greedy_facebook_686_attack[4], greedy_polbooks_attack[4], greedy_jazz_attack[4], greedy_facebook_348_attack[4], greedy_circuits_s420_attack[4], greedy_corruption_attack[4], greedy_facebook_0_attack[4], greedy_circuits_s838_attack[4], greedy_crime_net_attack[4], greedy_facebook_1684_attack[4]]
data_frame_R.to_csv("../results/R_values.csv")

data_frame_InterE= pd.DataFrame(columns=cols)
data_frame_InterE['network']=net_names

data_frame_InterE['BC']=[BC_bn_macaque_rhesus_brain_2_attack[5], BC_word_adjacencies_attack[5], BC_circuits_s208_attack[5], BC_bn_cat_mixed_species_brain_1_attack[5], BC_facebook_414_attack[5], BC_facebook_686_attack[5], BC_polbooks_attack[5], BC_jazz_attack[5], BC_facebook_348_attack[5], BC_circuits_s420_attack[5], BC_corruption_attack[5], BC_facebook_0_attack[5], BC_circuits_s838_attack[5], BC_crime_net_attack[5], BC_facebook_1684_attack[5]]
data_frame_InterE['CC']=[CC_bn_macaque_rhesus_brain_2_attack[5], CC_word_adjacencies_attack[5], CC_circuits_s208_attack[5], CC_bn_cat_mixed_species_brain_1_attack[5], CC_facebook_414_attack[5], CC_facebook_686_attack[5], CC_polbooks_attack[5], CC_jazz_attack[5], CC_facebook_348_attack[5], CC_circuits_s420_attack[5], CC_corruption_attack[5], CC_facebook_0_attack[5], CC_circuits_s838_attack[5], CC_crime_net_attack[5], CC_facebook_1684_attack[5]]
data_frame_InterE['DC']=[DC_bn_macaque_rhesus_brain_2_attack[5], DC_word_adjacencies_attack[5], DC_circuits_s208_attack[5], DC_bn_cat_mixed_species_brain_1_attack[5], DC_facebook_414_attack[5], DC_facebook_686_attack[5], DC_polbooks_attack[5], DC_jazz_attack[5], DC_facebook_348_attack[5], DC_circuits_s420_attack[5], DC_corruption_attack[5], DC_facebook_0_attack[5], DC_circuits_s838_attack[5], DC_crime_net_attack[5], DC_facebook_1684_attack[5]]
data_frame_InterE['NBC']=[NBC_bn_macaque_rhesus_brain_2_attack[5], NBC_word_adjacencies_attack[5], NBC_circuits_s208_attack[5], NBC_bn_cat_mixed_species_brain_1_attack[5], NBC_facebook_414_attack[5], NBC_facebook_686_attack[5], NBC_polbooks_attack[5], NBC_jazz_attack[5], NBC_facebook_348_attack[5], NBC_circuits_s420_attack[5], NBC_corruption_attack[5], NBC_facebook_0_attack[5], NBC_circuits_s838_attack[5], NBC_crime_net_attack[5], NBC_facebook_1684_attack[5]]
data_frame_InterE['PLCi']=[PLCi_bn_macaque_rhesus_brain_2_attack[5], PLCi_word_adjacencies_attack[5], PLCi_circuits_s208_attack[5], PLCi_bn_cat_mixed_species_brain_1_attack[5], PLCi_facebook_414_attack[5], PLCi_facebook_686_attack[5], PLCi_polbooks_attack[5], PLCi_jazz_attack[5], PLCi_facebook_348_attack[5], PLCi_circuits_s420_attack[5], PLCi_corruption_attack[5], PLCi_facebook_0_attack[5], PLCi_circuits_s838_attack[5], PLCi_crime_net_attack[5], PLCi_facebook_1684_attack[5]]
data_frame_InterE['PR']=[PR_bn_macaque_rhesus_brain_2_attack[5], PR_word_adjacencies_attack[5], PR_circuits_s208_attack[5], PR_bn_cat_mixed_species_brain_1_attack[5], PR_facebook_414_attack[5], PR_facebook_686_attack[5], PR_polbooks_attack[5], PR_jazz_attack[5], PR_facebook_348_attack[5], PR_circuits_s420_attack[5], PR_corruption_attack[5], PR_facebook_0_attack[5], PR_circuits_s838_attack[5], PR_crime_net_attack[5], PR_facebook_1684_attack[5]]
data_frame_InterE['MEB']=[MEB_bn_macaque_rhesus_brain_2_attack[5], MEB_word_adjacencies_attack[5], MEB_circuits_s208_attack[5], MEB_bn_cat_mixed_species_brain_1_attack[5], MEB_facebook_414_attack[5], MEB_facebook_686_attack[5], MEB_polbooks_attack[5], MEB_jazz_attack[5], MEB_facebook_348_attack[5], MEB_circuits_s420_attack[5], MEB_corruption_attack[5], MEB_facebook_0_attack[5], MEB_circuits_s838_attack[5], MEB_crime_net_attack[5], MEB_facebook_1684_attack[5]]
data_frame_InterE['MEC']=[MEC_bn_macaque_rhesus_brain_2_attack[5], MEC_word_adjacencies_attack[5], MEC_circuits_s208_attack[5], MEC_bn_cat_mixed_species_brain_1_attack[5], MEC_facebook_414_attack[5], MEC_facebook_686_attack[5], MEC_polbooks_attack[5], MEC_jazz_attack[5], MEC_facebook_348_attack[5], MEC_circuits_s420_attack[5], MEC_corruption_attack[5], MEC_facebook_0_attack[5], MEC_circuits_s838_attack[5], MEC_crime_net_attack[5], MEC_facebook_1684_attack[5]]
data_frame_InterE['MED']=[MED_bn_macaque_rhesus_brain_2_attack[5], MED_word_adjacencies_attack[5], MED_circuits_s208_attack[5], MED_bn_cat_mixed_species_brain_1_attack[5], MED_facebook_414_attack[5], MED_facebook_686_attack[5], MED_polbooks_attack[5], MED_jazz_attack[5], MED_facebook_348_attack[5], MED_circuits_s420_attack[5], MED_corruption_attack[5], MED_facebook_0_attack[5], MED_circuits_s838_attack[5], MED_crime_net_attack[5], MED_facebook_1684_attack[5]]
data_frame_InterE['DWT']=[DWT_bn_macaque_rhesus_brain_2_attack[5], DWT_word_adjacencies_attack[5], DWT_circuits_s208_attack[5], DWT_bn_cat_mixed_species_brain_1_attack[5], DWT_facebook_414_attack[5], DWT_facebook_686_attack[5], DWT_polbooks_attack[5], DWT_jazz_attack[5], DWT_facebook_348_attack[5], DWT_circuits_s420_attack[5], DWT_corruption_attack[5], DWT_facebook_0_attack[5], DWT_circuits_s838_attack[5], DWT_crime_net_attack[5], DWT_facebook_1684_attack[5]]
data_frame_InterE['random']=[random_bn_macaque_rhesus_brain_2_attack[5], random_word_adjacencies_attack[5], random_circuits_s208_attack[5], random_bn_cat_mixed_species_brain_1_attack[5], random_facebook_414_attack[5], random_facebook_686_attack[5], random_polbooks_attack[5], random_jazz_attack[5], random_facebook_348_attack[5], random_circuits_s420_attack[5], random_corruption_attack[5], random_facebook_0_attack[5], random_circuits_s838_attack[5], random_crime_net_attack[5], random_facebook_1684_attack[5]]
data_frame_InterE['greedy']=[greedy_bn_macaque_rhesus_brain_2_attack[5], greedy_word_adjacencies_attack[5], greedy_circuits_s208_attack[5], greedy_bn_cat_mixed_species_brain_1_attack[5], greedy_facebook_414_attack[5], greedy_facebook_686_attack[5], greedy_polbooks_attack[5], greedy_jazz_attack[5], greedy_facebook_348_attack[5], greedy_circuits_s420_attack[5], greedy_corruption_attack[5], greedy_facebook_0_attack[5], greedy_circuits_s838_attack[5], greedy_crime_net_attack[5], greedy_facebook_1684_attack[5]]
data_frame_InterE.to_csv("../results/InterE_values.csv")

data_frame_time= pd.DataFrame(columns=cols)
data_frame_time['network']=net_names

data_frame_time['BC']=[BC_bn_macaque_rhesus_brain_2_attack[6], BC_word_adjacencies_attack[6], BC_circuits_s208_attack[6], BC_bn_cat_mixed_species_brain_1_attack[6], BC_facebook_414_attack[6], BC_facebook_686_attack[6], BC_polbooks_attack[6], BC_jazz_attack[6], BC_facebook_348_attack[6], BC_circuits_s420_attack[6], BC_corruption_attack[6], BC_facebook_0_attack[6], BC_circuits_s838_attack[6], BC_crime_net_attack[6], BC_facebook_1684_attack[6]]
data_frame_time['CC']=[CC_bn_macaque_rhesus_brain_2_attack[6], CC_word_adjacencies_attack[6], CC_circuits_s208_attack[6], CC_bn_cat_mixed_species_brain_1_attack[6], CC_facebook_414_attack[6], CC_facebook_686_attack[6], CC_polbooks_attack[6], CC_jazz_attack[6], CC_facebook_348_attack[6], CC_circuits_s420_attack[6], CC_corruption_attack[6], CC_facebook_0_attack[6], CC_circuits_s838_attack[6], CC_crime_net_attack[6], CC_facebook_1684_attack[6]]
data_frame_time['DC']=[DC_bn_macaque_rhesus_brain_2_attack[6], DC_word_adjacencies_attack[6], DC_circuits_s208_attack[6], DC_bn_cat_mixed_species_brain_1_attack[6], DC_facebook_414_attack[6], DC_facebook_686_attack[6], DC_polbooks_attack[6], DC_jazz_attack[6], DC_facebook_348_attack[6], DC_circuits_s420_attack[6], DC_corruption_attack[6], DC_facebook_0_attack[6], DC_circuits_s838_attack[6], DC_crime_net_attack[6], DC_facebook_1684_attack[6]]
data_frame_time['NBC']=[NBC_bn_macaque_rhesus_brain_2_attack[6], NBC_word_adjacencies_attack[6], NBC_circuits_s208_attack[6], NBC_bn_cat_mixed_species_brain_1_attack[6], NBC_facebook_414_attack[6], NBC_facebook_686_attack[6], NBC_polbooks_attack[6], NBC_jazz_attack[6], NBC_facebook_348_attack[6], NBC_circuits_s420_attack[6], NBC_corruption_attack[6], NBC_facebook_0_attack[6], NBC_circuits_s838_attack[6], NBC_crime_net_attack[6], NBC_facebook_1684_attack[6]]
data_frame_time['PLCi']=[PLCi_bn_macaque_rhesus_brain_2_attack[6], PLCi_word_adjacencies_attack[6], PLCi_circuits_s208_attack[6], PLCi_bn_cat_mixed_species_brain_1_attack[6], PLCi_facebook_414_attack[6], PLCi_facebook_686_attack[6], PLCi_polbooks_attack[6], PLCi_jazz_attack[6], PLCi_facebook_348_attack[6], PLCi_circuits_s420_attack[6], PLCi_corruption_attack[6], PLCi_facebook_0_attack[6], PLCi_circuits_s838_attack[6], PLCi_crime_net_attack[6], PLCi_facebook_1684_attack[6]]
data_frame_time['PR']=[PR_bn_macaque_rhesus_brain_2_attack[6], PR_word_adjacencies_attack[6], PR_circuits_s208_attack[6], PR_bn_cat_mixed_species_brain_1_attack[6], PR_facebook_414_attack[6], PR_facebook_686_attack[6], PR_polbooks_attack[6], PR_jazz_attack[6], PR_facebook_348_attack[6], PR_circuits_s420_attack[6], PR_corruption_attack[6], PR_facebook_0_attack[6], PR_circuits_s838_attack[6], PR_crime_net_attack[6], PR_facebook_1684_attack[6]]
data_frame_time['MEB']=[MEB_bn_macaque_rhesus_brain_2_attack[6], MEB_word_adjacencies_attack[6], MEB_circuits_s208_attack[6], MEB_bn_cat_mixed_species_brain_1_attack[6], MEB_facebook_414_attack[6], MEB_facebook_686_attack[6], MEB_polbooks_attack[6], MEB_jazz_attack[6], MEB_facebook_348_attack[6], MEB_circuits_s420_attack[6], MEB_corruption_attack[6], MEB_facebook_0_attack[6], MEB_circuits_s838_attack[6], MEB_crime_net_attack[6], MEB_facebook_1684_attack[6]]
data_frame_time['MEC']=[MEC_bn_macaque_rhesus_brain_2_attack[6], MEC_word_adjacencies_attack[6], MEC_circuits_s208_attack[6], MEC_bn_cat_mixed_species_brain_1_attack[6], MEC_facebook_414_attack[6], MEC_facebook_686_attack[6], MEC_polbooks_attack[6], MEC_jazz_attack[6], MEC_facebook_348_attack[6], MEC_circuits_s420_attack[6], MEC_corruption_attack[6], MEC_facebook_0_attack[6], MEC_circuits_s838_attack[6], MEC_crime_net_attack[6], MEC_facebook_1684_attack[6]]
data_frame_time['MED']=[MED_bn_macaque_rhesus_brain_2_attack[6], MED_word_adjacencies_attack[6], MED_circuits_s208_attack[6], MED_bn_cat_mixed_species_brain_1_attack[6], MED_facebook_414_attack[6], MED_facebook_686_attack[6], MED_polbooks_attack[6], MED_jazz_attack[6], MED_facebook_348_attack[6], MED_circuits_s420_attack[6], MED_corruption_attack[6], MED_facebook_0_attack[6], MED_circuits_s838_attack[6], MED_crime_net_attack[6], MED_facebook_1684_attack[6]]
data_frame_time['DWT']=[DWT_bn_macaque_rhesus_brain_2_attack[6], DWT_word_adjacencies_attack[6], DWT_circuits_s208_attack[6], DWT_bn_cat_mixed_species_brain_1_attack[6], DWT_facebook_414_attack[6], DWT_facebook_686_attack[6], DWT_polbooks_attack[6], DWT_jazz_attack[6], DWT_facebook_348_attack[6], DWT_circuits_s420_attack[6], DWT_corruption_attack[6], DWT_facebook_0_attack[6], DWT_circuits_s838_attack[6], DWT_crime_net_attack[6], DWT_facebook_1684_attack[6]]
data_frame_time['random']=[random_bn_macaque_rhesus_brain_2_attack[6], random_word_adjacencies_attack[6], random_circuits_s208_attack[6], random_bn_cat_mixed_species_brain_1_attack[6], random_facebook_414_attack[6], random_facebook_686_attack[6], random_polbooks_attack[6], random_jazz_attack[6], random_facebook_348_attack[6], random_circuits_s420_attack[6], random_corruption_attack[6], random_facebook_0_attack[6], random_circuits_s838_attack[6], random_crime_net_attack[6], random_facebook_1684_attack[6]]
data_frame_time['greedy']=[greedy_bn_macaque_rhesus_brain_2_attack[6], greedy_word_adjacencies_attack[6], greedy_circuits_s208_attack[6], greedy_bn_cat_mixed_species_brain_1_attack[6], greedy_facebook_414_attack[6], greedy_facebook_686_attack[6], greedy_polbooks_attack[6], greedy_jazz_attack[6], greedy_facebook_348_attack[6], greedy_circuits_s420_attack[6], greedy_corruption_attack[6], greedy_facebook_0_attack[6], greedy_circuits_s838_attack[6], greedy_crime_net_attack[6], greedy_facebook_1684_attack[6]]
data_frame_time.to_csv("../results/time_values.csv")

##### community_attacks

In [15]:
data_frame_lcc_com_batch = pd.DataFrame(columns=['network','BC','NBC','MEB', 'MEC', 'MED'])
data_frame_lcc_com_batch['network']=net_names

data_frame_lcc_com_batch['BC']=[list(BC_bn_macaque_rhesus_brain_2_community_attack[0].values()), list(BC_word_adjacencies_community_attack[0].values()), list(BC_circuits_s208_community_attack[0].values()), list(BC_bn_cat_mixed_species_brain_1_community_attack[0].values()), list(BC_facebook_414_community_attack[0].values()), list(BC_facebook_686_community_attack[0].values()), list(BC_polbooks_community_attack[0].values()), list(BC_jazz_community_attack[0].values()), list(BC_facebook_348_community_attack[0].values()), list(BC_circuits_s420_community_attack[0].values()), list(BC_corruption_community_attack[0].values()), list(BC_facebook_0_community_attack[0].values()), list(BC_circuits_s838_community_attack[0].values()), list(BC_crime_net_community_attack[0].values()), list(BC_facebook_1684_community_attack[0].values())]
data_frame_lcc_com_batch['NBC']=[list(NBC_bn_macaque_rhesus_brain_2_community_attack[0].values()), list(NBC_word_adjacencies_community_attack[0].values()), list(NBC_circuits_s208_community_attack[0].values()), list(NBC_bn_cat_mixed_species_brain_1_community_attack[0].values()), list(NBC_facebook_414_community_attack[0].values()), list(NBC_facebook_686_community_attack[0].values()), list(NBC_polbooks_community_attack[0].values()), list(NBC_jazz_community_attack[0].values()), list(NBC_facebook_348_community_attack[0].values()), list(NBC_circuits_s420_community_attack[0].values()), list(NBC_corruption_community_attack[0].values()), list(NBC_facebook_0_community_attack[0].values()), list(NBC_circuits_s838_community_attack[0].values()), list(NBC_crime_net_community_attack[0].values()), list(NBC_facebook_1684_community_attack[0].values())]
data_frame_lcc_com_batch['MEB']=[list(MEB_bn_macaque_rhesus_brain_2_community_attack[0].values()), list(MEB_word_adjacencies_community_attack[0].values()), list(MEB_circuits_s208_community_attack[0].values()), list(MEB_bn_cat_mixed_species_brain_1_community_attack[0].values()), list(MEB_facebook_414_community_attack[0].values()), list(MEB_facebook_686_community_attack[0].values()), list(MEB_polbooks_community_attack[0].values()), list(MEB_jazz_community_attack[0].values()), list(MEB_facebook_348_community_attack[0].values()), list(MEB_circuits_s420_community_attack[0].values()), list(MEB_corruption_community_attack[0].values()), list(MEB_facebook_0_community_attack[0].values()), list(MEB_circuits_s838_community_attack[0].values()), list(MEB_crime_net_community_attack[0].values()), list(MEB_facebook_1684_community_attack[0].values())]
data_frame_lcc_com_batch['MEC']=[list(MEC_bn_macaque_rhesus_brain_2_community_attack[0].values()), list(MEC_word_adjacencies_community_attack[0].values()), list(MEC_circuits_s208_community_attack[0].values()), list(MEC_bn_cat_mixed_species_brain_1_community_attack[0].values()), list(MEC_facebook_414_community_attack[0].values()), list(MEC_facebook_686_community_attack[0].values()), list(MEC_polbooks_community_attack[0].values()), list(MEC_jazz_community_attack[0].values()), list(MEC_facebook_348_community_attack[0].values()), list(MEC_circuits_s420_community_attack[0].values()), list(MEC_corruption_community_attack[0].values()), list(MEC_facebook_0_community_attack[0].values()), list(MEC_circuits_s838_community_attack[0].values()), list(MEC_crime_net_community_attack[0].values()), list(MEC_facebook_1684_community_attack[0].values())]
data_frame_lcc_com_batch['MED']=[list(MED_bn_macaque_rhesus_brain_2_community_attack[0].values()), list(MED_word_adjacencies_community_attack[0].values()), list(MED_circuits_s208_community_attack[0].values()), list(MED_bn_cat_mixed_species_brain_1_community_attack[0].values()), list(MED_facebook_414_community_attack[0].values()), list(MED_facebook_686_community_attack[0].values()), list(MED_polbooks_community_attack[0].values()), list(MED_jazz_community_attack[0].values()), list(MED_facebook_348_community_attack[0].values()), list(MED_circuits_s420_community_attack[0].values()), list(MED_corruption_community_attack[0].values()), list(MED_facebook_0_community_attack[0].values()), list(MED_circuits_s838_community_attack[0].values()), list(MED_crime_net_community_attack[0].values()), list(MED_facebook_1684_community_attack[0].values())]
data_frame_lcc_com_batch.to_csv("../results/lcc_com_batch_values.csv")

##############################################################################################

data_frame_efficiency_com_batch = pd.DataFrame(columns=['network','BC','NBC','MEB', 'MEC', 'MED'])
data_frame_efficiency_com_batch['network']=net_names

data_frame_efficiency_com_batch['BC']=[list(BC_bn_macaque_rhesus_brain_2_community_attack[1].values()), list(BC_word_adjacencies_community_attack[1].values()), list(BC_circuits_s208_community_attack[1].values()), list(BC_bn_cat_mixed_species_brain_1_community_attack[1].values()), list(BC_facebook_414_community_attack[1].values()), list(BC_facebook_686_community_attack[1].values()), list(BC_polbooks_community_attack[1].values()), list(BC_jazz_community_attack[1].values()), list(BC_facebook_348_community_attack[1].values()), list(BC_circuits_s420_community_attack[1].values()), list(BC_corruption_community_attack[1].values()), list(BC_facebook_0_community_attack[1].values()), list(BC_circuits_s838_community_attack[1].values()), list(BC_crime_net_community_attack[1].values()), list(BC_facebook_1684_community_attack[1].values())]
data_frame_efficiency_com_batch['NBC']=[list(NBC_bn_macaque_rhesus_brain_2_community_attack[1].values()), list(NBC_word_adjacencies_community_attack[1].values()), list(NBC_circuits_s208_community_attack[1].values()), list(NBC_bn_cat_mixed_species_brain_1_community_attack[1].values()), list(NBC_facebook_414_community_attack[1].values()), list(NBC_facebook_686_community_attack[1].values()), list(NBC_polbooks_community_attack[1].values()), list(NBC_jazz_community_attack[1].values()), list(NBC_facebook_348_community_attack[1].values()), list(NBC_circuits_s420_community_attack[1].values()), list(NBC_corruption_community_attack[1].values()), list(NBC_facebook_0_community_attack[1].values()), list(NBC_circuits_s838_community_attack[1].values()), list(NBC_crime_net_community_attack[1].values()), list(NBC_facebook_1684_community_attack[1].values())]
data_frame_efficiency_com_batch['MEB']=[list(MEB_bn_macaque_rhesus_brain_2_community_attack[1].values()), list(MEB_word_adjacencies_community_attack[1].values()), list(MEB_circuits_s208_community_attack[1].values()), list(MEB_bn_cat_mixed_species_brain_1_community_attack[1].values()), list(MEB_facebook_414_community_attack[1].values()), list(MEB_facebook_686_community_attack[1].values()), list(MEB_polbooks_community_attack[1].values()), list(MEB_jazz_community_attack[1].values()), list(MEB_facebook_348_community_attack[1].values()), list(MEB_circuits_s420_community_attack[1].values()), list(MEB_corruption_community_attack[1].values()), list(MEB_facebook_0_community_attack[1].values()), list(MEB_circuits_s838_community_attack[1].values()), list(MEB_crime_net_community_attack[1].values()), list(MEB_facebook_1684_community_attack[1].values())]
data_frame_efficiency_com_batch['MEC']=[list(MEC_bn_macaque_rhesus_brain_2_community_attack[1].values()), list(MEC_word_adjacencies_community_attack[1].values()), list(MEC_circuits_s208_community_attack[1].values()), list(MEC_bn_cat_mixed_species_brain_1_community_attack[1].values()), list(MEC_facebook_414_community_attack[1].values()), list(MEC_facebook_686_community_attack[1].values()), list(MEC_polbooks_community_attack[1].values()), list(MEC_jazz_community_attack[1].values()), list(MEC_facebook_348_community_attack[1].values()), list(MEC_circuits_s420_community_attack[1].values()), list(MEC_corruption_community_attack[1].values()), list(MEC_facebook_0_community_attack[1].values()), list(MEC_circuits_s838_community_attack[1].values()), list(MEC_crime_net_community_attack[1].values()), list(MEC_facebook_1684_community_attack[1].values())]
data_frame_efficiency_com_batch['MED']=[list(MED_bn_macaque_rhesus_brain_2_community_attack[1].values()), list(MED_word_adjacencies_community_attack[1].values()), list(MED_circuits_s208_community_attack[1].values()), list(MED_bn_cat_mixed_species_brain_1_community_attack[1].values()), list(MED_facebook_414_community_attack[1].values()), list(MED_facebook_686_community_attack[1].values()), list(MED_polbooks_community_attack[1].values()), list(MED_jazz_community_attack[1].values()), list(MED_facebook_348_community_attack[1].values()), list(MED_circuits_s420_community_attack[1].values()), list(MED_corruption_community_attack[1].values()), list(MED_facebook_0_community_attack[1].values()), list(MED_circuits_s838_community_attack[1].values()), list(MED_crime_net_community_attack[1].values()), list(MED_facebook_1684_community_attack[1].values())]
data_frame_efficiency_com_batch.to_csv("../results/efficiency_com_batch_values.csv")
###########################################################################################

data_frame_lcc_seq_com_batch = pd.DataFrame(columns=['network','BC','NBC','MEB', 'MEC', 'MED'])
data_frame_lcc_seq_com_batch['network']=net_names

data_frame_lcc_seq_com_batch['BC']=[list(BC_bn_macaque_rhesus_brain_2_community_attack[2].values()), list(BC_word_adjacencies_community_attack[2].values()), list(BC_circuits_s208_community_attack[2].values()), list(BC_bn_cat_mixed_species_brain_1_community_attack[2].values()), list(BC_facebook_414_community_attack[2].values()), list(BC_facebook_686_community_attack[2].values()), list(BC_polbooks_community_attack[2].values()), list(BC_jazz_community_attack[2].values()), list(BC_facebook_348_community_attack[2].values()), list(BC_circuits_s420_community_attack[2].values()), list(BC_corruption_community_attack[2].values()), list(BC_facebook_0_community_attack[2].values()), list(BC_circuits_s838_community_attack[2].values()), list(BC_crime_net_community_attack[2].values()), list(BC_facebook_1684_community_attack[2].values())]
data_frame_lcc_seq_com_batch['NBC']=[list(NBC_bn_macaque_rhesus_brain_2_community_attack[2].values()), list(NBC_word_adjacencies_community_attack[2].values()), list(NBC_circuits_s208_community_attack[2].values()), list(NBC_bn_cat_mixed_species_brain_1_community_attack[2].values()), list(NBC_facebook_414_community_attack[2].values()), list(NBC_facebook_686_community_attack[2].values()), list(NBC_polbooks_community_attack[2].values()), list(NBC_jazz_community_attack[2].values()), list(NBC_facebook_348_community_attack[2].values()), list(NBC_circuits_s420_community_attack[2].values()), list(NBC_corruption_community_attack[2].values()), list(NBC_facebook_0_community_attack[2].values()), list(NBC_circuits_s838_community_attack[2].values()), list(NBC_crime_net_community_attack[2].values()), list(NBC_facebook_1684_community_attack[2].values())]
data_frame_lcc_seq_com_batch['MEB']=[list(MEB_bn_macaque_rhesus_brain_2_community_attack[2].values()), list(MEB_word_adjacencies_community_attack[2].values()), list(MEB_circuits_s208_community_attack[2].values()), list(MEB_bn_cat_mixed_species_brain_1_community_attack[2].values()), list(MEB_facebook_414_community_attack[2].values()), list(MEB_facebook_686_community_attack[2].values()), list(MEB_polbooks_community_attack[2].values()), list(MEB_jazz_community_attack[2].values()), list(MEB_facebook_348_community_attack[2].values()), list(MEB_circuits_s420_community_attack[2].values()), list(MEB_corruption_community_attack[2].values()), list(MEB_facebook_0_community_attack[2].values()), list(MEB_circuits_s838_community_attack[2].values()), list(MEB_crime_net_community_attack[2].values()), list(MEB_facebook_1684_community_attack[2].values())]
data_frame_lcc_seq_com_batch['MEC']=[list(MEC_bn_macaque_rhesus_brain_2_community_attack[2].values()), list(MEC_word_adjacencies_community_attack[2].values()), list(MEC_circuits_s208_community_attack[2].values()), list(MEC_bn_cat_mixed_species_brain_1_community_attack[2].values()), list(MEC_facebook_414_community_attack[2].values()), list(MEC_facebook_686_community_attack[2].values()), list(MEC_polbooks_community_attack[2].values()), list(MEC_jazz_community_attack[2].values()), list(MEC_facebook_348_community_attack[2].values()), list(MEC_circuits_s420_community_attack[2].values()), list(MEC_corruption_community_attack[2].values()), list(MEC_facebook_0_community_attack[2].values()), list(MEC_circuits_s838_community_attack[2].values()), list(MEC_crime_net_community_attack[2].values()), list(MEC_facebook_1684_community_attack[2].values())]
data_frame_lcc_seq_com_batch['MED']=[list(MED_bn_macaque_rhesus_brain_2_community_attack[2].values()), list(MED_word_adjacencies_community_attack[2].values()), list(MED_circuits_s208_community_attack[2].values()), list(MED_bn_cat_mixed_species_brain_1_community_attack[2].values()), list(MED_facebook_414_community_attack[2].values()), list(MED_facebook_686_community_attack[2].values()), list(MED_polbooks_community_attack[2].values()), list(MED_jazz_community_attack[2].values()), list(MED_facebook_348_community_attack[2].values()), list(MED_circuits_s420_community_attack[2].values()), list(MED_corruption_community_attack[2].values()), list(MED_facebook_0_community_attack[2].values()), list(MED_circuits_s838_community_attack[2].values()), list(MED_crime_net_community_attack[2].values()), list(MED_facebook_1684_community_attack[2].values())]
data_frame_lcc_seq_com_batch.to_csv("../results/lcc_seq_com_batch_values.csv")
###########################################################################################

data_frame_efficiency_seq_com_batch = pd.DataFrame(columns=['network','BC','NBC','MEB', 'MEC', 'MED'])
data_frame_efficiency_seq_com_batch['network']=net_names

data_frame_efficiency_seq_com_batch['BC']=[list(BC_bn_macaque_rhesus_brain_2_community_attack[3].values()), list(BC_word_adjacencies_community_attack[3].values()), list(BC_circuits_s208_community_attack[3].values()), list(BC_bn_cat_mixed_species_brain_1_community_attack[3].values()), list(BC_facebook_414_community_attack[3].values()), list(BC_facebook_686_community_attack[3].values()), list(BC_polbooks_community_attack[3].values()), list(BC_jazz_community_attack[3].values()), list(BC_facebook_348_community_attack[3].values()), list(BC_circuits_s420_community_attack[3].values()), list(BC_corruption_community_attack[3].values()), list(BC_facebook_0_community_attack[3].values()), list(BC_circuits_s838_community_attack[3].values()), list(BC_crime_net_community_attack[3].values()), list(BC_facebook_1684_community_attack[3].values())]
data_frame_efficiency_seq_com_batch['NBC']=[list(NBC_bn_macaque_rhesus_brain_2_community_attack[3].values()), list(NBC_word_adjacencies_community_attack[3].values()), list(NBC_circuits_s208_community_attack[3].values()), list(NBC_bn_cat_mixed_species_brain_1_community_attack[3].values()), list(NBC_facebook_414_community_attack[3].values()), list(NBC_facebook_686_community_attack[3].values()), list(NBC_polbooks_community_attack[3].values()), list(NBC_jazz_community_attack[3].values()), list(NBC_facebook_348_community_attack[3].values()), list(NBC_circuits_s420_community_attack[3].values()), list(NBC_corruption_community_attack[3].values()), list(NBC_facebook_0_community_attack[3].values()), list(NBC_circuits_s838_community_attack[3].values()), list(NBC_crime_net_community_attack[3].values()), list(NBC_facebook_1684_community_attack[3].values())]
data_frame_efficiency_seq_com_batch['MEB']=[list(MEB_bn_macaque_rhesus_brain_2_community_attack[3].values()), list(MEB_word_adjacencies_community_attack[3].values()), list(MEB_circuits_s208_community_attack[3].values()), list(MEB_bn_cat_mixed_species_brain_1_community_attack[3].values()), list(MEB_facebook_414_community_attack[3].values()), list(MEB_facebook_686_community_attack[3].values()), list(MEB_polbooks_community_attack[3].values()), list(MEB_jazz_community_attack[3].values()), list(MEB_facebook_348_community_attack[3].values()), list(MEB_circuits_s420_community_attack[3].values()), list(MEB_corruption_community_attack[3].values()), list(MEB_facebook_0_community_attack[3].values()), list(MEB_circuits_s838_community_attack[3].values()), list(MEB_crime_net_community_attack[3].values()), list(MEB_facebook_1684_community_attack[3].values())]
data_frame_efficiency_seq_com_batch['MEC']=[list(MEC_bn_macaque_rhesus_brain_2_community_attack[3].values()), list(MEC_word_adjacencies_community_attack[3].values()), list(MEC_circuits_s208_community_attack[3].values()), list(MEC_bn_cat_mixed_species_brain_1_community_attack[3].values()), list(MEC_facebook_414_community_attack[3].values()), list(MEC_facebook_686_community_attack[3].values()), list(MEC_polbooks_community_attack[3].values()), list(MEC_jazz_community_attack[3].values()), list(MEC_facebook_348_community_attack[3].values()), list(MEC_circuits_s420_community_attack[3].values()), list(MEC_corruption_community_attack[3].values()), list(MEC_facebook_0_community_attack[3].values()), list(MEC_circuits_s838_community_attack[3].values()), list(MEC_crime_net_community_attack[3].values()), list(MEC_facebook_1684_community_attack[3].values())]
data_frame_efficiency_seq_com_batch['MED']=[list(MED_bn_macaque_rhesus_brain_2_community_attack[3].values()), list(MED_word_adjacencies_community_attack[3].values()), list(MED_circuits_s208_community_attack[3].values()), list(MED_bn_cat_mixed_species_brain_1_community_attack[3].values()), list(MED_facebook_414_community_attack[3].values()), list(MED_facebook_686_community_attack[3].values()), list(MED_polbooks_community_attack[3].values()), list(MED_jazz_community_attack[3].values()), list(MED_facebook_348_community_attack[3].values()), list(MED_circuits_s420_community_attack[3].values()), list(MED_corruption_community_attack[3].values()), list(MED_facebook_0_community_attack[3].values()), list(MED_circuits_s838_community_attack[3].values()), list(MED_crime_net_community_attack[3].values()), list(MED_facebook_1684_community_attack[3].values())]
data_frame_efficiency_seq_com_batch.to_csv("../results/efficiency_seq_com_batch_values.csv")
###########################################################################################

data_frame_R_com = pd.DataFrame(columns=['network','BC','NBC','MEB', 'MEC', 'MED'])
data_frame_R_com['network']=net_names

data_frame_R_com['BC']=[BC_bn_macaque_rhesus_brain_2_community_attack[4], BC_word_adjacencies_community_attack[4], BC_circuits_s208_community_attack[4], BC_bn_cat_mixed_species_brain_1_community_attack[4], BC_facebook_414_community_attack[4], BC_facebook_686_community_attack[4], BC_polbooks_community_attack[4], BC_jazz_community_attack[4], BC_facebook_348_community_attack[4], BC_circuits_s420_community_attack[4], BC_corruption_community_attack[4], BC_facebook_0_community_attack[4], BC_circuits_s838_community_attack[4], BC_crime_net_community_attack[4], BC_facebook_1684_community_attack[4]]
data_frame_R_com['NBC']=[NBC_bn_macaque_rhesus_brain_2_community_attack[4], NBC_word_adjacencies_community_attack[4], NBC_circuits_s208_community_attack[4], NBC_bn_cat_mixed_species_brain_1_community_attack[4], NBC_facebook_414_community_attack[4], NBC_facebook_686_community_attack[4], NBC_polbooks_community_attack[4], NBC_jazz_community_attack[4], NBC_facebook_348_community_attack[4], NBC_circuits_s420_community_attack[4], NBC_corruption_community_attack[4], NBC_facebook_0_community_attack[4], NBC_circuits_s838_community_attack[4], NBC_crime_net_community_attack[4], NBC_facebook_1684_community_attack[4]]
data_frame_R_com['MEB']=[MEB_bn_macaque_rhesus_brain_2_community_attack[4], MEB_word_adjacencies_community_attack[4], MEB_circuits_s208_community_attack[4], MEB_bn_cat_mixed_species_brain_1_community_attack[4], MEB_facebook_414_community_attack[4], MEB_facebook_686_community_attack[4], MEB_polbooks_community_attack[4], MEB_jazz_community_attack[4], MEB_facebook_348_community_attack[4], MEB_circuits_s420_community_attack[4], MEB_corruption_community_attack[4], MEB_facebook_0_community_attack[4], MEB_circuits_s838_community_attack[4], MEB_crime_net_community_attack[4], MEB_facebook_1684_community_attack[4]]
data_frame_R_com['MEC']=[MEC_bn_macaque_rhesus_brain_2_community_attack[4], MEC_word_adjacencies_community_attack[4], MEC_circuits_s208_community_attack[4], MEC_bn_cat_mixed_species_brain_1_community_attack[4], MEC_facebook_414_community_attack[4], MEC_facebook_686_community_attack[4], MEC_polbooks_community_attack[4], MEC_jazz_community_attack[4], MEC_facebook_348_community_attack[4], MEC_circuits_s420_community_attack[4], MEC_corruption_community_attack[4], MEC_facebook_0_community_attack[4], MEC_circuits_s838_community_attack[4], MEC_crime_net_community_attack[4], MEC_facebook_1684_community_attack[4]]
data_frame_R_com['MED']=[MED_bn_macaque_rhesus_brain_2_community_attack[4], MED_word_adjacencies_community_attack[4], MED_circuits_s208_community_attack[4], MED_bn_cat_mixed_species_brain_1_community_attack[4], MED_facebook_414_community_attack[4], MED_facebook_686_community_attack[4], MED_polbooks_community_attack[4], MED_jazz_community_attack[4], MED_facebook_348_community_attack[4], MED_circuits_s420_community_attack[4], MED_corruption_community_attack[4], MED_facebook_0_community_attack[4], MED_circuits_s838_community_attack[4], MED_crime_net_community_attack[4], MED_facebook_1684_community_attack[4]]
data_frame_R_com.to_csv("../results/R_com_values.csv")
###########################################################################################

data_frame_InterE_com = pd.DataFrame(columns=['network','BC','NBC','MEB', 'MEC', 'MED'])
data_frame_InterE_com['network']=net_names

data_frame_InterE_com['BC']=[BC_bn_macaque_rhesus_brain_2_community_attack[5], BC_word_adjacencies_community_attack[5], BC_circuits_s208_community_attack[5], BC_bn_cat_mixed_species_brain_1_community_attack[5], BC_facebook_414_community_attack[5], BC_facebook_686_community_attack[5], BC_polbooks_community_attack[5], BC_jazz_community_attack[5], BC_facebook_348_community_attack[5], BC_circuits_s420_community_attack[5], BC_corruption_community_attack[5], BC_facebook_0_community_attack[5], BC_circuits_s838_community_attack[5], BC_crime_net_community_attack[5], BC_facebook_1684_community_attack[5]]
data_frame_InterE_com['NBC']=[NBC_bn_macaque_rhesus_brain_2_community_attack[5], NBC_word_adjacencies_community_attack[5], NBC_circuits_s208_community_attack[5], NBC_bn_cat_mixed_species_brain_1_community_attack[5], NBC_facebook_414_community_attack[5], NBC_facebook_686_community_attack[5], NBC_polbooks_community_attack[5], NBC_jazz_community_attack[5], NBC_facebook_348_community_attack[5], NBC_circuits_s420_community_attack[5], NBC_corruption_community_attack[5], NBC_facebook_0_community_attack[5], NBC_circuits_s838_community_attack[5], NBC_crime_net_community_attack[5], NBC_facebook_1684_community_attack[5]]
data_frame_InterE_com['MEB']=[MEB_bn_macaque_rhesus_brain_2_community_attack[5], MEB_word_adjacencies_community_attack[5], MEB_circuits_s208_community_attack[5], MEB_bn_cat_mixed_species_brain_1_community_attack[5], MEB_facebook_414_community_attack[5], MEB_facebook_686_community_attack[5], MEB_polbooks_community_attack[5], MEB_jazz_community_attack[5], MEB_facebook_348_community_attack[5], MEB_circuits_s420_community_attack[5], MEB_corruption_community_attack[5], MEB_facebook_0_community_attack[5], MEB_circuits_s838_community_attack[5], MEB_crime_net_community_attack[5], MEB_facebook_1684_community_attack[5]]
data_frame_InterE_com['MEC']=[MEC_bn_macaque_rhesus_brain_2_community_attack[5], MEC_word_adjacencies_community_attack[5], MEC_circuits_s208_community_attack[5], MEC_bn_cat_mixed_species_brain_1_community_attack[5], MEC_facebook_414_community_attack[5], MEC_facebook_686_community_attack[5], MEC_polbooks_community_attack[5], MEC_jazz_community_attack[5], MEC_facebook_348_community_attack[5], MEC_circuits_s420_community_attack[5], MEC_corruption_community_attack[5], MEC_facebook_0_community_attack[5], MEC_circuits_s838_community_attack[5], MEC_crime_net_community_attack[5], MEC_facebook_1684_community_attack[5]]
data_frame_InterE_com['MED']=[MED_bn_macaque_rhesus_brain_2_community_attack[5], MED_word_adjacencies_community_attack[5], MED_circuits_s208_community_attack[5], MED_bn_cat_mixed_species_brain_1_community_attack[5], MED_facebook_414_community_attack[5], MED_facebook_686_community_attack[5], MED_polbooks_community_attack[5], MED_jazz_community_attack[5], MED_facebook_348_community_attack[5], MED_circuits_s420_community_attack[5], MED_corruption_community_attack[5], MED_facebook_0_community_attack[5], MED_circuits_s838_community_attack[5], MED_crime_net_community_attack[5], MED_facebook_1684_community_attack[5]]
data_frame_InterE_com.to_csv("../results/InterE_com_values.csv")
###########################################################################################

data_frame_time_com = pd.DataFrame(columns=['network','BC','NBC','MEB', 'MEC', 'MED'])
data_frame_time_com['network']=net_names

data_frame_time_com['BC']=[BC_bn_macaque_rhesus_brain_2_community_attack[6], BC_word_adjacencies_community_attack[6], BC_circuits_s208_community_attack[6], BC_bn_cat_mixed_species_brain_1_community_attack[6], BC_facebook_414_community_attack[6], BC_facebook_686_community_attack[6], BC_polbooks_community_attack[6], BC_jazz_community_attack[6], BC_facebook_348_community_attack[6], BC_circuits_s420_community_attack[6], BC_corruption_community_attack[6], BC_facebook_0_community_attack[6], BC_circuits_s838_community_attack[6], BC_crime_net_community_attack[6], BC_facebook_1684_community_attack[6]]
data_frame_time_com['NBC']=[NBC_bn_macaque_rhesus_brain_2_community_attack[6], NBC_word_adjacencies_community_attack[6], NBC_circuits_s208_community_attack[6], NBC_bn_cat_mixed_species_brain_1_community_attack[6], NBC_facebook_414_community_attack[6], NBC_facebook_686_community_attack[6], NBC_polbooks_community_attack[6], NBC_jazz_community_attack[6], NBC_facebook_348_community_attack[6], NBC_circuits_s420_community_attack[6], NBC_corruption_community_attack[6], NBC_facebook_0_community_attack[6], NBC_circuits_s838_community_attack[6], NBC_crime_net_community_attack[6], NBC_facebook_1684_community_attack[6]]
data_frame_time_com['MEB']=[MEB_bn_macaque_rhesus_brain_2_community_attack[6], MEB_word_adjacencies_community_attack[6], MEB_circuits_s208_community_attack[6], MEB_bn_cat_mixed_species_brain_1_community_attack[6], MEB_facebook_414_community_attack[6], MEB_facebook_686_community_attack[6], MEB_polbooks_community_attack[6], MEB_jazz_community_attack[6], MEB_facebook_348_community_attack[6], MEB_circuits_s420_community_attack[6], MEB_corruption_community_attack[6], MEB_facebook_0_community_attack[6], MEB_circuits_s838_community_attack[6], MEB_crime_net_community_attack[6], MEB_facebook_1684_community_attack[6]]
data_frame_time_com['MEC']=[MEC_bn_macaque_rhesus_brain_2_community_attack[6], MEC_word_adjacencies_community_attack[6], MEC_circuits_s208_community_attack[6], MEC_bn_cat_mixed_species_brain_1_community_attack[6], MEC_facebook_414_community_attack[6], MEC_facebook_686_community_attack[6], MEC_polbooks_community_attack[6], MEC_jazz_community_attack[6], MEC_facebook_348_community_attack[6], MEC_circuits_s420_community_attack[6], MEC_corruption_community_attack[6], MEC_facebook_0_community_attack[6], MEC_circuits_s838_community_attack[6], MEC_crime_net_community_attack[6], MEC_facebook_1684_community_attack[6]]
data_frame_time_com['MED']=[MED_bn_macaque_rhesus_brain_2_community_attack[6], MED_word_adjacencies_community_attack[6], MED_circuits_s208_community_attack[6], MED_bn_cat_mixed_species_brain_1_community_attack[6], MED_facebook_414_community_attack[6], MED_facebook_686_community_attack[6], MED_polbooks_community_attack[6], MED_jazz_community_attack[6], MED_facebook_348_community_attack[6], MED_circuits_s420_community_attack[6], MED_corruption_community_attack[6], MED_facebook_0_community_attack[6], MED_circuits_s838_community_attack[6], MED_crime_net_community_attack[6], MED_facebook_1684_community_attack[6]]
data_frame_time_com.to_csv("../results/time_com_values.csv")



##### edge_attacks

In [302]:
data_frame_lcc_edge = pd.DataFrame(columns=['network','random','BC','greedy'])
data_frame_lcc_edge['network']=net_names[:len(net_names)-1]

data_frame_lcc_edge['random']=[list(random_bn_macaque_rhesus_brain_2_edge_attack[0].values()), list(random_word_adjacencies_edge_attack[0].values()), list(random_circuits_s208_edge_attack[0].values()), list(random_bn_cat_mixed_species_brain_1_edge_attack[0].values()), list(random_facebook_414_edge_attack[0].values()), list(random_facebook_686_edge_attack[0].values()), list(random_polbooks_edge_attack[0].values()), list(random_jazz_edge_attack[0].values()), list(random_facebook_348_edge_attack[0].values()), list(random_circuits_s420_edge_attack[0].values()), list(random_corruption_edge_attack[0].values()), list(random_facebook_0_edge_attack[0].values()), list(random_circuits_s838_edge_attack[0].values()), list(random_crime_net_edge_attack[0].values())]
data_frame_lcc_edge['BC']=[list(BC_bn_macaque_rhesus_brain_2_edge_attack[0].values()), list(BC_word_adjacencies_edge_attack[0].values()), list(BC_circuits_s208_edge_attack[0].values()), list(BC_bn_cat_mixed_species_brain_1_edge_attack[0].values()), list(BC_facebook_414_edge_attack[0].values()), list(BC_facebook_686_edge_attack[0].values()), list(BC_polbooks_edge_attack[0].values()), list(BC_jazz_edge_attack[0].values()), list(BC_facebook_348_edge_attack[0].values()), list(BC_circuits_s420_edge_attack[0].values()), list(BC_corruption_edge_attack[0].values()), list(BC_facebook_0_edge_attack[0].values()), list(BC_circuits_s838_edge_attack[0].values()), list(BC_crime_net_edge_attack[0].values())]
# data_frame_lcc_edge['community_BC']=[list(community_BC_bn_macaque_rhesus_brain_2_edge_attack[0].values()), list(community_BC_word_adjacencies_edge_attack[0].values()), list(community_BC_circuits_s208_edge_attack[0].values()), list(community_BC_bn_cat_mixed_species_brain_1_edge_attack[0].values()), list(community_BC_facebook_414_edge_attack[0].values()), list(community_BC_facebook_686_edge_attack[0].values()), list(community_BC_polbooks_edge_attack[0].values()), list(community_BC_jazz_edge_attack[0].values()), list(community_BC_facebook_348_edge_attack[0].values()), list(community_BC_circuits_s420_edge_attack[0].values()), list(community_BC_corruption_edge_attack[0].values()), list(community_BC_facebook_0_edge_attack[0].values()), list(community_BC_circuits_s838_edge_attack[0].values()), list(community_BC_crime_net_edge_attack[0].values())]
data_frame_lcc_edge['greedy']=[list(greedy_bn_macaque_rhesus_brain_2_edge_attack[0].values()), list(greedy_word_adjacencies_edge_attack[0].values()), list(greedy_circuits_s208_edge_attack[0].values()), list(greedy_bn_cat_mixed_species_brain_1_edge_attack[0].values()), list(greedy_facebook_414_edge_attack[0].values()), list(greedy_facebook_686_edge_attack[0].values()), list(greedy_polbooks_edge_attack[0].values()), list(greedy_jazz_edge_attack[0].values()), list(greedy_facebook_348_edge_attack[0].values()), list(greedy_circuits_s420_edge_attack[0].values()), list(greedy_corruption_edge_attack[0].values()), list(greedy_facebook_0_edge_attack[0].values()), list(greedy_circuits_s838_edge_attack[0].values()), list(greedy_crime_net_edge_attack[0].values())]
data_frame_lcc_edge.to_csv("../results/lcc_edge_values.csv")
##############################################################################################

data_frame_efficiency_edge = pd.DataFrame(columns=['network','random','BC','greedy'])
data_frame_efficiency_edge['network']=net_names[:len(net_names)-1]

data_frame_efficiency_edge['random']=[list(random_bn_macaque_rhesus_brain_2_edge_attack[1].values()), list(random_word_adjacencies_edge_attack[1].values()), list(random_circuits_s208_edge_attack[1].values()), list(random_bn_cat_mixed_species_brain_1_edge_attack[1].values()), list(random_facebook_414_edge_attack[1].values()), list(random_facebook_686_edge_attack[1].values()), list(random_polbooks_edge_attack[1].values()), list(random_jazz_edge_attack[1].values()), list(random_facebook_348_edge_attack[1].values()), list(random_circuits_s420_edge_attack[1].values()), list(random_corruption_edge_attack[1].values()), list(random_facebook_0_edge_attack[1].values()), list(random_circuits_s838_edge_attack[1].values()), list(random_crime_net_edge_attack[1].values())]
data_frame_efficiency_edge['BC']=[list(BC_bn_macaque_rhesus_brain_2_edge_attack[1].values()), list(BC_word_adjacencies_edge_attack[1].values()), list(BC_circuits_s208_edge_attack[1].values()), list(BC_bn_cat_mixed_species_brain_1_edge_attack[1].values()), list(BC_facebook_414_edge_attack[1].values()), list(BC_facebook_686_edge_attack[1].values()), list(BC_polbooks_edge_attack[1].values()), list(BC_jazz_edge_attack[1].values()), list(BC_facebook_348_edge_attack[1].values()), list(BC_circuits_s420_edge_attack[1].values()), list(BC_corruption_edge_attack[1].values()), list(BC_facebook_0_edge_attack[1].values()), list(BC_circuits_s838_edge_attack[1].values()), list(BC_crime_net_edge_attack[1].values())]
# data_frame_efficiency_edge['community_BC']=[list(community_BC_bn_macaque_rhesus_brain_2_edge_attack[1].values()), list(community_BC_word_adjacencies_edge_attack[1].values()), list(community_BC_circuits_s208_edge_attack[1].values()), list(community_BC_bn_cat_mixed_species_brain_1_edge_attack[1].values()), list(community_BC_facebook_414_edge_attack[1].values()), list(community_BC_facebook_686_edge_attack[1].values()), list(community_BC_polbooks_edge_attack[1].values()), list(community_BC_jazz_edge_attack[1].values()), list(community_BC_facebook_348_edge_attack[1].values()), list(community_BC_circuits_s420_edge_attack[1].values()), list(community_BC_corruption_edge_attack[1].values()), list(community_BC_facebook_0_edge_attack[1].values()), list(community_BC_circuits_s838_edge_attack[1].values()), list(community_BC_crime_net_edge_attack[1].values())]
data_frame_efficiency_edge['greedy']=[list(greedy_bn_macaque_rhesus_brain_2_edge_attack[1].values()), list(greedy_word_adjacencies_edge_attack[1].values()), list(greedy_circuits_s208_edge_attack[1].values()), list(greedy_bn_cat_mixed_species_brain_1_edge_attack[1].values()), list(greedy_facebook_414_edge_attack[1].values()), list(greedy_facebook_686_edge_attack[1].values()), list(greedy_polbooks_edge_attack[1].values()), list(greedy_jazz_edge_attack[1].values()), list(greedy_facebook_348_edge_attack[1].values()), list(greedy_circuits_s420_edge_attack[1].values()), list(greedy_corruption_edge_attack[1].values()), list(greedy_facebook_0_edge_attack[1].values()), list(greedy_circuits_s838_edge_attack[1].values()), list(greedy_crime_net_edge_attack[1].values())]
data_frame_efficiency_edge.to_csv("../results/efficiency_edge_values.csv")
##############################################################################################

data_frame_lcc_seq_edge = pd.DataFrame(columns=['network','random','BC','greedy'])
data_frame_lcc_seq_edge['network']=net_names[:len(net_names)-1]

data_frame_lcc_seq_edge['random']=[list(random_bn_macaque_rhesus_brain_2_edge_attack[2].values()), list(random_word_adjacencies_edge_attack[2].values()), list(random_circuits_s208_edge_attack[2].values()), list(random_bn_cat_mixed_species_brain_1_edge_attack[2].values()), list(random_facebook_414_edge_attack[2].values()), list(random_facebook_686_edge_attack[2].values()), list(random_polbooks_edge_attack[2].values()), list(random_jazz_edge_attack[2].values()), list(random_facebook_348_edge_attack[2].values()), list(random_circuits_s420_edge_attack[2].values()), list(random_corruption_edge_attack[2].values()), list(random_facebook_0_edge_attack[2].values()), list(random_circuits_s838_edge_attack[2].values()), list(random_crime_net_edge_attack[2].values())]
data_frame_lcc_seq_edge['BC']=[list(BC_bn_macaque_rhesus_brain_2_edge_attack[2].values()), list(BC_word_adjacencies_edge_attack[2].values()), list(BC_circuits_s208_edge_attack[2].values()), list(BC_bn_cat_mixed_species_brain_1_edge_attack[2].values()), list(BC_facebook_414_edge_attack[2].values()), list(BC_facebook_686_edge_attack[2].values()), list(BC_polbooks_edge_attack[2].values()), list(BC_jazz_edge_attack[2].values()), list(BC_facebook_348_edge_attack[2].values()), list(BC_circuits_s420_edge_attack[2].values()), list(BC_corruption_edge_attack[2].values()), list(BC_facebook_0_edge_attack[2].values()), list(BC_circuits_s838_edge_attack[2].values()), list(BC_crime_net_edge_attack[2].values())]
# data_frame_lcc_seq_edge['community_BC']=[list(community_BC_bn_macaque_rhesus_brain_2_edge_attack[2].values()), list(community_BC_word_adjacencies_edge_attack[2].values()), list(community_BC_circuits_s208_edge_attack[2].values()), list(community_BC_bn_cat_mixed_species_brain_1_edge_attack[2].values()), list(community_BC_facebook_414_edge_attack[2].values()), list(community_BC_facebook_686_edge_attack[2].values()), list(community_BC_polbooks_edge_attack[2].values()), list(community_BC_jazz_edge_attack[2].values()), list(community_BC_facebook_348_edge_attack[2].values()), list(community_BC_circuits_s420_edge_attack[2].values()), list(community_BC_corruption_edge_attack[2].values()), list(community_BC_facebook_0_edge_attack[2].values()), list(community_BC_circuits_s838_edge_attack[2].values()), list(community_BC_crime_net_edge_attack[2].values())]
data_frame_lcc_seq_edge['greedy']=[list(greedy_bn_macaque_rhesus_brain_2_edge_attack[2].values()), list(greedy_word_adjacencies_edge_attack[2].values()), list(greedy_circuits_s208_edge_attack[2].values()), list(greedy_bn_cat_mixed_species_brain_1_edge_attack[2].values()), list(greedy_facebook_414_edge_attack[2].values()), list(greedy_facebook_686_edge_attack[2].values()), list(greedy_polbooks_edge_attack[2].values()), list(greedy_jazz_edge_attack[2].values()), list(greedy_facebook_348_edge_attack[2].values()), list(greedy_circuits_s420_edge_attack[2].values()), list(greedy_corruption_edge_attack[2].values()), list(greedy_facebook_0_edge_attack[2].values()), list(greedy_circuits_s838_edge_attack[2].values()), list(greedy_crime_net_edge_attack[2].values())]
data_frame_lcc_seq_edge.to_csv("../results/lcc_seq_edge_values.csv")
##############################################################################################

data_frame_efficiency_seq_edge = pd.DataFrame(columns=['network','random','BC','greedy'])
data_frame_efficiency_seq_edge['network']=net_names[:len(net_names)-1]

data_frame_efficiency_seq_edge['random']=[list(random_bn_macaque_rhesus_brain_2_edge_attack[3].values()), list(random_word_adjacencies_edge_attack[3].values()), list(random_circuits_s208_edge_attack[3].values()), list(random_bn_cat_mixed_species_brain_1_edge_attack[3].values()), list(random_facebook_414_edge_attack[3].values()), list(random_facebook_686_edge_attack[3].values()), list(random_polbooks_edge_attack[3].values()), list(random_jazz_edge_attack[3].values()), list(random_facebook_348_edge_attack[3].values()), list(random_circuits_s420_edge_attack[3].values()), list(random_corruption_edge_attack[3].values()), list(random_facebook_0_edge_attack[3].values()), list(random_circuits_s838_edge_attack[3].values()), list(random_crime_net_edge_attack[3].values())]
data_frame_efficiency_seq_edge['BC']=[list(BC_bn_macaque_rhesus_brain_2_edge_attack[3].values()), list(BC_word_adjacencies_edge_attack[3].values()), list(BC_circuits_s208_edge_attack[3].values()), list(BC_bn_cat_mixed_species_brain_1_edge_attack[3].values()), list(BC_facebook_414_edge_attack[3].values()), list(BC_facebook_686_edge_attack[3].values()), list(BC_polbooks_edge_attack[3].values()), list(BC_jazz_edge_attack[3].values()), list(BC_facebook_348_edge_attack[3].values()), list(BC_circuits_s420_edge_attack[3].values()), list(BC_corruption_edge_attack[3].values()), list(BC_facebook_0_edge_attack[3].values()), list(BC_circuits_s838_edge_attack[3].values()), list(BC_crime_net_edge_attack[3].values())]
# data_fame_efficiency_seq_edge['community_BC']=[list(community_BC_bn_macaque_rhesus_brain_2_edge_attack[3].values()), list(community_BC_word_adjacencies_edge_attack[3].values()), list(community_BC_circuits_s208_edge_attack[3].values()), list(community_BC_bn_cat_mixed_species_brain_1_edge_attack[3].values()), list(community_BC_facebook_414_edge_attack[3].values()), list(community_BC_facebook_686_edge_attack[3].values()), list(community_BC_polbooks_edge_attack[3].values()), list(community_BC_jazz_edge_attack[3].values()), list(community_BC_facebook_348_edge_attack[3].values()), list(community_BC_circuits_s420_edge_attack[3].values()), list(community_BC_corruption_edge_attack[3].values()), list(community_BC_facebook_0_edge_attack[3].values()), list(community_BC_circuits_s838_edge_attack[3].values()), list(community_BC_crime_net_edge_attack[3].values())]
data_frame_efficiency_seq_edge['greedy']=[list(greedy_bn_macaque_rhesus_brain_2_edge_attack[3].values()), list(greedy_word_adjacencies_edge_attack[3].values()), list(greedy_circuits_s208_edge_attack[3].values()), list(greedy_bn_cat_mixed_species_brain_1_edge_attack[3].values()), list(greedy_facebook_414_edge_attack[3].values()), list(greedy_facebook_686_edge_attack[3].values()), list(greedy_polbooks_edge_attack[3].values()), list(greedy_jazz_edge_attack[3].values()), list(greedy_facebook_348_edge_attack[3].values()), list(greedy_circuits_s420_edge_attack[3].values()), list(greedy_corruption_edge_attack[3].values()), list(greedy_facebook_0_edge_attack[3].values()), list(greedy_circuits_s838_edge_attack[3].values()), list(greedy_crime_net_edge_attack[3].values())]
data_frame_efficiency_seq_edge.to_csv("../results/efficiency_seq_edge_values.csv")
##############################################################################################

data_frame_R_edge = pd.DataFrame(columns=['network','random','BC','greedy'])
data_frame_R_edge['network']=net_names[:len(net_names)-1]

data_frame_R_edge['random']=[random_bn_macaque_rhesus_brain_2_edge_attack[4], random_word_adjacencies_edge_attack[4], random_circuits_s208_edge_attack[4], random_bn_cat_mixed_species_brain_1_edge_attack[4], random_facebook_414_edge_attack[4], random_facebook_686_edge_attack[4], random_polbooks_edge_attack[4], random_jazz_edge_attack[4], random_facebook_348_edge_attack[4], random_circuits_s420_edge_attack[4], random_corruption_edge_attack[4], random_facebook_0_edge_attack[4], random_circuits_s838_edge_attack[4], random_crime_net_edge_attack[4]]
data_frame_R_edge['BC']=[BC_bn_macaque_rhesus_brain_2_edge_attack[4], BC_word_adjacencies_edge_attack[4], BC_circuits_s208_edge_attack[4], BC_bn_cat_mixed_species_brain_1_edge_attack[4], BC_facebook_414_edge_attack[4], BC_facebook_686_edge_attack[4], BC_polbooks_edge_attack[4], BC_jazz_edge_attack[4], BC_facebook_348_edge_attack[4], BC_circuits_s420_edge_attack[4], BC_corruption_edge_attack[4], BC_facebook_0_edge_attack[4], BC_circuits_s838_edge_attack[4], BC_crime_net_edge_attack[4]]
# data_frame_R_edge['community_BC']=[community_BC_bn_macaque_rhesus_brain_2_edge_attack[4], community_BC_word_adjacencies_edge_attack[4], community_BC_circuits_s208_edge_attack[4], community_BC_bn_cat_mixed_species_brain_1_edge_attack[4], community_BC_facebook_414_edge_attack[4], community_BC_facebook_686_edge_attack[4], community_BC_polbooks_edge_attack[4], community_BC_jazz_edge_attack[4], community_BC_facebook_348_edge_attack[4], community_BC_circuits_s420_edge_attack[4], community_BC_corruption_edge_attack[4], community_BC_facebook_0_edge_attack[4], community_BC_circuits_s838_edge_attack[4], community_BC_crime_net_edge_attack[4]]
data_frame_R_edge['greedy']=[greedy_bn_macaque_rhesus_brain_2_edge_attack[4], greedy_word_adjacencies_edge_attack[4], greedy_circuits_s208_edge_attack[4], greedy_bn_cat_mixed_species_brain_1_edge_attack[4], greedy_facebook_414_edge_attack[4], greedy_facebook_686_edge_attack[4], greedy_polbooks_edge_attack[4], greedy_jazz_edge_attack[4], greedy_facebook_348_edge_attack[4], greedy_circuits_s420_edge_attack[4], greedy_corruption_edge_attack[4], greedy_facebook_0_edge_attack[4], greedy_circuits_s838_edge_attack[4], greedy_crime_net_edge_attack[4]]
data_frame_R_edge.to_csv("../results/R_edge_values.csv")
##############################################################################################

data_frame_InterE_edge = pd.DataFrame(columns=['network','random','BC','greedy'])
data_frame_InterE_edge['network']=net_names[:len(net_names)-1]

data_frame_InterE_edge['random']=[random_bn_macaque_rhesus_brain_2_edge_attack[5], random_word_adjacencies_edge_attack[5], random_circuits_s208_edge_attack[5], random_bn_cat_mixed_species_brain_1_edge_attack[5], random_facebook_414_edge_attack[5], random_facebook_686_edge_attack[5], random_polbooks_edge_attack[5], random_jazz_edge_attack[5], random_facebook_348_edge_attack[5], random_circuits_s420_edge_attack[5], random_corruption_edge_attack[5], random_facebook_0_edge_attack[5], random_circuits_s838_edge_attack[5], random_crime_net_edge_attack[5]]
data_frame_InterE_edge['BC']=[BC_bn_macaque_rhesus_brain_2_edge_attack[5], BC_word_adjacencies_edge_attack[5], BC_circuits_s208_edge_attack[5], BC_bn_cat_mixed_species_brain_1_edge_attack[5], BC_facebook_414_edge_attack[5], BC_facebook_686_edge_attack[5], BC_polbooks_edge_attack[5], BC_jazz_edge_attack[5], BC_facebook_348_edge_attack[5], BC_circuits_s420_edge_attack[5], BC_corruption_edge_attack[5], BC_facebook_0_edge_attack[5], BC_circuits_s838_edge_attack[5], BC_crime_net_edge_attack[5]]
# data_frame_InterE_edge['community_BC']=[community_BC_bn_macaque_rhesus_brain_2_edge_attack[5], community_BC_word_adjacencies_edge_attack[5], community_BC_circuits_s208_edge_attack[5], community_BC_bn_cat_mixed_species_brain_1_edge_attack[5], community_BC_facebook_414_edge_attack[5], community_BC_facebook_686_edge_attack[5], community_BC_polbooks_edge_attack[5], community_BC_jazz_edge_attack[5], community_BC_facebook_348_edge_attack[5], community_BC_circuits_s420_edge_attack[5], community_BC_corruption_edge_attack[5], community_BC_facebook_0_edge_attack[5], community_BC_circuits_s838_edge_attack[5], community_BC_crime_net_edge_attack[5]]
data_frame_InterE_edge['greedy']=[greedy_bn_macaque_rhesus_brain_2_edge_attack[5], greedy_word_adjacencies_edge_attack[5], greedy_circuits_s208_edge_attack[5], greedy_bn_cat_mixed_species_brain_1_edge_attack[5], greedy_facebook_414_edge_attack[5], greedy_facebook_686_edge_attack[5], greedy_polbooks_edge_attack[5], greedy_jazz_edge_attack[5], greedy_facebook_348_edge_attack[5], greedy_circuits_s420_edge_attack[5], greedy_corruption_edge_attack[5], greedy_facebook_0_edge_attack[5], greedy_circuits_s838_edge_attack[5], greedy_crime_net_edge_attack[5]]
data_frame_InterE_edge.to_csv("../results/InterE_edge_values.csv")
##############################################################################################

data_frame_time_edge = pd.DataFrame(columns=['network','random','BC','greedy'])
data_frame_time_edge['network']=net_names[:len(net_names)-1]

data_frame_time_edge['random']=[random_bn_macaque_rhesus_brain_2_edge_attack[6], random_word_adjacencies_edge_attack[6], random_circuits_s208_edge_attack[6], random_bn_cat_mixed_species_brain_1_edge_attack[6], random_facebook_414_edge_attack[6], random_facebook_686_edge_attack[6], random_polbooks_edge_attack[6], random_jazz_edge_attack[6], random_facebook_348_edge_attack[6], random_circuits_s420_edge_attack[6], random_corruption_edge_attack[6], random_facebook_0_edge_attack[6], random_circuits_s838_edge_attack[6], random_crime_net_edge_attack[6]]
data_frame_time_edge['BC']=[BC_bn_macaque_rhesus_brain_2_edge_attack[6], BC_word_adjacencies_edge_attack[6], BC_circuits_s208_edge_attack[6], BC_bn_cat_mixed_species_brain_1_edge_attack[6], BC_facebook_414_edge_attack[6], BC_facebook_686_edge_attack[6], BC_polbooks_edge_attack[6], BC_jazz_edge_attack[6], BC_facebook_348_edge_attack[6], BC_circuits_s420_edge_attack[6], BC_corruption_edge_attack[6], BC_facebook_0_edge_attack[6], BC_circuits_s838_edge_attack[6], BC_crime_net_edge_attack[6]]
# data_frame_time_edge['community_BC']=[community_BC_bn_macaque_rhesus_brain_2_edge_attack[6], community_BC_word_adjacencies_edge_attack[6], community_BC_circuits_s208_edge_attack[6], community_BC_bn_cat_mixed_species_brain_1_edge_attack[6], community_BC_facebook_414_edge_attack[6], community_BC_facebook_686_edge_attack[6], community_BC_polbooks_edge_attack[6], community_BC_jazz_edge_attack[6], community_BC_facebook_348_edge_attack[6], community_BC_circuits_s420_edge_attack[6], community_BC_corruption_edge_attack[6], community_BC_facebook_0_edge_attack[6], community_BC_circuits_s838_edge_attack[6], community_BC_crime_net_edge_attack[6]]
data_frame_time_edge['greedy']=[greedy_bn_macaque_rhesus_brain_2_edge_attack[6], greedy_word_adjacencies_edge_attack[6], greedy_circuits_s208_edge_attack[6], greedy_bn_cat_mixed_species_brain_1_edge_attack[6], greedy_facebook_414_edge_attack[6], greedy_facebook_686_edge_attack[6], greedy_polbooks_edge_attack[6], greedy_jazz_edge_attack[6], greedy_facebook_348_edge_attack[6], greedy_circuits_s420_edge_attack[6], greedy_corruption_edge_attack[6], greedy_facebook_0_edge_attack[6], greedy_circuits_s838_edge_attack[6], greedy_crime_net_edge_attack[6]]
data_frame_time_edge.to_csv("../results/time_edge_values.csv")

#### Graphs:

Saving graphs at different time steps for viz. Graph saved is _corruption_

In [298]:
import os

#in each folder in ../graphs/nodes/ add a list of folder of names cols[1:]
#looping through the directory:
# dir = "../graphs/nodes/bn_cat_mixed_species_brain_1/"
# for i in range(10):
#     nx.write_gml(nx.read_gml(MED_facebook_1684_attack[7][i], dir+str(i)))

for i in range(10):
    nx.write_gml(MED_bn_cat_mixed_species_brain_1_attack[7][i], "../graphs/nodes/bn_cat_mixed_species_brain_1/"+str(i)+".gml")

In [61]:
#save it communnity partition (for visualization)
# community_partition_corruption

In [113]:
cor_attack = community_attack_node(corruption, community_partition_corruption, MEB)
G0= cor_attack[7][0] ; G1=cor_attack[7][1]; G2=cor_attack[7][2]; G3=cor_attack[7][3]; G4=cor_attack[7][4]; G5=cor_attack[7][5]; G6=cor_attack[7][6]; G7=cor_attack[7][7]; G8=cor_attack[7][8]; G9=cor_attack[7][9]
if not os.path.exists("../graphs/nodes/corruption/"):
    os.makedirs("../graphs/nodes/corruption/")
G=[G0,G1,G2,G3,G4,G5,G6,G7,G8,G9]
for i in range(10):
    nx.write_gml(G[i], "../graphs/nodes/corruption/"+str(i)+".gml")